In [23]:
from dotenv import load_dotenv
load_dotenv()

import re
from time import sleep
from json import dump
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from openai import OpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pydantic import BaseModel
from typing import Optional
from enum import Enum

from utils.storage import list_processed_mmd_files, download_processed_mmd_file, download_file_metadata

In [2]:
client = OpenAI()

In [3]:
splitter = RecursiveCharacterTextSplitter(
	chunk_size=35000,
	chunk_overlap=500,
	length_function=len,
	is_separator_regex=False,
)

In [4]:
files = list_processed_mmd_files()
len(files)

73345

In [5]:
with ThreadPoolExecutor() as executor:
	mds = list(tqdm(executor.map(download_processed_mmd_file, files), total=len(files)))

100%|██████████| 73345/73345 [11:56<00:00, 102.31it/s] 


In [6]:
with ThreadPoolExecutor() as executor:
	metadata = list(tqdm(executor.map(download_file_metadata, files), total=len(files)))

100%|██████████| 73345/73345 [06:18<00:00, 194.01it/s]


In [12]:
def prepare_samples(limit=10000):
	samples = []
	filenames = []

	for filename, md, meta in tqdm(zip(files, mds, metadata), total=len(files)):
		if len(meta["results"]) == 0:
			continue

		clean = re.sub(r"(?<=\w)-\n(?=\w)", "", md)
		clean = re.sub(r"(?<=\w)\n(?=\w)", " ", clean)

		chunks = splitter.split_text(clean)

		for result in meta["results"]:
			if result["value"] not in md or result["metric"] not in md:
				continue

			for chunk in chunks:
				if result["value"] in chunk and result["metric"] in chunk:
					if chunk in samples:
						continue

					samples.append(chunk)
					filenames.append(filename)

					if len(samples) == limit:
						return list(zip(filenames, samples))

	return list(zip(filenames, samples))

samples = prepare_samples()

100%|██████████| 73345/73345 [00:26<00:00, 2734.71it/s]


In [8]:
class Task(str, Enum):
	newproductsalesforecasting = "new product sales forecasting"
	weaklysupervisedclassification = "weakly supervised classification"
	fingertipdetection = "fingertip detection"
	stressstrainrelation = "stress-strain relation"
	tgifaction = "tgif-action"
	thoracicdiseaseclassification = "thoracic disease classification"
	mutualinformationestimation = "mutual information estimation"
	surgicalphaserecognition = "surgical phase recognition"
	biomedicalinformationretrieval = "biomedical information retrieval"
	humordetection = "humor detection"
	heartbeatclassification = "heartbeat classification"
	visualcommonsensereasoning = "visual commonsense reasoning"
	headlinegeneration = "headline generation"
	dialoguerewriting = "dialogue rewriting"
	inversetonemapping = "inverse tone mapping"
	scientificresultsextraction = "scientific results extraction"
	texttocodegeneration = "text-to-code generation"
	selfsupervisedimageclassification = "self-supervised image classification"
	skinlesionidentification = "skin lesion identification"
	vietnameseimagecaptioning = "vietnamese image captioning"
	decoder = "decoder"
	initialstructuretorelaxedenergyis2redirect = "initial structure to relaxed energy (is2re) direct"
	d3denseshapecorrespondence = "3d dense shape correspondence"
	memeclassification = "meme classification"
	cropclassification = "crop classification"
	multidomainsentimentclassification = "multi-domain sentiment classification"
	taskarithmetic = "task arithmetic"
	videoediting = "video editing"
	variabledetection = "variable detection"
	insulatordefectdetection = "insulator defect detection"
	prosodyprediction = "prosody prediction"
	speechsynthesismarathi = "speech synthesis - marathi"
	retrievalaugmentedgeneration = "retrieval-augmented generation"
	pointcloudsegmentation = "point cloud segmentation"
	instrumentplayingtechniquedetection = "instrument playing technique detection"
	scheduling = "scheduling"
	referringexpressionsegmentation = "referring expression segmentation"
	generalizedzeroshotskeletalactionrecognition = "generalized zero shot skeletal action recognition"
	electrontomography = "electron tomography"
	extrememultilabelclassification = "extreme multi-label classification"
	triplet = "triplet"
	stocktrendprediction = "stock trend prediction"
	reasoningchainexplanations = "reasoning chain explanations"
	conversationalsentimentquadrupleextraction = "conversational sentiment quadruple extraction"
	spectrumcartography = "spectrum cartography"
	continuallearning = "continual learning"
	assortmentoptimization = "assortment optimization"
	groundedlanguagelearning = "grounded language learning"
	claimevidencepairextractioncepe = "claim-evidence pair extraction (cepe)"
	firedetection = "fire detection"
	oneshotpartsegmentationofgraspaffordanceintraclass = "one-shot part segmentation of grasp affordance - intra class"
	facereconstruction = "face reconstruction"
	shortquestionanswers = "short question answers"
	zeroshotscenegraphgeneration = "zero-shot scene graph generation"
	musictranscription = "music transcription"
	oneshotunsuperviseddomainadaptation = "one-shot unsupervised domain adaptation"
	federatedlearningvideosuperresolution = "federated learning (video super-resolution)"
	promptdrivenzeroshotdomainadaptation = "prompt-driven zero-shot domain adaptation"
	prostatezonessegmentation = "prostate zones segmentation"
	d3humanposeestimation = "3d human pose estimation"
	textlineextraction = "text-line extraction"
	trainingfree3dpointcloudclassification = "training-free 3d point cloud classification"
	imagesuperresolution = "image super-resolution"
	adroidrelocatecloned = "adroid relocate-cloned"
	neuralarchitecturesearch = "neural architecture search"
	coreferenceresolution = "coreference resolution"
	videocorpusmomentretrieval = "video corpus moment retrieval"
	overlapped101 = "overlapped 10-1"
	naturallanguagequeries = "natural language queries"
	documentlevelrewithincompletelabeling = "document-level re with incomplete labeling"
	blocking = "blocking"
	robotposeestimation = "robot pose estimation"
	zeroshotvisualquestionanswring = "zero-shot visual question answring"
	enumerativesearch = "enumerative search"
	dealiasing = "de-aliasing"
	graphclustering = "graph clustering"
	unsupervisedtextstyletransfer = "unsupervised text style transfer"
	fewshotrelationclassification = "few-shot relation classification"
	fadcurveanalysis = "fad curve analysis"
	goalorienteddialoguesystems = "goal-oriented dialogue systems"
	surfacenormalsestimation = "surface normals estimation"
	dynamicfacialexpressionrecognition = "dynamic facial expression recognition"
	timeoffsetcalibration = "time offset calibration"
	evaluatinginformationessentiality = "evaluating information essentiality"
	entitylinking = "entity linking"
	d3featurematching = "3d feature matching"
	semantictextmatching = "semantic text matching"
	informativeness = "informativeness"
	machineunlearning = "machine unlearning"
	atomicforces = "atomic forces"
	onestageanchorfreeorientedobjectdetection = "one-stage anchor-free oriented object detection"
	paraphrasegeneration = "paraphrase generation"
	professionalpsychology = "professional psychology"
	arzenspeechrecognition = "arzen speech recognition"
	crowdsourcedtextaggregation = "crowdsourced text aggregation"
	objectreconstruction = "object reconstruction"
	novelclassdiscovery = "novel class discovery"
	continualfewshotlearning = "continual few-shot learning"
	clozemultichoicesoneshot = "cloze (multi-choices) (one-shot)"
	multiviewdetection = "multi-view detection"
	imagemorphing = "image morphing"
	llmjailbreak = "llm jailbreak"
	extractivedocumentsummarization = "extractive document summarization"
	imageforensics = "image forensics"
	noreferenceimagequalityassessment = "no-reference image quality assessment"
	domainclassification = "domain classification"
	newsauthenticityidentification = "news authenticity identification"
	activityrecognition = "activity recognition"
	clozemultichoicesfewshot = "cloze (multi-choices) (few-shot)"
	crossmodalplacerecognition = "cross-modal place recognition"
	attentivesegmentationnetworks = "attentive segmentation networks"
	logicalreasoningquestionanswering = "logical reasoning question answering"
	readingcomprehensiononeshot = "reading comprehension (one-shot)"
	newsgeneration = "news generation"
	gallbladdercancerdetection = "gallbladder cancer detection"
	finegrainedfacialediting = "fine-grained facial editing"
	seizureprediction = "seizure prediction"
	decompensation = "decompensation"
	d3humanshapeestimation = "3d human shape estimation"
	valenceestimation = "valence estimation"
	arbitraryconditionaldensityestimation = "arbitrary conditional density estimation"
	affordancedetection = "affordance detection"
	belebele = "belebele"
	nodeclassification = "node classification"
	jointentityandrelationextractiononscientificdata = "joint entity and relation extraction on scientific data"
	personadialogueinstory = "persona dialogue in story"
	d3roomlayoutsfromasinglergbpanorama = "3d room layouts from a single rgb panorama"
	conversationalrecommendation = "conversational recommendation"
	oneshotpartsegmentationofpoundaffordanceintraclass = "one-shot part segmentation of pound affordance - intra class"
	predictionofcancercelllinesensitivity = "prediction of cancer cell line sensitivity"
	zeroshotcrosslingualvisualquestionanswering = "zero-shot cross-lingual visual question answering"
	naturallanguageinferencezeroshot = "natural language inference (zero-shot)"
	pedestriandetection = "pedestrian detection"
	multisourceunsuperviseddomainadaptation = "multi-source unsupervised domain adaptation"
	facialexpressiontranslation = "facial expression translation"
	semisupervisedinstancesegmentation = "semi-supervised instance segmentation"
	realtimevisualtracking = "real-time visual tracking"
	textaugmentation = "text augmentation"
	setting14 = "setting-1/4"
	thesemanticsegmentationofremotesensingimagery = "the semantic segmentation of remote sensing imagery"
	eventcoreferenceresolution = "event coreference resolution"
	hatefulmemeclassification = "hateful meme classification"
	floodextentforecasting = "flood extent forecasting"
	actionunderstanding = "action understanding"
	stereotypicalbiasanalysis = "stereotypical bias analysis"
	openvocabularyobjectcounting = "open-vocabulary object counting"
	facerecognition = "face recognition"
	actionspotting = "action spotting"
	tweetretrieval = "tweet retrieval"
	unsupervised3dpointcloudlinearevaluation = "unsupervised 3d point cloud linear evaluation"
	learningtorank = "learning-to-rank"
	textguidedgeneration = "text-guided-generation"
	omniverseisaacgym = "omniverse isaac gym"
	facequalityassessement = "face quality assessement"
	rawreconstruction = "raw reconstruction"
	facetofacetranslation = "face to face translation"
	d3semanticscenecompletionfromasinglergbimage = "3d semantic scene completion from a single rgb image"
	reinforcementlearningrl = "reinforcement learning (rl)"
	homographyestimation = "homography estimation"
	wordlevelpronunciationscoring = "word-level pronunciation scoring"
	oneshotinstancesegmentation = "one-shot instance segmentation"
	multimodalclassification = "multi-modal classification"
	machinereadingcomprehension = "machine reading comprehension"
	visualrelationshipdetection = "visual relationship detection"
	objectsegmentation = "object segmentation"
	intelligentcommunication = "intelligent communication"
	eventexpansion = "event expansion"
	propagandaspanidentification = "propaganda span identification"
	uncropping = "uncropping"
	malwaretypedetection = "malware type detection"
	facedetection = "face detection"
	multiobjecttrackingandsegmentation = "multi-object tracking and segmentation"
	multiplechoice = "multiple-choice"
	timeseriesforecasting = "time series forecasting"
	communityquestionanswering = "community question answering"
	hyperparameteroptimization = "hyperparameter optimization"
	readingcomprehension = "reading comprehension"
	codecompletion = "code completion"
	multidomaindialoguestatetracking = "multi-domain dialogue state tracking"
	hurricaneforecasting = "hurricane forecasting"
	semanticcompression = "semantic compression"
	d3multipersonposeestimationrootrelative = "3d multi-person pose estimation (root-relative)"
	fewshotcameraadaptivecolorconstancy = "few-shot camera-adaptive color constancy"
	actiontripletrecognition = "action triplet recognition"
	temporaltagging = "temporal tagging"
	openvocabularyactionrecognition = "open vocabulary action recognition"
	finegrainedimageinpainting = "fine-grained image inpainting"
	aspectextraction = "aspect extraction"
	multimodalactivityrecognition = "multimodal activity recognition"
	propagandatechniqueidentification = "propaganda technique identification"
	multipleinstancelearning = "multiple instance learning"
	queryfocusedsummarization = "query-focused summarization"
	selforganizedclustering = "self-organized clustering"
	genderclassification = "gender classification"
	jointmultilingualsentencerepresentations = "joint multilingual sentence representations"
	weaklysupervisedobjectsegmentation = "weakly-supervised object segmentation"
	counterfactual = "counterfactual"
	graphpartitioning = "graph partitioning"
	testagnosticlongtailedlearning = "test agnostic long-tailed learning"
	documentclassification = "document classification"
	featureengineering = "feature engineering"
	videomme = "video mme"
	semisupervisedanomalydetection = "semi-supervised anomaly detection"
	energymanagement = "energy management"
	hyperbaton = "hyperbaton"
	openvocabularypanopticsegmentation = "open vocabulary panoptic segmentation"
	multilingualwordembeddings = "multilingual word embeddings"
	epigeneticmarksprediction = "epigenetic marks prediction"
	textretrieval = "text retrieval"
	headposeestimation = "head pose estimation"
	attractivenessestimation = "attractiveness estimation"
	repetitiveactioncounting = "repetitive action counting"
	audioemotionrecognition = "audio emotion recognition"
	sociology = "sociology"
	imagematching = "image matching"
	sentencesummarization = "sentence summarization"
	mathinformationretrieval = "math information retrieval"
	orientedobjectdetection = "oriented object detection"
	apisequencerecommendation = "api sequence recommendation"
	answerabilityprediction = "answerability prediction"
	gpr = "gpr"
	commonsensecausalreasoning = "commonsense causal reasoning"
	geometricmatching = "geometric matching"
	relationmapping = "relation mapping"
	pointclouds = "point clouds"
	multimodalintentrecognition = "multimodal intent recognition"
	phototorestgeneralization = "photo to rest generalization"
	artificialglobalworkspace = "artificial global workspace"
	spectralreconstruction = "spectral reconstruction"
	mentalstressdetection = "mental stress detection"
	fingerveinrecognition = "finger vein recognition"
	singlesourcedomaingeneralization = "single-source domain generalization"
	densecontactestimation = "dense contact estimation"
	anomalysegmentation = "anomaly segmentation"
	agentbasedmodelinverseproblem = "agent-based model inverse problem"
	soundsourcelocalization = "sound source localization"
	facialattributeclassification = "facial attribute classification"
	automaticlyricstranscription = "automatic lyrics transcription"
	objectskeletondetection = "object skeleton detection"
	lesionsegmentation = "lesion segmentation"
	videodeinterlacing = "video deinterlacing"
	crossmodalinformationretrieval = "cross-modal information retrieval"
	videocompression = "video compression"
	specularsegmentation = "specular segmentation"
	d3poseestimation = "3d pose estimation"
	skinlesionsegmentation = "skin lesion segmentation"
	vcgbenchdiverse = "vcgbench-diverse"
	defocusblurdetection = "defocus blur detection"
	domainlabelling = "domain labelling"
	quantummachinelearning = "quantum machine learning"
	cardiacsegmentation = "cardiac segmentation"
	openaigym = "openai gym"
	speechintentclassification = "speech intent classification"
	chemicalentityrecognition = "chemical entity recognition"
	occluded3dobjectsymmetrydetection = "occluded 3d object symmetry detection"
	citationintentclassification = "citation intent classification"
	continualanomalydetection = "continual anomaly detection"
	facetransfer = "face transfer"
	audiovideoquestionansweringavqa = "audio-video question answering (avqa)"
	droneposeestimation = "drone pose estimation"
	rgb3danomalydetectionandsegmentation = "rgb+3d anomaly detection and segmentation"
	anomalyforecasting = "anomaly forecasting"
	algorithmictrading = "algorithmic trading"
	flue = "flue"
	adversarialrobustness = "adversarial robustness"
	federatedlifelongpersonreid = "federated lifelong person reid"
	situationrecognition = "situation recognition"
	hallucinationpairwisedetection1ref = "hallucination pair-wise detection (1-ref)"
	speechtokenization = "speech tokenization"
	handgesturesegmentation = "hand gesture segmentation"
	survivalprediction = "survival prediction"
	storygeneration = "story generation"
	ary4relationextraction = "4-ary relation extraction"
	kcomplexdetection = "k-complex detection"
	imagetogpsverification = "image-to-gps verification"
	fewshotactionrecognition = "few-shot action recognition"
	hatespeechdetection = "hate speech detection"
	d3shapegeneration = "3d shape generation"
	normalisingflows = "normalising flows"
	isac = "isac"
	flareremoval = "flare removal"
	incrementalconstrainedclustering = "incremental constrained clustering"
	plan2scene = "plan2scene"
	newsclassification = "news classification"
	alzheimersdetection = "alzheimer's detection"
	tabledetection = "table detection"
	evidenceselection = "evidence selection"
	sentencecompression = "sentence compression"
	specularreflectionmitigation = "specular reflection mitigation"
	lidarsemanticsegmentation = "lidar semantic segmentation"
	proteinligandaffinityprediction = "protein-ligand affinity prediction"
	rag = "rag"
	timeseriesalignment = "time series alignment"
	faultlocalization = "fault localization"
	aestheticsqualityassessment = "aesthetics quality assessment"
	gameofchess = "game of chess"
	imageforgerydetection = "image forgery detection"
	simplesemanticaidedfewshotlearning = "simple semantic-aided few-shot learning"
	dynamictimewarping = "dynamic time warping"
	sceneclassificationunifiedclasses = "scene classification (unified classes)"
	scenetextrecognition = "scene text recognition"
	photoretouching = "photo retouching"
	multitasklanguageunderstanding = "multi-task language understanding"
	d3volumetricreconstruction = "3d volumetric reconstruction"
	respiratoryfailure = "respiratory failure"
	edittask = "edit task"
	d3binpacking = "3d bin packing"
	adversarialdefense = "adversarial defense"
	tabulardatageneration = "tabular data generation"
	sentenceembedding = "sentence embedding"
	displacedpeoplerecognition = "displaced people recognition"
	extractaspect = "extract aspect"
	occlusionestimation = "occlusion estimation"
	nestedtermrecognition = "nested term recognition"
	unsupervisedimagedecomposition = "unsupervised image decomposition"
	unsupervisedpartofspeechtagging = "unsupervised part-of-speech tagging"
	additivemodels = "additive models"
	oneshotpartsegmentationofcutaffordanceintraclass = "one-shot part segmentation of cut affordance - intra class"
	unsupervisedmonoculardepthestimation = "unsupervised monocular depth estimation"
	emshowersclusterization = "em showers clusterization"
	withinsessionmotorimageryallclasses = "within-session motor imagery (all classes)"
	cameraposeestimation = "camera pose estimation"
	remotesensingimageclassification = "remote sensing image classification"
	crosslingualsentimentclassification = "cross-lingual sentiment classification"
	multimodalentityalignment = "multi-modal entity alignment"
	humanfitting = "human fitting"
	informationretrieva = "information retrieva"
	ventricularfibrillationdetection = "ventricular fibrillation detection"
	sstod = "sstod"
	drugresponseprediction = "drug response prediction"
	collisionavoidance = "collision avoidance"
	tag = "tag"
	dynamicneuralnetworks = "dynamic neural networks"
	automaticpostediting = "automatic post-editing"
	productrelationclassification = "product relation classification"
	handwrittenlinesegmentation = "handwritten line segmentation"
	singingvoicesynthesis = "singing voice synthesis"
	sentimentanalysis = "sentiment analysis"
	sceneparsing = "scene parsing"
	imagedefocusdeblurring = "image defocus deblurring"
	longformquestionanswering = "long form question answering"
	complexqueryanswering = "complex query answering"
	textnormalization = "text normalization"
	stancedetectionuselection2020trump = "stance detection (us election 2020 - trump)"
	visualspeechrecognition = "visual speech recognition"
	depthandcameramotion = "depth and camera motion"
	analogicalquestions = "analogical questions"
	indoorlocalization = "indoor localization"
	videomatting = "video matting"
	missingelements = "missing elements"
	featurecompression = "feature compression"
	radiointerferometry = "radio interferometry"
	textureimageretrieval = "texture image retrieval"
	lowresourcenmt = "low resource nmt"
	eventdataclassification = "event data classification"
	multimodalmachinetranslation = "multimodal machine translation"
	keypointdetectionandimagematching = "keypoint detection and image matching"
	scanpathprediction = "scanpath prediction"
	qqp = "qqp"
	zeroshotstancedetection = "zero-shot stance detection"
	colorectalglandsegmentation = "colorectal gland segmentation:"
	hierarchicaltextsegmentation = "hierarchical text segmentation"
	virtualtryon = "virtual try-on"
	multimodalassociation = "multimodal association"
	malwarefamilydetection = "malware family detection"
	speechtofaciallandmark = "speech to facial landmark"
	ischemicstrokelesionsegmentation = "ischemic stroke lesion segmentation"
	semanticsegmentationonscannet = "semantic segmentation on scannet"
	morphologicaldisambiguation = "morphological disambiguation"
	crosslingualner = "cross-lingual ner"
	bench2drive = "bench2drive"
	fewshottextclassification = "few-shot text classification"
	multiframesuperresolution = "multi-frame super-resolution"
	occludedfacedetection = "occluded face detection"
	geometryawareprocessing = "geometry-aware processing"
	diffeomorphicmedicalimageregistration = "diffeomorphic medical image registration"
	hardwareawareneuralarchitecturesearch = "hardware aware neural architecture search"
	humanbehaviorforecasting = "human behavior forecasting"
	tablebasedfactverification = "table-based fact verification"
	multimodalgeneration = "multimodal generation"
	clickbaitdetection = "clickbait detection"
	layoutgeneration = "layout generation"
	retinaloctlayersegmentation = "retinal oct layer segmentation"
	singleparticleanalysis = "single particle analysis"
	datasummarization = "data summarization"
	kinshipfacegeneration = "kinship face generation"
	fewshotsemanticsegmentation = "few-shot semantic segmentation"
	dynamicgraphembedding = "dynamic graph embedding"
	conformalprediction = "conformal prediction"
	audiosignalprocessing = "audio signal processing"
	semisupervisedvideoobjectsegmentation = "semi-supervised video object segmentation"
	saxsregression = "saxs regression"
	imagedeclipping = "image declipping"
	cryogenicelectrontomography = "cryogenic electron tomography"
	hyperspectralimagesuperresolution = "hyperspectral image super-resolution"
	adversarialattackonvideoclassification = "adversarial attack on video classification"
	transferreinforcementlearning = "transfer reinforcement learning"
	fewshotpointcloudclassification = "few-shot point cloud classification"
	amrparsing = "amr parsing"
	adroidrelocatehuman = "adroid relocate-human"
	lifelike3dhumangeneration = "lifelike 3d human generation"
	multitaskaudiosourceseperation = "multi-task audio source seperation"
	semanticentitylabeling = "semantic entity labeling"
	global3dhumanposeestimation = "global 3d human pose estimation"
	vietnamesehatespeechdetection = "vietnamese hate speech detection"
	videoprediction = "video prediction"
	punctuationrestoration = "punctuation restoration"
	commonsensereasoningoneshot = "common sense reasoning (one-shot)"
	montezumasrevenge = "montezuma's revenge"
	continualpretraining = "continual pretraining"
	lowlightimagedeblurringandenhancement = "low-light image deblurring and enhancement"
	automatedtheoremproving = "automated theorem proving"
	antibodyantigenbindingprediction = "antibody-antigen binding prediction"
	pathfinder = "pathfinder"
	crosslingualfactchecking = "cross-lingual fact-checking"
	imageclassificationwithlabelnoise = "image classification with label noise"
	politicalinfluencedetection = "political influence detection"
	roomimpulseresponserir = "room impulse response (rir)"
	crosslingualtransfer = "cross-lingual transfer"
	tabletypedetection = "table type detection"
	parkinsondetectionfromspeech = "parkinson detection from speech"
	graphgeneration = "graph generation"
	zeroshotactionrecognitiononhmdb51 = "zero-shot action recognition on hmdb51"
	foggyscenesegmentation = "foggy scene segmentation"
	variationalinference = "variational inference"
	earlyclassification = "early classification"
	sleeparousaldetection = "sleep arousal detection"
	eegemotionrecognition = "eeg emotion recognition"
	visualprompting = "visual prompting"
	relationshipdetection = "relationship detection"
	imageguidedcomposition = "image-guided composition"
	chemicalreactionprediction = "chemical reaction prediction"
	artifactdetection = "artifact detection"
	translation = "translation"
	fewshotobjectcountinganddetection = "few-shot object counting and detection"
	d3objectdetectionroi = "3d object detection (roi)"
	creditscore = "credit score"
	mllmevaluationaesthetics = "mllm evaluation: aesthetics"
	videopropagation = "video propagation"
	patentclassification = "patent classification"
	graphsampling = "graph sampling"
	procedurelearning = "procedure learning"
	transparentobjectdetection = "transparent object detection"
	automatedpulmonarynoduledetectionandclassification = "automated pulmonary nodule detection and classification"
	solarirradianceforecasting = "solar irradiance forecasting"
	timeseriesaveraging = "time series averaging"
	contextner = "contextner"
	definitionmodelling = "definition modelling"
	mmsqlperformance = "mmsql performance"
	documentlevelmachinetranslation = "document level machine translation"
	marineanimalsegmentation = "marine animal segmentation"
	classificationwithbinaryweightnetwork = "classification with binary weight network"
	oneshotpartsegmentationofcontainaffordanceinterclass = "one-shot part segmentation of contain affordance - inter class"
	webcamrgbimageclassification = "webcam (rgb) image classification"
	leadershipinference = "leadership inference"
	d3objectcaptioning = "3d object captioning"
	shapefromtexture = "shape from texture"
	outlierensembles = "outlier ensembles"
	personalizedfederatedlearning = "personalized federated learning"
	onlinevectorizedhdmapconstruction = "online vectorized hd map construction"
	androidmalwaredetection = "android malware detection"
	geophysics = "geophysics"
	neuralnetworksimulation = "neural network simulation"
	classificationoftoxicengagingfactclaimingcomments = "classification of toxic, engaging, fact-claiming comments"
	zeroshotcrosslingualfactchecking = "zero-shot cross-lingual fact-checking"
	foveadetection = "fovea detection"
	entityembeddings = "entity embeddings"
	spatialawareimageediting = "spatial-aware image editing"
	copydetection = "copy detection"
	spectralefficiencyanalysisofuplinkdownlinkdecoupledaccessincv2xnetworks = "spectral efficiency analysis of uplink-downlink decoupled access in c-v2x networks"
	shadowdetectionandremoval = "shadow detection and removal"
	dataimputation70 = "data imputation 70%"
	selfsupervisedhumanactionrecognition = "self-supervised human action recognition"
	brats2021 = "brats2021"
	ontologymatching = "ontology matching"
	facialexpressionrecognitionfer = "facial expression recognition (fer)"
	denseobjectdetection = "dense object detection"
	semanticcontourprediction = "semantic contour prediction"
	nuclearsegmentation = "nuclear segmentation"
	selfknowledgedistillation = "self-knowledge distillation"
	videoindividualcounting = "video individual counting"
	imagestylization = "image stylization"
	relation = "relation"
	sciencequestionanswering = "science question answering"
	selfsupervisedvideoretrieval = "self-supervised video retrieval"
	videodenoising = "video denoising"
	selfsupervisedactionrecognitionlinear = "self-supervised action recognition linear"
	legalreasoning = "legal reasoning"
	d3pointcloudlinearclassification = "3d point cloud linear classification"
	retrievalaugmentedfewshotincontextaudiocaptioning = "retrieval-augmented few-shot in-context audio captioning"
	semisupervisedvideoactiondetection = "semi-supervised video action detection"
	d3sourcefreedomainadaptation = "3d source-free domain adaptation"
	patentfiguredescriptiongeneration = "patent figure description generation"
	literaturemining = "literature mining"
	chemicaldiseaseinteractionextraction = "chemical-disease interaction extraction"
	colorectalpolypscharacterization = "colorectal polyps characterization"
	languagemodeling = "language modeling"
	openendedquestionanswering = "open-ended question answering"
	audiovisualsynchronization = "audio-visual synchronization"
	unsupervisedactionsegmentation = "unsupervised action segmentation"
	constraineddiffeomorphicimageregistration = "constrained diffeomorphic image registration"
	segmentedmultimodalnamedentityrecognition = "segmented multimodal named entity recognition"
	boundarycaptioning = "boundary captioning"
	colormanipulation = "color manipulation"
	emotioncauseextraction = "emotion cause extraction"
	pseudolabelfiltering = "pseudo label filtering"
	jobprediction = "job prediction"
	conversationalresponsegeneration = "conversational response generation"
	metareinforcementlearning = "meta reinforcement learning"
	activelearning = "active learning"
	selfsupervisedlearning = "self-supervised learning"
	headdetection = "head detection"
	crossdomainnamedentityrecognition = "cross-domain named entity recognition"
	singleobjectdiscovery = "single-object discovery"
	actionparsing = "action parsing"
	unconditionalmoleculegeneration = "unconditional molecule generation"
	humanorganssensesmultiplechoice = "human organs senses multiple choice"
	wordembeddings = "word embeddings"
	cadenza1task2incar = "cadenza 1 - task 2 - in car"
	referringvideoobjectsegmentation = "referring video object segmentation"
	unsupervisedmnist = "unsupervised mnist"
	ageinvariantfacerecognition = "age-invariant face recognition"
	knowledgegraphcompletion = "knowledge graph completion"
	unsupervisedimagesegmentation = "unsupervised image segmentation"
	uie = "uie"
	actionunitdetection = "action unit detection"
	labelshiftofblendedtargetdomainadaptation = "label shift of blended-target domain adaptation"
	cervicalcancerbiopsyidentification = "cervical cancer biopsy identification"
	testtimeadaptation = "test-time adaptation"
	autismdetection = "autism detection"
	eventdriventrading = "event-driven trading"
	compressedimagequalityassessment = "compressed image quality assessment"
	outdoorlocalization = "outdoor localization"
	languageidentification = "language identification"
	scientificconceptextraction = "scientific concept extraction"
	reasoningvideoobjectsegmentation = "reasoning video object segmentation"
	visualstorytelling = "visual storytelling"
	inductiveknowledgegraphcompletion = "inductive knowledge graph completion"
	unsupervisedopinionsummarization = "unsupervised opinion summarization"
	naturallanguagevisualgrounding = "natural language visual grounding"
	esen = "es-en"
	noiseestimation = "noise estimation"
	analogicalsimilarity = "analogical similarity"
	japanesewordsegmentation = "japanese word segmentation"
	hypergraphembedding = "hypergraph embedding"
	imagesegmentation = "image segmentation"
	stochasticoptimization = "stochastic optimization"
	seismicinterpretation = "seismic interpretation"
	avg = "avg"
	informationretrieval = "information retrieval"
	chinesesentimentanalysis = "chinese sentiment analysis"
	videoclassification = "video classification"
	geometrybasedoperatorlearning = "geometry-based operator learning"
	dataintegration = "data integration"
	cyberattackinvestigation = "cyber attack investigation"
	tokenreduction = "token reduction"
	imagesimilaritydetection = "image similarity detection"
	texttospeechsynthesis = "text-to-speech synthesis"
	texttoimagegeneration = "text-to-image generation"
	brainimagesegmentation = "brain image segmentation"
	documentsummarization = "document summarization"
	plantphenotyping = "plant phenotyping"
	logorecognition = "logo recognition"
	traveltimeestimation = "travel time estimation"
	ancestordescendantprediction = "ancestor-descendant prediction"
	localdistortion = "local distortion"
	acousticmodelling = "acoustic modelling"
	d3semanticoccupancyprediction = "3d semantic occupancy prediction"
	speechdereverberation = "speech dereverberation"
	stsb = "sts-b"
	namedentityrecognitionner = "named entity recognition (ner)"
	visu = "visu"
	soundscapeevaluation = "soundscape evaluation"
	textbasedgames = "text-based games"
	rgbdreconstruction = "rgb-d reconstruction"
	tablefunctionalanalysis = "table functional analysis"
	autonomousdriving = "autonomous driving"
	imputation = "imputation"
	predictclinicaloutcome = "predict clinical outcome"
	abstractivequestionanswering = "abstractive question answering"
	photogeolocationestimation = "photo geolocation estimation"
	spatiotemporalvideogrounding = "spatio-temporal video grounding"
	nodeclassificationonnonhomophilicheterophilicgraphs = "node classification on non-homophilic (heterophilic) graphs"
	composedvideoretrievalcovr = "composed video retrieval (covr)"
	paraphraseidentificationwithinbiencoder = "paraphrase identification within bi-encoder"
	videofixedregioninpainting = "video fixed region inpainting"
	gymhalfcheetahrandom = "gym halfcheetah-random"
	stylechangedetection = "style change detection"
	fracturedetection = "fracture detection"
	featuresplatting = "feature splatting"
	toponymresolution = "toponym resolution"
	licenseplaterecognition = "license plate recognition"
	safetyalignment = "safety alignment"
	adversarialattackdetection = "adversarial attack detection"
	finegrainedurbanflowinference = "fine-grained urban flow inference"
	generalizedzeroshotobjectdetection = "generalized zero-shot object detection"
	musicgenreclassification = "music genre classification"
	crossdomainfacialexpressionrecognition = "cross-domain facial expression recognition"
	singleimagederaining = "single image deraining"
	trec2019passageranking = "trec 2019 passage ranking"
	outofdistributiongeneralization = "out-of-distribution generalization"
	mathematicalquestionanswering = "mathematical question answering"
	codeswitchedtranslationtoeng = "code-switched translation to eng"
	partiallylabeleddatasets = "partially labeled datasets"
	motionexpressionsguidedvideosegmentation = "motion expressions guided video segmentation"
	datapoisoning = "data poisoning"
	talkingheadgeneration = "talking head generation"
	trajectorylongtaildistributionformutiobjecttracking = "trajectory long-tail distribution for muti-object tracking"
	clusteringmultivariatetimeseries = "clustering multivariate time series"
	visuallyguidedsoundsourceseparation = "visually guided sound source separation"
	stablemcivsprogressivemci = "stable mci vs progressive mci"
	painintensityregression = "pain intensity regression"
	speechsynthesisgujarati = "speech synthesis - gujarati"
	tokenclassification = "token classification"
	smartgridprediction = "smart grid prediction"
	superdiagnostic = "super-diagnostic"
	cubicsplinesimageregistration = "cubic splines image registration"
	swappedoperands = "swapped operands"
	enusslotfilling = "en-us slot filling"
	parkingspaceoccupancy = "parking space occupancy"
	sequentialquantileestimation = "sequential quantile estimation"
	adversarialdefenseagainstfgsmattack = "adversarial defense against fgsm attack"
	attributemining = "attribute mining"
	speakerattributioningermanparliamentarydebatesgermeval2023subtask1 = "speaker attribution in german parliamentary debates (germeval 2023, subtask 1)"
	crystalsystemclassification = "crystal system classification"
	sourcefreedomainadaptation = "source-free domain adaptation"
	cantileverbeam = "cantilever beam"
	settographprediction = "set-to-graph prediction"
	physicsinformedmachinelearning = "physics-informed machine learning"
	molecularrepresentation = "molecular representation"
	carlamapleaderboard = "carla map leaderboard"
	graphsimilarity = "graph similarity"
	englishproverbs = "english proverbs"
	realtimestrategygames = "real-time strategy games"
	wordalignment = "word alignment"
	videostyletransfer = "video style transfer"
	senceclassification = "sence classification"
	humanobjectinteractionconceptdiscovery = "human-object interaction concept discovery"
	clouddetection = "cloud detection"
	crosslingualquestionanswering = "cross-lingual question answering"
	autonomousracing = "autonomous racing"
	semisupervisedaudioclassification = "semi-supervised audio classification"
	mmrtotal = "mmr total"
	crossdocumentlanguagemodeling = "cross-document language modeling"
	carposeestimation = "car pose estimation"
	audioqualityassessment = "audio quality assessment"
	autonomouswebnavigation = "autonomous web navigation"
	learningwithnoisylabels = "learning with noisy labels"
	thermalimagesegmentation = "thermal image segmentation"
	functiondocstringmismatch = "function-docstring mismatch"
	posecontrastivelearning = "pose contrastive learning"
	spectralsuperresolution = "spectral super-resolution"
	humanobjectinteractionanticipation = "human-object interaction anticipation"
	syntheticsongdetection = "synthetic song detection"
	sportsanalytics = "sports analytics"
	sarshipdetection = "sar ship detection"
	sequentialdistributionfunctionestimation = "sequential distribution function estimation"
	highlevelsynthesis = "high-level synthesis"
	triadprediction = "triad prediction"
	speechsynthesistamil = "speech synthesis - tamil"
	opdsingleview3dopenablepartdetection = "opd: single-view 3d openable part detection"
	capacityestimation = "capacity estimation"
	semisuperviseddomainadaptation = "semi-supervised domain adaptation"
	proteindesign = "protein design"
	latentaspectdetection = "latent aspect detection"
	geometryproblemsolving = "geometry problem solving"
	longrangemodeling = "long-range modeling"
	virtualtryoff = "virtual try-off"
	epilepsyprediction = "epilepsy prediction"
	flowchartgroundeddialogresponsegeneration = "flowchart grounded dialog response generation"
	representationlearning = "representation learning"
	thermalinfraredpedestriandetection = "thermal infrared pedestrian detection"
	zeroshotlongvideoquestionanswering = "zero-shot long video question answering"
	oneshotpartsegmentationofwrapgraspaffordanceinterclass = "one-shot part segmentation of wrap-grasp affordance - inter class"
	roomenvv2 = "roomenv-v2"
	multipersonposeforecasting = "multi-person pose forecasting"
	temporalrelationextraction = "temporal relation extraction"
	formationenergy = "formation energy"
	remaininglengthofstay = "remaining length of stay"
	fakeimageattribution = "fake image attribution"
	neuralrendering = "neural rendering"
	medicalnamedentityrecognition = "medical named entity recognition"
	edgedetection = "edge detection"
	unsupervisedhumanposeestimation = "unsupervised human pose estimation"
	unsupervisedfewshotaudioclassification = "unsupervised few-shot audio classification"
	crosslingualinformationretrieval = "cross-lingual information retrieval"
	covid19imagesegmentation = "covid-19 image segmentation"
	twosampletesting = "two-sample testing"
	multispectralimagesuperresolution = "multispectral image super-resolution"
	crosslingualnaturallanguageinference = "cross-lingual natural language inference"
	spikesorting = "spike sorting"
	stockprediction = "stock prediction"
	weaklysupervised3dpointcloudsegmentation = "weakly supervised 3d point cloud segmentation"
	anachronisms = "anachronisms"
	formalfallaciessyllogismsnegation = "formal fallacies syllogisms negation"
	personrecognition = "person recognition"
	multidomainrecommendersystems = "multi-domain recommender systems"
	emggesturerecognition = "emg gesture recognition"
	boardgames = "board games"
	tensordecomposition = "tensor decomposition"
	d3singleobjecttracking = "3d single object tracking"
	imbalancedclassification = "imbalanced classification"
	rotatedmnist = "rotated mnist"
	radarodometry = "radar odometry"
	medicalimagesegmentation = "medical image segmentation"
	tumourclassification = "tumour classification"
	textbasedstockprediction = "text-based stock prediction"
	textclassification = "text classification"
	financialanalysis = "financial analysis"
	zeroshottexttovideogeneration = "zero-shot text-to-video generation"
	fren = "fr-en"
	audiosuperresolution = "audio super-resolution"
	handwrittenmathmaticalexpressionrecognition = "handwritten mathmatical expression recognition"
	adroidpencloned = "adroid pen-cloned"
	connectivedetection = "connective detection"
	streamingtargetsoundextraction = "streaming target sound extraction"
	factualvisualquestionanswering = "factual visual question answering"
	keywordspottingongooglespeechcommands = "keyword spotting on google speech commands"
	mathwordproblemsolving = "math word problem solving"
	skincancersegmentation = "skin cancer segmentation"
	speechsynthesisassamese = "speech synthesis - assamese"
	multidialectvietnamese = "multi-dialect vietnamese"
	lexicalnormalization = "lexical normalization"
	zeroshotgesturerecognition = "zero-shot gesture recognition"
	unsupervisedimageclassification = "unsupervised image classification"
	croptypemapping = "crop type mapping"
	videotoimageaffordancegrounding = "video-to-image affordance grounding"
	nmrjcoupling = "nmr j-coupling"
	argumentretrieval = "argument retrieval"
	medicaldiagnosis = "medical diagnosis"
	ecgdigitization = "ecg digitization"
	collaborativeplanacquisition = "collaborative plan acquisition"
	securitystudies = "security studies"
	acutestrokelesionsegmentation = "acute stroke lesion segmentation"
	hierarchicalmultilabelclassification = "hierarchical multi-label classification"
	imagedocumentclustering = "image/document clustering"
	jointnerandclassification = "joint ner and classification"
	raindropremoval = "raindrop removal"
	zeroshotinstancesegmentation = "zero-shot instance segmentation"
	affineimageregistration = "affine image registration"
	dialmeterreading = "dial meter reading"
	timeseriesstreams = "time series streams"
	motioncaptioning = "motion captioning"
	cowidentification = "cow identification"
	gazeestimation = "gaze estimation"
	visualodometry = "visual odometry"
	unsupervisedlandmarkdetection = "unsupervised landmark detection"
	phrasegrounding = "phrase grounding"
	visualsocialrelationshiprecognition = "visual social relationship recognition"
	personidentificationzeroshot = "person identification (zero-shot)"
	textureclassification = "texture classification"
	passageranking = "passage ranking"
	zslvideoclassification = "zsl video classification"
	roadsegmentation = "road segmentation"
	visualprompttuning = "visual prompt tuning"
	linesegmentdetection = "line segment detection"
	graphconstruction = "graph construction"
	hintgeneration = "hint generation"
	networkinterpretation = "network interpretation"
	d3assembly = "3d assembly"
	d3depthestimation = "3d depth estimation"
	semanticcommunication = "semantic communication"
	missinglabels = "missing labels"
	mistakedetection = "mistake detection"
	epidemiology = "epidemiology"
	automaticsleepstageclassification = "automatic sleep stage classification"
	arabictextdiacritization = "arabic text diacritization"
	densityratioestimation = "density ratio estimation"
	collaborativefairness = "collaborative fairness"
	nlpbasedpersonretrival = "nlp based person retrival"
	textvqa = "textvqa"
	metadataquality = "metadata quality"
	diffusiongeneratedfacesdetection = "diffusion-generated faces detection"
	audiovisualeventlocalization = "audio-visual event localization"
	humanhumaninteractionrecognition = "human-human interaction recognition"
	loopclosuredetection = "loop closure detection"
	oddoneout = "odd one out"
	personalizedimagegeneration = "personalized image generation"
	misogynisticaggressionidentification = "misogynistic aggression identification"
	partawarepanopticsegmentation = "part-aware panoptic segmentation"
	audiovisualspeechrecognition = "audio-visual speech recognition"
	precipitationforecasting = "precipitation forecasting"
	cosalientobjectdetection = "co-salient object detection"
	d3shapeclassification = "3d shape classification"
	opendomainquestionanswering = "open-domain question answering"
	gourndvideosynthesisfromsatelliteimage = "gournd video synthesis from satellite image"
	demosaicking = "demosaicking"
	causalinference = "causal inference"
	taskorienteddialoguesystems = "task-oriented dialogue systems"
	commentgeneration = "comment generation"
	githubissueresolution = "github issue resolution"
	multiclassclassification = "multi-class classification"
	graphclassification = "graph classification"
	crossviewreferringmultiobjecttracking = "cross-view referring multi-object tracking"
	datavaluation = "data valuation"
	calvingfrontdelineationinsyntheticapertureradarimagerywithfixedtrainingamount = "calving front delineation in synthetic aperture radar imagery with fixed training amount"
	bayesianoptimisation = "bayesian optimisation"
	d3multipersonposeestimation = "3d multi-person pose estimation"
	proteinstructureprediction = "protein structure prediction"
	imageandvideoforgerydetection = "image and video forgery detection"
	sceneawaredialogue = "scene-aware dialogue"
	selfsupervisedimageclassificationonimagenet = "self-supervised image classification on imagenet"
	d3inpainting = "3d inpainting"
	wholeslideimages = "whole slide images"
	finegrainedimageclassification = "fine-grained image classification"
	visualquestionansweringvqasplitb = "visual question answering (vqa) split b"
	propernoun = "proper noun"
	overlapped10050 = "overlapped 100-50"
	alzheimersdiseasedetection = "alzheimer's disease detection"
	vietnamesemultimodallearning = "vietnamese multimodal learning"
	d3interactinghandposeestimation = "3d interacting hand pose estimation"
	d2tinyobjectdetection = "2d tiny object detection"
	moleculargraphgeneration = "molecular graph generation"
	germeval2024sharedtask1subtask1 = "germeval2024 shared task 1 subtask 1"
	generalreinforcementlearning = "general reinforcement learning"
	longtaillearning = "long-tail learning"
	benchmarking = "benchmarking"
	aspectcategorysentimentanalysis = "aspect category sentiment analysis"
	ecgwavedelineation = "ecg wave delineation"
	singleshothdrreconstruction = "single-shot hdr reconstruction"
	ingenuity = "ingenuity"
	trafficclassification = "traffic classification"
	agebiasconflicting = "age/bias-conflicting"
	nriqa = "nr-iqa"
	texttogql = "text-to-gql"
	fewshotinstancesegmentation = "few-shot instance segmentation"
	knowledgegraphs = "knowledge graphs"
	inductivelogicprogramming = "inductive logic programming"
	roomenvv0 = "roomenv-v0"
	openvocabulary3dinstancesegmentation = "open vocabulary 3d instance segmentation"
	ageandgenderestimation = "age and gender estimation"
	datamining = "data mining"
	selectionbias = "selection bias"
	zeroshotanomalydetection = "zero-shot anomaly detection"
	editscriptgeneration = "edit script generation"
	partofspeechtagging = "part-of-speech tagging"
	reliableintelligenceidentification = "reliable intelligence identification"
	sessionsearch = "session search"
	documentdating = "document dating"
	graphrepresentationlearning = "graph representation learning"
	scenerecognition = "scene recognition"
	informationcascadepopularityprediction = "information cascade popularity prediction"
	newssummarization = "news summarization"
	musicgenretransfer = "music genre transfer"
	mujoco = "mujoco"
	atrialfibrillationdetection = "atrial fibrillation detection"
	zeroshotactiondetection = "zero-shot action detection"
	depictioninvariantobjectrecognition = "depiction invariant object recognition"
	deadreckoningprediction = "dead-reckoning prediction"
	speechsynthesismalayalam = "speech synthesis - malayalam"
	fewshotskeletonbasedactionrecognition = "few-shot skeleton-based action recognition"
	referringimagemattingexpressionbased = "referring image matting (expression-based)"
	selfadaptivesystem = "self adaptive system"
	covid19modelling = "covid-19 modelling"
	videosegmentation = "video segmentation"
	eventbasedvideoreconstruction = "event-based video reconstruction"
	systematicgeneralization = "systematic generalization"
	cardgames = "card games"
	ccgsupertagging = "ccg supertagging"
	datadrivenoptimalcontrol = "data driven optimal control"
	d2panopticsegmentation = "2d panoptic segmentation"
	foodrecommendation = "food recommendation"
	k16 = "16k"
	rolefillerentityextraction = "role-filler entity extraction"
	unsupervisedanomalydetectionwithspecifiedsettings1anomaly = "unsupervised anomaly detection with specified settings -- 1% anomaly"
	highschoolphysics = "high school physics"
	participantinterventioncomparisonoutcomeextraction = "participant intervention comparison outcome extraction"
	pointcloudgeneration = "point cloud generation"
	jointentityandrelationextraction = "joint entity and relation extraction"
	sst2 = "sst-2"
	densepixelcorrespondenceestimation = "dense pixel correspondence estimation"
	monocularcrossviewroadsceneparsingvehicle = "monocular cross-view road scene parsing(vehicle)"
	vietnamesescenetext = "vietnamese scene text"
	directtransferpersonreidentification = "direct transfer person re-identification"
	reconstruction = "reconstruction"
	footballactionvaluation = "football action valuation"
	imagesmoothing = "image smoothing"
	geneinteractionprediction = "gene interaction prediction"
	overlapped155 = "overlapped 15-5"
	modelediting = "model editing"
	activityrecognitioninvideos = "activity recognition in videos"
	recognizingemotioncauseinconversations = "recognizing emotion cause in conversations"
	genreclassification = "genre classification"
	zeroshotenvironmentsoundclassification = "zero-shot environment sound classification"
	recommendationsystems = "recommendation systems"
	extractaspectpolaritytuple = "extract aspect-polarity tuple"
	hyperspectralimageanalysis = "hyperspectral image analysis"
	d3panopticsegmentation = "3d panoptic segmentation"
	cleanlabelbackdoorattack0024 = "clean-label backdoor attack (0.024%)"
	hellaswag = "hellaswag"
	persuasionstrategies = "persuasion strategies"
	singleimageblinddeblurring = "single-image blind deblurring"
	eegdenoising = "eeg denoising"
	weaklysupervisedsemanticsegmentation = "weakly-supervised semantic segmentation"
	zeroshotlearningdomaingeneralization = "zero-shot learning + domain generalization"
	semisupervisedformalitystyletransfer = "semi-supervised formality style transfer"
	vietnameselexicalnormalization = "vietnamese lexical normalization"
	semanticretrieval = "semantic retrieval"
	partbasedrepresentationlearning = "part-based representation learning"
	crossmodalpersonreidentification = "cross-modal person re-identification"
	datafreeknowledgedistillation = "data-free knowledge distillation"
	openvocabularyvideosegmentation = "open-vocabulary video segmentation"
	stereoscopicimagequalityassessment = "stereoscopic image quality assessment"
	aerialsceneclassification = "aerial scene classification"
	horizonlineestimation = "horizon line estimation"
	groundedopenvocabularyacquisition = "grounded open vocabulary acquisition"
	filmsimulation = "film simulation"
	avbhigh = "a-vb high"
	pendulumv1 = "pendulum-v1"
	argumentpairextractionape = "argument pair extraction (ape)"
	floodinundationmapping = "flood inundation mapping"
	outlierdetection = "outlier detection"
	positionregression = "position regression"
	texttofacegeneration = "text-to-face generation"
	superpixelimageclassification = "superpixel image classification"
	sentencepairmodeling = "sentence pair modeling"
	foveation = "foveation"
	sensorfusion = "sensor fusion"
	documentimageclassification = "document image classification"
	spanishtextdiacritization = "spanish text diacritization"
	relationalpatternlearning = "relational pattern learning"
	rgbdsemanticsegmentation = "rgbd semantic segmentation"
	personalizedsegmentation = "personalized segmentation"
	depressiondetection = "depression detection"
	predictionintervals = "prediction intervals"
	tablerecognition = "table recognition"
	opticcupsegmentation = "optic cup segmentation"
	extractivetagssummarization = "extractive tags summarization"
	mathematicalinduction = "mathematical induction"
	nethackscore = "nethack score"
	transcriptionfactorbindingsitepredictionhuman = "transcription factor binding site prediction (human)"
	collegemathematics = "college mathematics"
	complexscenebreakingandsynthesis = "complex scene breaking and synthesis"
	multiview3dhumanposeestimation = "multi-view 3d human pose estimation"
	transportationmodedetection = "transportation mode detection"
	opticalcharacterrecognitionocr = "optical character recognition (ocr)"
	functionaldescriptiongeneration = "functional description generation"
	arzencodeswitchedtranslationtoeng = "arzen code-switched translation to eng"
	fontgeneration = "font generation"
	variableselection = "variable selection"
	skeletonrigprediction = "skeleton rig prediction"
	onlineactiondetection = "online action detection"
	surrogatehydrodynamicmodeling = "surrogate hydrodynamic modeling"
	unsupervisedextractivesummarization = "unsupervised extractive summarization"
	crossdomainfewshot = "cross-domain few-shot"
	imagerecognition = "image recognition"
	onlinemistakedetection = "online mistake detection"
	dota2 = "dota 2"
	timeseries = "time series"
	pornographydetection = "pornography detection"
	multiplepeopletracking = "multiple people tracking"
	semisupervisedmedicalimagesegmentation = "semi-supervised medical image segmentation"
	imagelevelsupervisedinstancesegmentation = "image-level supervised instance segmentation"
	persuasivewritingstrategydetectionlevel3 = "persuasive writing strategy detection level-3"
	medicalimagedeblurring = "medical image deblurring"
	dialoguegeneration = "dialogue generation"
	linearevaluation = "linear evaluation"
	annotatedcodesearch = "annotated code search"
	salientobjectdetection = "salient object detection"
	fg1pg1 = "fg-1-pg-1"
	audiotagging = "audio tagging"
	sceneunderstanding = "scene understanding"
	imagetextretrieval = "image-text retrieval"
	psoconvnetsdynamics1 = "pso-convnets dynamics 1"
	multiclassanomalydetection = "multi-class anomaly detection"
	instrumentrecognition = "instrument recognition"
	overlapped5050 = "overlapped 50-50"
	dynamicreconstruction = "dynamic reconstruction"
	scenetextdetection = "scene text detection"
	diffusionpersonalization = "diffusion personalization"
	changepointdetection = "change point detection"
	sqltotext = "sql-to-text"
	datasetgeneration = "dataset generation"
	sportsballdetectionandtracking = "sports ball detection and tracking"
	brainvisualreconstructionfromfmri = "brain visual reconstruction from fmri"
	saltandpeppernoiseremoval = "salt-and-pepper noise removal"
	avbculture = "a-vb culture"
	incrementallearningoncifar100b05stepsof20classes = "incremental learning on cifar100-b0(5steps of 20 classes)"
	logicgridpuzzle = "logic grid puzzle"
	evolvingdomaingeneralization = "evolving domain generalization"
	endtoendrstparsing = "end-to-end rst parsing"
	burstimagesuperresolution = "burst image super-resolution"
	authorityinvolvementdetection = "authority involvement detection"
	conversationalsearch = "conversational search"
	knowledgedistillation = "knowledge distillation"
	timeseriesgeneration = "time series generation"
	visualinstructionfollowing = "visual instruction following"
	chinesedependencyparsing = "chinese dependency parsing"
	visualdialog = "visual dialog"
	conversationalquestionanswering = "conversational question answering"
	moralscenarios = "moral scenarios"
	networkpruning = "network pruning"
	memexquestionanswering = "memex question answering"
	videobasedpersonreidentification = "video-based person re-identification"
	generalactionvideoanomalydetection = "general action video anomaly detection"
	actionqualityassessmentreportgeneration = "action quality assessment report generation"
	bloodpressureestimation = "blood pressure estimation"
	sts = "sts"
	retinalvesselsegmentation = "retinal vessel segmentation"
	d4rl = "d4rl"
	speechtotext = "speech-to-text"
	d3objectclassification = "3d object classification"
	audiodeepfakedetection = "audio deepfake detection"
	relevancedetection = "relevance detection"
	birdseyeviewobjectdetection = "birds eye view object detection"
	geographicquestionanswering = "geographic question answering"
	musicinformationretrieval = "music information retrieval"
	diseaseprediction = "disease prediction"
	segmentation = "segmentation"
	industrialrobots = "industrial robots"
	stereolidarfusion = "stereo-lidar fusion"
	zeroshotflowchartgroundeddialogresponsegeneration = "zero-shot flowchart grounded dialog response generation"
	humanbodyvolumeestimation = "human body volume estimation"
	ai2reasoningchallenge = "ai2 reasoning challenge"
	lexicalentailment = "lexical entailment"
	multigoalreinforcementlearning = "multi-goal reinforcement learning"
	d3humandynamics = "3d human dynamics"
	robotmanipulation = "robot manipulation"
	d3objectreconstruction = "3d object reconstruction"
	oneshotfacestylization = "one-shot face stylization"
	detectingimagemanipulation = "detecting image manipulation"
	bidirectionalrelationshipclassification = "bidirectional relationship classification"
	rebasin = "re-basin"
	syntheticimagedetection = "synthetic image detection"
	videoinstancesegmentation = "video instance segmentation"
	predictingpatientoutcomes = "predicting patient outcomes"
	handwrittenchinesetextrecognition = "handwritten chinese text recognition"
	openknowledgegraphembedding = "open knowledge graph embedding"
	topicmodels = "topic models"
	preferencemapping = "preference mapping"
	zeroshotcrosslingualvisualreasoning = "zero-shot cross-lingual visual reasoning"
	sessionbasedrecommendations = "session-based recommendations"
	audioclassification = "audio classification"
	shorttextclustering = "short text clustering"
	imageretrieval = "image retrieval"
	hyperspectralimageclustering = "hyperspectral image clustering"
	geolocalization = "geo-localization"
	arousalestimation = "arousal estimation"
	multiviewgaitrecognition = "multiview gait recognition"
	textbaseddenovomoleculegeneration = "text-based de novo molecule generation"
	typefacecompletion = "typeface completion"
	backdoorattack = "backdoor attack"
	crowdflowsprediction = "crowd flows prediction"
	sentenceambiguity = "sentence ambiguity"
	brainsegmentation = "brain segmentation"
	keypointestimation = "keypoint estimation"
	physicalvideoanomalydetection = "physical video anomaly detection"
	lippasswordclassification = "lip password classification"
	tgifframe = "tgif-frame"
	aspectcategoryopinionsentimentquadrupleextraction = "aspect-category-opinion-sentiment quadruple extraction"
	selfsupervisedskeletonbasedactionrecognition = "self-supervised skeleton-based action recognition"
	tonemapping = "tone mapping"
	predicateclassification = "predicate classification"
	nestedtermrecognitionfromflatsupervision = "nested term recognition from flat supervision"
	visionandlanguagenavigation = "vision and language navigation"
	motionforecasting = "motion forecasting"
	fad = "fad"
	htr = "htr"
	multianimaltrackingwithidentification = "multi-animal tracking with identification"
	mixedreality = "mixed reality"
	eegartifactremoval = "eeg artifact removal"
	factbasedtextediting = "fact-based text editing"
	crassai = "crass ai"
	microexpressionrecognition = "micro-expression recognition"
	subjectivityanalysis = "subjectivity analysis"
	musicgeneration = "music generation"
	irisrecognition = "iris recognition"
	backdoordefensefordatafreedistillationwithpoisonedteachers = "backdoor defense for data-free distillation with poisoned teachers"
	shorttermobjectinteractionanticipation = "short-term object interaction anticipation"
	subdiagnostic = "sub-diagnostic"
	imagedeepnetworks = "image deep networks"
	cleanlabelbackdoorattack005 = "clean-label backdoor attack (0.05%)"
	unsupervisedimageregistration = "unsupervised image registration"
	zeroshotsemanticsegmentation = "zero-shot semantic segmentation"
	banglaspellingerrorcorrection = "bangla spelling error correction"
	handjointreconstruction = "hand joint reconstruction"
	actionclassification1shot = "action classification (1-shot)"
	sentimentclassification = "sentiment classification"
	genericeventboundarydetection = "generic event boundary detection"
	opticalcharacterrecognition = "optical character recognition"
	languagebasedtemporallocalization = "language-based temporal localization"
	sequentialdecisionmaking = "sequential decision making"
	emotionrecognition = "emotion recognition"
	handobjectinteractiondetection = "hand-object interaction detection"
	transparentobjectdepthestimation = "transparent object depth estimation"
	kinshipverification = "kinship verification"
	whitematterfibertractography = "white matter fiber tractography"
	motionretargeting = "motion retargeting"
	multilabeltextclassification = "multi-label text classification"
	multiplesequencealignment = "multiple sequence alignment"
	audiovisuallearning = "audio-visual learning"
	fillmask = "fill mask"
	videoobjectsegmentation = "video object segmentation"
	experimentaldesign = "experimental design"
	intentdiscovery = "intent discovery"
	utterancelevelpronounciationscoring = "utterance-level pronounciation scoring"
	generalizablenovelviewsynthesis = "generalizable novel view synthesis"
	pitchcontrol = "pitch control"
	fewshotlearning = "few-shot learning"
	denovopeptidesequencing = "de novo peptide sequencing"
	multitasklearning = "multi-task learning"
	eegsignalclassification = "eeg signal classification"
	labelerrordetection = "label error detection"
	colormismatchcorrection = "color mismatch correction"
	objectlocalization = "object localization"
	relationalreasoning = "relational reasoning"
	elementarymathematics = "elementary mathematics"
	transformavideointoacomics = "transform a video into a comics"
	xrdregression = "xrd regression"
	multimodallexicaltranslation = "multimodal lexical translation"
	semisupervisedimageregression = "semi-supervised image regression"
	ageestimation = "age estimation"
	timetoeventprediction = "time-to-event prediction"
	textbasedpersonretrieval = "text-based person retrieval"
	tirod = "tirod"
	anorexiadetection = "anorexia detection"
	timeseriesprediction = "time series prediction"
	landmarkbasedsegmentation = "landmark-based segmentation"
	audiocaps = "audiocaps"
	lexicalanalysis = "lexical analysis"
	riddlesense = "riddle sense"
	deepreinforcementlearning = "deep reinforcement learning"
	corepromoterdetection = "core promoter detection"
	crosscorpus = "cross-corpus"
	zeroshotcrosslingualimagetotextretrieval = "zero-shot cross-lingual image-to-text retrieval"
	rfbasedactionrecognition = "rf-based action recognition"
	opinionsummarization = "opinion summarization"
	novelobjectdetection = "novel object detection"
	deletedtask2 = "(deleted task 2)"
	questionanswering = "question answering"
	monocularreconstruction = "monocular reconstruction"
	bloodcellcount = "blood cell count"
	gesturetogesturetranslation = "gesture-to-gesture translation"
	conversationsummarization = "conversation summarization"
	generalizedfewshotclassification = "generalized few-shot classification"
	laminarturbulentflowlocalisation = "laminar-turbulent flow localisation"
	textspotting = "text spotting"
	multitargetregression = "multi-target regression"
	detectingshadows = "detecting shadows"
	zeroshothumanobjectinteractiondetection = "zero-shot human-object interaction detection"
	imagerestoration = "image restoration"
	quantumcircuitclassificationquantumml = "quantum circuit classification (quantum ml)"
	videogrounding = "video grounding"
	audiofingerprint = "audio fingerprint"
	weaklysupervisedtemporalactionlocalization = "weakly-supervised temporal action localization"
	visuallocalization = "visual localization"
	domainadaptivepersonreidentification = "domain adaptive person re-identification"
	nonparametricclustering = "nonparametric clustering"
	multihopquestionanswering = "multi-hop question answering"
	taskplanning = "task planning"
	logicalreasoning = "logical reasoning"
	weaklysupervisedobjectdetection = "weakly supervised object detection"
	semanticrolelabeling = "semantic role labeling"
	unsupervisedtextrecognition = "unsupervised text recognition"
	toxicspansdetection = "toxic spans detection"
	zeroshotmachinetranslation = "zero-shot machine translation"
	posetracking = "pose tracking"
	d3moleculegeneration = "3d molecule generation"
	geneticriskprediction = "genetic risk prediction"
	longtermanticipation = "long term anticipation"
	crossencoderreranking = "cross encoder reranking"
	medicalimageregistration = "medical image registration"
	arzencodeswitchedtranslationtoara = "arzen code-switched translation to ara"
	timeseriesanalysis = "time series analysis"
	arc = "arc"
	deepnonparametricclustering = "deep nonparametric clustering"
	codegeneration = "code generation"
	im2spec = "im2spec"
	globaloptimization = "global-optimization"
	keypointdetection = "keypoint detection"
	openvocabularysemanticsegmentation = "open-vocabulary semantic segmentation"
	cerebrovascularnetworksegmentation = "cerebrovascular network segmentation"
	fewshotnli = "few-shot nli"
	pointofnoreturnpnrtemporallocalization = "point- of-no-return (pnr) temporal localization"
	rgbdepthanomalydetectionandsegmentation = "rgb+depth anomaly detection and segmentation"
	unsupervisedzeroshotpanopticsegmentation = "unsupervised zero-shot panoptic segmentation"
	celltracking = "cell tracking"
	nestedmentionrecognition = "nested mention recognition"
	tablebasedquestionanswering = "table-based question answering"
	handgesturerecognition = "hand gesture recognition"
	morph = "morph"
	userconstrainedthumbnailgeneration = "user constrained thumbnail generation"
	sceneflowestimationonkitti2015sceneflowtest = "scene flow estimation on kitti 2015 scene flow test"
	talkingfacegeneration = "talking face generation"
	beattracking = "beat tracking"
	melodyextraction = "melody extraction"
	jpegcompressionartifactreduction = "jpeg compression artifact reduction"
	temporalactionproposalgeneration = "temporal action proposal generation"
	expressivespeechsynthesis = "expressive speech synthesis"
	negationandspeculationscoperesolution = "negation and speculation scope resolution"
	defocusestimation = "defocus estimation"
	xlmr = "xlm-r"
	informationthreading = "information threading"
	recognizingandlocalizinghumanactions = "recognizing and localizing human actions"
	pointcloudcompletion = "point cloud completion"
	objectdetectioninaerialimages = "object detection in aerial images"
	multimodaldeeplearning = "multimodal deep learning"
	regionproposal = "region proposal"
	sketchcolorization = "sketch colorization"
	designsynthesis = "design synthesis"
	momentretrieval = "moment retrieval"
	spatialinterpolation = "spatial interpolation"
	slovaktextdiacritization = "slovak text diacritization"
	cadenza1task1headphone = "cadenza 1 - task 1 - headphone"
	multiobjectivereinforcementlearning = "multi-objective reinforcement learning"
	humanoidcontrol = "humanoid control"
	ensemblepruning = "ensemble pruning"
	visualanalogies = "visual analogies"
	pointcloudqualityassessment = "point cloud quality assessment"
	chartunderstanding = "chart understanding"
	safetyperceptionrecognition = "safety perception recognition"
	hypergraphrepresentations = "hypergraph representations"
	unsupervised3dmultipersonposeestimation = "unsupervised 3d multi-person pose estimation"
	semanticobjectinteractionclassification = "semantic object interaction classification"
	unsupervisedskeletonbasedactionrecognition = "unsupervised skeleton based action recognition"
	multipleobjectforecasting = "multiple object forecasting"
	videotosoundgeneration = "video-to-sound generation"
	trajectoryprediction = "trajectory prediction"
	automaticmodulationrecognition = "automatic modulation recognition"
	dateunderstanding = "date understanding"
	datafreequantization = "data free quantization"
	subgraphcountingtriangle = "subgraph counting - triangle"
	imagedeblurring = "image deblurring"
	subgroupdiscovery = "subgroup discovery"
	sarimagedespeckling = "sar image despeckling"
	amodaltracking = "amodal tracking"
	landslidesegmentation = "landslide segmentation"
	textsimilarity = "text similarity"
	speakerdiarization = "speaker diarization"
	unsupervisedsaliencydetection = "unsupervised saliency detection"
	offlinerl = "offline rl"
	xrayvisualquestionanswering = "x-ray visual question answering"
	sqlparsing = "sql parsing"
	modelextraction = "model extraction"
	clustering = "clustering"
	descriptive = "descriptive"
	simulatedgaussianmanipulation = "simulated gaussian manipulation"
	birdseyeviewsemanticsegmentation = "bird's-eye view semantic segmentation"
	weaklysupervisedanomalydetection = "weakly-supervised anomaly detection"
	leftventriclesegmentation = "left ventricle segmentation"
	textsummarization = "text summarization"
	videototextretrieval = "video to text retrieval"
	adroiddoorcloned = "adroid door-cloned"
	overlapped151 = "overlapped 15-1"
	zeroshottextclassification = "zero-shot text classification"
	videobasedgenerativeperformancebenchmarkingdetailorientation = "video-based generative performance benchmarking (detail orientation))"
	vesseldetection = "vessel detection"
	wnli = "wnli"
	chordrecognition = "chord recognition"
	episodeclassification = "episode classification"
	nmt = "nmt"
	newsretrieval = "news retrieval"
	crosslingualdocumentclassification = "cross-lingual document classification"
	replacepo = "replace - po"
	housegeneration = "house generation"
	semisupervisedrgbdsemanticsegmentation = "semi-supervised rgbd semantic segmentation"
	weaklysuperviseddatadenoising = "weakly supervised data denoising"
	l2regularization = "l2 regularization"
	poseguidedimagegeneration = "pose-guided image generation"
	knowledgegraphembeddings = "knowledge graph embeddings"
	pupildilation = "pupil dilation"
	opendomaindialog = "open-domain dialog"
	hatespanidentification = "hate span identification"
	snowremoval = "snow removal"
	linguisticacceptability = "linguistic acceptability"
	robustclassification = "robust classification"
	zeroshot3dclassification = "zero-shot 3d classification"
	diabetesprediction = "diabetes prediction"
	changedetectionforremotesensingimages = "change detection for remote sensing images"
	carlalongest6 = "carla longest6"
	humanposeforecasting = "human pose forecasting"
	d2poseestimation = "2d pose estimation"
	rgbdsalientobjectdetection = "rgb-d salient object detection"
	lwrclassification = "lwr classification"
	aestheticimagecaptioning = "aesthetic image captioning"
	clusteringalgorithmsevaluation = "clustering algorithms evaluation"
	gamedesign = "game design"
	filetypeidentification = "file type identification"
	naturalquestions = "natural questions"
	productrecommendation = "product recommendation"
	predictfuturevideoframes = "predict future video frames"
	disentanglement = "disentanglement"
	unsupervisedsemanticparsing = "unsupervised semantic parsing"
	crosslingualasr = "cross-lingual asr"
	proteinstabilityprediction = "protein stability prediction"
	poseprediction = "pose prediction"
	stmoel41bfinetuned = "st-moe-l 4.1b (fine-tuned)"
	malwareclassification = "malware classification"
	unsupervisedsentencecompression = "unsupervised sentence compression"
	imageretrievalonfashioniq = "image retrieval on fashion iq"
	interactivevideoobjectsegmentation = "interactive video object segmentation"
	motionsynthesis = "motion synthesis"
	twittersentimentanalysis = "twitter sentiment analysis"
	perpetualviewgeneration = "perpetual view generation"
	openworldsemisupervisedlearning = "open-world semi-supervised learning"
	imageeditingdection = "image editing dection"
	zeroshotobjectdetection = "zero-shot object detection"
	imageto3d = "image to 3d"
	scenelabeling = "scene labeling"
	verticalfederatedlearning = "vertical federated learning"
	generalizationbounds = "generalization bounds"
	genderbiasdetection = "gender bias detection"
	noiselevelprediction = "noise level prediction"
	booksummarization = "book summarization"
	dqnreplaydataset = "dqn replay dataset"
	openvocabularyactiondetection = "open vocabulary action detection"
	trainingfreeobjectcounting = "training-free object counting"
	passclassification = "pass classification"
	webcredibility = "web credibility"
	matrixfactorizationdecomposition = "matrix factorization / decomposition"
	graphdomainadaptation = "graph domain adaptation"
	d3geometryperception = "3d geometry perception"
	sentenceembeddings = "sentence embeddings"
	classincrementalsemanticsegmentation = "class-incremental semantic segmentation"
	answergeneration = "answer generation"
	zeroshottransferimageclassification = "zero-shot transfer image classification"
	nextbasketrecommendation = "next-basket recommendation"
	persuasiveness = "persuasiveness"
	videotemporalconsistency = "video temporal consistency"
	unusualactivitylocalization = "unusual activity localization"
	videonarrationcaptioning = "video narration captioning"
	mrireconstruction = "mri reconstruction"
	humanaging = "human aging"
	variationalmontecarlo = "variational monte carlo"
	voicepathologydetection = "voice pathology detection"
	continualforgetting = "continual forgetting"
	vocaltechniqueclassification = "vocal technique classification"
	d3humanposeandshapeestimation = "3d human pose and shape estimation"
	superviseddimensionalityreduction = "supervised dimensionality reduction"
	gotoanything = "go to anything"
	totalmagnetization = "total magnetization"
	crossdomainfewshotobjectdetection = "cross-domain few-shot object detection"
	transferability = "transferability"
	descriptionguidedmoleculegeneration = "description-guided molecule generation"
	speechenhancement = "speech enhancement"
	d3architecture = "3d architecture"
	machinetranslation = "machine translation"
	mitigatingcontextualbias = "mitigating contextual bias"
	timexnormalization = "timex normalization"
	compressedimagesuperresolution = "compressed image super-resolution"
	semanticrolelabelingpredictedpredicates = "semantic role labeling (predicted predicates)"
	weaklysupervisedactionsegmentationtranscript = "weakly supervised action segmentation (transcript)"
	audioinpainting = "audio inpainting"
	crosspartcrowdcounting = "cross-part crowd counting"
	raceh = "race-h"
	vectorgraphics = "vector graphics"
	humaninteractionrecognition = "human interaction recognition"
	lungnoduledetection = "lung nodule detection"
	authorprofiling = "author profiling"
	temporalinformationextraction = "temporal information extraction"
	arithmeticreasoning = "arithmetic reasoning"
	runtimeranking = "runtime ranking"
	dataablation = "data ablation"
	philosophicalreflection = "philosophical reflection"
	multimodalfashionimageediting = "multimodal fashion image editing"
	moleculardescriptiongeneration = "molecular description generation"
	featureselection = "feature selection"
	trafficsignrecognition = "traffic sign recognition"
	softwaretesting = "software testing"
	spatialrelationrecognition = "spatial relation recognition"
	oneshot3dactionrecognition = "one-shot 3d action recognition"
	culturalvocalburstsintensityprediction = "cultural vocal bursts intensity prediction"
	constituencyparsing = "constituency parsing"
	parameterestimation = "parameter estimation"
	d3humanreconstruction = "3d human reconstruction"
	clinicalknowledge = "clinical knowledge"
	realworldadversarialattack = "real-world adversarial attack"
	crosslingualentitylinking = "cross-lingual entity linking"
	safeexploration = "safe exploration"
	hierarchicaltextclustering = "hierarchical text clustering"
	voiceconversion = "voice conversion"
	textgeneration = "text generation"
	penguinsinatable = "penguins in a table"
	accidentanticipation = "accident anticipation"
	speakerattributioningermanparliamentarydebatesgermeval2023 = "speaker attribution in german parliamentary debates (germeval 2023, subtask 2)"
	histopathologicalsegmentation = "histopathological segmentation"
	jpegforgerylocalization = "jpeg forgery localization"
	textvariation = "text-variation"
	stereodepthestimation = "stereo depth estimation"
	ruinnames = "ruin names"
	drugdiscovery = "drug discovery"
	viewpointestimation = "viewpoint estimation"
	partiallyviewalignedmultiviewlearning = "partially view-aligned multi-view learning"
	forwardreactionprediction = "forward reaction prediction"
	proteinfunctionprediction = "protein function prediction"
	deidentification = "de-identification"
	faceageediting = "face age editing"
	universaldomainadaptation = "universal domain adaptation"
	genomeunderstanding = "genome understanding"
	gaitrecognitioninthewild = "gait recognition in the wild"
	generalizedzeroshotlearning = "generalized zero-shot learning"
	materialrecognition = "material recognition"
	onlinedomainadaptation = "online domain adaptation"
	addpq = "add - pq"
	hardattention = "hard attention"
	anatomy = "anatomy"
	qnli = "qnli"
	zeroshotvideoretrieval = "zero-shot video retrieval"
	textstyletransfer = "text style transfer"
	hyperspectralimagedenoising = "hyperspectral image denoising"
	d3anomalydetection = "3d anomaly detection"
	hyperedgeclassification = "hyperedge classification"
	imageaugmentation = "image augmentation"
	blindfacerestoration = "blind face restoration"
	vocalburstsvalenceprediction = "vocal bursts valence prediction"
	intensityimagedenoising = "intensity image denoising"
	buildingchangedetectionforremotesensingimages = "building change detection for remote sensing images"
	cancermetastasisdetection = "cancer metastasis detection"
	visualcrowdanalysis = "visual crowd analysis"
	mmvet = "mm-vet"
	drugatcclassification = "drug atc classification"
	libraryorientedcodegeneration = "library-oriented code generation"
	weaklysupervisednamedentityrecognition = "weakly-supervised named entity recognition"
	econometrics = "econometrics"
	inferenceoptimization = "inference optimization"
	videobasedgenerativeperformancebenchmarkingtemporalunderstanding = "video-based generative performance benchmarking (temporal understanding)"
	winowhy = "winowhy"
	motorimagerydecodinglefthandvsrighthand = "motor imagery decoding (left-hand vs right-hand)"
	texttovideogeneration = "text-to-video generation"
	d3facialexpressionrecognition = "3d facial expression recognition"
	detectingclimateenvironmentaltopics = "detecting climate/environmental topics"
	controlwithprametrisedactions = "control with prametrised actions"
	multilingualparaphrasegeneration = "multilingual paraphrase generation"
	sleepapneadetection = "sleep apnea detection"
	weaklysupervisedobjectlocalization = "weakly-supervised object localization"
	poemmetersclassification = "poem meters classification"
	multigrainednamedentityrecognition = "multi-grained named entity recognition"
	efficientneuralnetwork = "efficient neural network"
	sents = "sents"
	exposurecorrection = "exposure correction"
	multichoicemrc = "multi-choice mrc"
	lengthofstayprediction = "length-of-stay prediction"
	binarization = "binarization"
	opticdiscdetection = "optic disc detection"
	temporalknowledgegraphcompletion = "temporal knowledge graph completion"
	sparselearning = "sparse learning"
	imagerelighting = "image relighting"
	d3characteranimationfromasinglephoto = "3d character animation from a single photo"
	packetlossconcealment = "packet loss concealment"
	intubationsupportprediction = "intubation support prediction"
	frauddetection = "fraud detection"
	continualrelationextraction = "continual relation extraction"
	unsupervisedpartdiscovery = "unsupervised part discovery"
	linearprobeclassification = "linear-probe classification"
	singlestepretrosynthesis = "single-step retrosynthesis"
	texttospeech = "text-to-speech"
	multimodalsentimentanalysis = "multimodal sentiment analysis"
	mobilesecurity = "mobile security"
	amrtotextgeneration = "amr-to-text generation"
	targetorientedopinionwordsextraction = "target-oriented opinion words extraction"
	scenegeneration = "scene generation"
	multifuturetrajectoryprediction = "multi-future trajectory prediction"
	ecgbasedsleepstaging = "ecg based sleep staging"
	responsegeneration = "response generation"
	graphneuralnetwork = "graph neural network"
	entitydisambiguation = "entity disambiguation"
	caricature = "caricature"
	jettagging = "jet tagging"
	vietnameseaspectbasedsentimentanalysis = "vietnamese aspect-based sentiment analysis"
	transparencyseparation = "transparency separation"
	wholemammogramclassification = "whole mammogram classification"
	morphologicalinflection = "morphological inflection"
	unsupervisedpanopticsegmentation = "unsupervised panoptic segmentation"
	vehicledetection = "vehicle detection"
	ndregression = "nd regression"
	lungnodulesegmentation = "lung nodule segmentation"
	imagemanipulation = "image manipulation"
	organsegmentation = "organ segmentation"
	videoemotionrecognition = "video emotion recognition"
	d3semanticscenecompletion = "3d semantic scene completion"
	finegrainedactiondetection = "fine-grained action detection"
	multitargetdomainadaptation = "multi-target domain adaptation"
	contextawareproductrecommendation = "context aware product recommendation"
	posetransfer = "pose transfer"
	hopespeechdetection = "hope speech detection"
	aiandsafety = "ai and safety"
	oneclassclassification = "one-class classification"
	animatedgifgeneration = "animated gif generation"
	keywordspottingcss = "keyword spotting css"
	fontrecognition = "font recognition"
	oneshotpartsegmentationofgraspaffordanceinterclass = "one-shot part segmentation of grasp affordance - inter class"
	jurisprudence = "jurisprudence"
	d3reconstruction = "3d reconstruction"
	opensetclassification = "open-set classification"
	programsynthesis = "program synthesis"
	pointinteractiveimagecolorization = "point-interactive image colorization"
	electromyographyemg = "electromyography (emg)"
	citationworthiness = "citation worthiness"
	weaklysupervised3dhumanposeestimation = "weakly-supervised 3d human pose estimation"
	groundedsituationrecognition = "grounded situation recognition"
	braintumorclassification = "brain tumor classification"
	lowresourcenamedentityrecognition = "low resource named entity recognition"
	monocular3dobjectdetection10pnet30 = "monocular 3d object detection (10 / pnet-30)"
	junctiondetection = "junction detection"
	semisupervisedimageclassificationcoldstart = "semi-supervised image classification (cold start)"
	offpolicyevaluation = "off-policy evaluation"
	vietnamesenaturallanguageunderstanding = "vietnamese natural language understanding"
	weaklysupervisedpanopticsegmentation = "weakly-supervised panoptic segmentation"
	namedentityrecognition = "named entity recognition"
	d3partsegmentation = "3d part segmentation"
	personalityrecognitioninconversation = "personality recognition in conversation"
	imagetextremoval = "image text removal"
	nutrition = "nutrition"
	dnntesting = "dnn testing"
	scenegraphclassification = "scene graph classification"
	eventsegmentation = "event segmentation"
	hyperrelationalextraction = "hyper-relational extraction"
	videosuperresolution = "video super-resolution"
	d3multipersonmeshrecovery = "3d multi-person mesh recovery"
	underwaterimagerestoration = "underwater image restoration"
	ifcentityclassification = "ifc entity classification"
	rootjointlocalization = "root joint localization"
	aggressionidentification = "aggression identification"
	inputfiltering = "input filtering"
	selfsupervisedaudioclassification = "self-supervised audio classification"
	disparityestimation = "disparity estimation"
	suggestionmining = "suggestion mining"
	imagedeconvolution = "image deconvolution"
	bloodcelldetection = "blood cell detection"
	d3shapereconstructionfromasingle2dimage = "3d shape reconstruction from a single 2d image"
	facialemotionrecognition = "facial emotion recognition"
	vocalburststypeprediction = "vocal bursts type prediction"
	relationshipextractiondistantsupervised = "relationship extraction (distant supervised)"
	withinsessionssvep = "within-session ssvep"
	statechangeobjectdetection = "state change object detection"
	lanedetection = "lane detection"
	videorelationship = "video relationship"
	readmissionprediction = "readmission prediction"
	latentdiffusionmodelfor3d = "latent diffusion model for 3d"
	videosynopsis = "video synopsis"
	chunking = "chunking"
	crosslingualsemantictextualsimilarity = "cross-lingual semantic textual similarity"
	audiovisualcaptioning = "audio-visual captioning"
	volumetricmedicalimagesegmentation = "volumetric medical image segmentation"
	facehallucination = "face hallucination"
	zeroshotcrosslingualvisualnaturallanguageinference = "zero-shot cross-lingual visual natural language inference"
	openvocabularyobjectdetection = "open-vocabulary object detection"
	lowlightimageenhancement = "low-light image enhancement"
	multiplechoicequestionansweringmcqa = "multiple choice question answering (mcqa)"
	longcontextunderstanding = "long-context understanding"
	gameofpoker = "game of poker"
	textcompression = "text compression"
	jpegdecompression = "jpeg decompression"
	calvingfrontdelineationinsyntheticapertureradarimagery = "calving front delineation in synthetic aperture radar imagery"
	sentencecompletion = "sentence completion"
	selfdrivingcars = "self-driving cars"
	movierecommendation = "movie recommendation"
	beamprediction = "beam prediction"
	referringimagematting = "referring image matting"
	retrieval = "retrieval"
	spokenlanguageunderstanding = "spoken language understanding"
	dialoguesummary = "dialogue summary"
	cognitivediagnosis = "cognitive diagnosis"
	hypertensiondetection = "hypertension detection"
	aspecttermextractionandsentimentclassification = "aspect term extraction and sentiment classification"
	classificationconsistency = "classification consistency"
	monocular3dobjectdetection = "monocular 3d object detection"
	mathematicalreasoning = "mathematical reasoning"
	vehiclereidentification = "vehicle re-identification"
	medicalobjectdetection = "medical object detection"
	videoactiondetection = "video action detection"
	animalposeestimation = "animal pose estimation"
	videotoshop = "video-to-shop"
	motionsegmentation = "motion segmentation"
	unsupervisedsemanticsegmentation = "unsupervised semantic segmentation"
	finegrainedimagerecognition = "fine-grained image recognition"
	anomalydetectioninedgestreams = "anomaly detection in edge streams"
	lifetimeimagedenoising = "lifetime image denoising"
	imagerescaling = "image rescaling"
	unconditionalcrystalgeneration = "unconditional crystal generation"
	textclustering = "text clustering"
	audiodequantization = "audio dequantization"
	semanticimagetextsimilarity = "semantic image-text similarity"
	integratedsensingandcommunication = "integrated sensing and communication"
	musicperformancerendering = "music performance rendering"
	contextqueryreformulation = "context query reformulation"
	textmatching = "text matching"
	modelbasedreinforcementlearning = "model-based reinforcement learning"
	persuasivewritingstrategydetectionlevel1 = "persuasive writing strategy detection level-1"
	poseestimation = "pose estimation"
	oneshotvisualobjectsegmentation = "one-shot visual object segmentation"
	tabularregression = "tabular-regression"
	opensetlearning = "open set learning"
	politicalsalientissueorientationdetection = "political salient issue orientation detection"
	memorization = "memorization"
	persiansentimentanalysis = "persian sentiment analysis"
	twittereventdetection = "twitter event detection"
	unsuperviseddomainadaptation = "unsupervised domain adaptation"
	videodeblurring = "video deblurring"
	d3medicalimagingsegmentation = "3d medical imaging segmentation"
	wordsensedisambiguation = "word sense disambiguation"
	earlyactionprediction = "early action prediction"
	multilabelclassificationofbiomedicaltexts = "multi-label classification of biomedical texts"
	realtocartoontranslation = "real-to-cartoon translation"
	outofsighttrajectoryprediction = "out-of-sight trajectory prediction"
	sleepstaging = "sleep staging"
	triviaqa = "triviaqa"
	faciallandmarkdetection = "facial landmark detection"
	dynamicnodeclassification = "dynamic node classification"
	videoobjecttracking = "video object tracking"
	crossmodalretrieval = "cross-modal retrieval"
	musiccaptioning = "music captioning"
	trajectoryplanning = "trajectory planning"
	unsupervisedspeechrecognition = "unsupervised speech recognition"
	imagecolorization = "image colorization"
	clinicalinformationretreival = "clinical information retreival"
	atomicactionrecognition = "atomic action recognition"
	videoquestionanswering = "video question answering"
	manufacturingqualitycontrol = "manufacturing quality control"
	texturesynthesis = "texture synthesis"
	fewshotvideoquestionanswering = "few-shot video question answering"
	autonomousvehicles = "autonomous vehicles"
	stereomatching = "stereo matching"
	openaivision = "openai vision"
	derendering = "derendering"
	authorshipattribution = "authorship attribution"
	depthmapsuperresolution = "depth map super-resolution"
	voicesimilarity = "voice similarity"
	selfsupervisedpersonreidentification = "self-supervised person re-identification"
	semanticparsing = "semantic parsing"
	classincrementalobjectdetection = "class-incremental object detection"
	federatedlearning = "federated learning"
	xrayclassification = "x-ray classification"
	jobshopscheduling = "job shop scheduling"
	fewshot3dpointcloudclassification = "few-shot 3d point cloud classification"
	coversongidentification = "cover song identification"
	missingmarkersreconstruction = "missing markers reconstruction"
	classificationwithbinaryneuralnetwork = "classification with binary neural network"
	intentdetection = "intent detection"
	anchorlinkprediction = "anchor link prediction"
	speechsynthesistelugu = "speech synthesis - telugu"
	pedestriantrajectoryprediction = "pedestrian trajectory prediction"
	visualplacerecognition = "visual place recognition"
	reflectionremoval = "reflection removal"
	scriptgeneration = "script generation"
	naturallanguageunderstanding = "natural language understanding"
	trademarkretrieval = "trademark retrieval"
	chinesewordsegmentation = "chinese word segmentation"
	optimalmotionplanning = "optimal motion planning"
	skillgeneralization = "skill generalization"
	videoharmonization = "video harmonization"
	efficientexploration = "efficient exploration"
	bandgap = "band gap"
	spatiotemporalextrapolation = "spatio-temporal extrapolation"
	semanticsimilarity = "semantic similarity"
	objectcategorization = "object categorization"
	classagnosticobjectdetection = "class-agnostic object detection"
	observationcompletion = "observation completion"
	slicediscovery = "slice discovery"
	scenegraphgeneration = "scene graph generation"
	d3faceanimation = "3d face animation"
	multilingualcrossmodalretrieval = "multilingual cross-modal retrieval"
	emotionrecognitionincontext = "emotion recognition in context"
	msssim = "ms-ssim"
	birdaudiodetection = "bird audio detection"
	multivariatetimeseriesforecastingm = "multivariate time series forecastingm"
	photometricredshiftestimation = "photometric redshift estimation"
	unsupervisedpersonreidentification = "unsupervised person re-identification"
	artanalysis = "art analysis"
	kolmogorovarnoldnetworks = "kolmogorov-arnold networks"
	turningpointidentification = "turning point identification"
	voicecloning = "voice cloning"
	d3shapereconstructionfromvideos = "3d shape reconstruction from videos"
	erp = "erp"
	domainmotifprediction = "domain/motif prediction"
	formallogic = "formal logic"
	continualsemanticsegmentation = "continual semantic segmentation"
	singleview3dreconstructiononshapenet = "single-view 3d reconstruction on shapenet"
	egoschema = "egoschema"
	orientedobjectdetcti = "oriented object detcti"
	camerashotboundarydetection = "camera shot boundary detection"
	subgraphcounting2star = "subgraph counting - 2 star"
	boxsupervisedinstancesegmentation = "box-supervised instance segmentation"
	syntheticspeechdetection = "synthetic speech detection"
	audiovisualactivespeakerdetection = "audio-visual active speaker detection"
	causaldiscovery = "causal discovery"
	linkpredictionondhkgs = "link prediction on dh-kgs"
	futureprediction = "future prediction"
	eventbasedopticalflow = "event-based optical flow"
	bevsegmentation = "bev segmentation"
	autodebugging = "auto debugging"
	periodestimation = "period estimation"
	implicatures = "implicatures"
	influenceapproximation = "influence approximation"
	multifocusimagefusion = "multi focus image fusion"
	onlinemultiobjecttracking = "online multi-object tracking"
	gameofdoom = "game of doom"
	factualinconsistencydetectioninchartcaptioning = "factual inconsistency detection in chart captioning"
	facereenactment = "face reenactment"
	speechseparation = "speech separation"
	openvocabularypanoramicsemanticsegmentation = "open-vocabulary panoramic semantic segmentation"
	proceduraltextunderstanding = "procedural text understanding"
	respiratorymotionforecasting = "respiratory motion forecasting"
	multiviewclustering = "multiview clustering"
	pico = "pico"
	deblurring = "deblurring"
	selfsupervisedimageclassificationwithclip = "self-supervised image classification (with clip)"
	polishtextdiacritization = "polish text diacritization"
	trajectoryrecovery = "trajectory recovery"
	materialclassification = "material classification"
	d3instancesegmentation = "3d instance segmentation"
	neuralstylization = "neural stylization"
	skillsassessment = "skills assessment"
	vietnamesesentimentanalysis = "vietnamese sentiment analysis"
	onlinevideoanomalydetection = "online video anomaly detection"
	roibasedimagegeneration = "roi-based image generation"
	longvideoactivityrecognition = "long-video activity recognition"
	hybridmachinelearning = "hybrid machine learning"
	brainvisualreconstruction = "brain visual reconstruction"
	uncertaintyquantification = "uncertainty quantification"
	denovomoleculegenerationfrommsmsspectrumbonuschemicalformulae = "de novo molecule generation from ms/ms spectrum (bonus chemical formulae)"
	zeroshotclassificationunifiedclasses = "zero-shot classification (unified classes)"
	openintentdetection = "open intent detection"
	cellentityannotation = "cell entity annotation"
	zeroshotvideoobjectsegmentation = "zero-shot video object segmentation"
	mannerofarticulationdetection = "manner of articulation detection"
	accentedspeechrecognition = "accented speech recognition"
	breastcancerhistologyimageclassification = "breast cancer histology image classification"
	severityprediction = "severity prediction"
	oneshotsegmentation = "one-shot segmentation"
	transcriptionfactorbindingsitepredictionmouse = "transcription factor binding site prediction (mouse)"
	zeroshotlongvideobreakpointmodelquestionanswering = "zero-shot long video breakpoint-model question answering"
	lightweightdeployment = "lightweight deployment"
	referringimagemattingkeywordbased = "referring image matting (keyword-based)"
	styledetection = "style detection"
	spokendialoguesystems = "spoken dialogue systems"
	naturallanguagemomentretrieval = "natural language moment retrieval"
	sceneclassification = "scene classification"
	appearancetransfer = "appearance transfer"
	downbeattracking = "downbeat tracking"
	selfsupervisedscenetextrecognition = "self-supervised scene text recognition"
	dreamgeneration = "dream generation"
	productrelationdetection = "product relation detection"
	polynomialneuralnetworks = "polynomial neural networks"
	speechrecognition = "speech recognition"
	probabilistictimeseriesforecasting = "probabilistic time series forecasting"
	chemicalprocess = "chemical process"
	robust3dsemanticsegmentation = "robust 3d semantic segmentation"
	electronmicroscopyimagesegmentation = "electron microscopy image segmentation"
	vietnamesevisualquestionanswering = "vietnamese visual question answering"
	mutualgaze = "mutual gaze"
	commonsensereasoningforrl = "commonsense reasoning for rl"
	facialinpainting = "facial inpainting"
	onlinerankerevaluation = "online ranker evaluation"
	gazeprediction = "gaze prediction"
	abstractargumentation = "abstract argumentation"
	ethics = "ethics"
	pointcloudupsampling = "point cloud upsampling"
	smalldataimageclassification = "small data image classification"
	logicalreasoningreadingcomprehension = "logical reasoning reading comprehension"
	footkeypointdetection = "foot keypoint detection"
	stsbenchmark = "sts benchmark"
	d3awareimagesynthesis = "3d-aware image synthesis"
	videoretrieval = "video retrieval"
	cameracalibration = "camera calibration"
	zeroshotregiondescription = "zero-shot region description"
	cryptanalysis = "cryptanalysis"
	multilingualnlp = "multilingual nlp"
	ssvep = "ssvep"
	acneseveritygrading = "acne severity grading"
	shadowremoval = "shadow removal"
	liptospeechsynthesis = "lip to speech synthesis"
	humananimation = "human animation"
	sequentialcorrelationestimation = "sequential correlation estimation"
	rumourdetection = "rumour detection"
	multiobjectiveoptimization = "multiobjective optimization"
	compressedsensing = "compressed sensing"
	unsupervisedobjectdetection = "unsupervised object detection"
	unsupervisedimagetoimagetranslation = "unsupervised image-to-image translation"
	physiologicalcomputing = "physiological computing"
	clothingattributerecognition = "clothing attribute recognition"
	vietnamesespeechrecognition = "vietnamese speech recognition"
	k2 = "2k"
	proteinfolding = "protein folding"
	supervisedvideosummarization = "supervised video summarization"
	addpo = "add - po"
	unsupervisedvideosummarization = "unsupervised video summarization"
	imagebasedautomaticmeterreading = "image-based automatic meter reading"
	cwattackdetection = "cw attack detection"
	ccode = "c++ code"
	transplantingdateprediction = "transplanting date prediction"
	textguidedimageediting = "text-guided-image-editing"
	birl = "birl"
	surfacereconstruction = "surface reconstruction"
	membershipinferenceattack = "membership inference attack"
	timeseriesclassification = "time series classification"
	colorconstancy = "color constancy"
	d3geometryprediction = "3d geometry prediction"
	d3placerecognition = "3d place recognition"
	zeroshotskeletalactionrecognition = "zero shot skeletal action recognition"
	multipersonposeestimationandtracking = "multi-person pose estimation and tracking"
	channelselection = "channel selection"
	camerarelocalization = "camera relocalization"
	camouflagedobjectsegmentation = "camouflaged object segmentation"
	unsupervisedkeypointestimation = "unsupervised keypoint estimation"
	phraserelatedness = "phrase relatedness"
	resynthesis = "resynthesis"
	panoptictracking = "panoptic tracking"
	onlinereviewrating = "online review rating"
	ensemblelearning = "ensemble learning"
	gzslvideoclassification = "gzsl video classification"
	unsupervisedanomalydetectionwithspecifiedsettings20anomaly = "unsupervised anomaly detection with specified settings -- 20% anomaly"
	hopespeechdetectionfortamil = "hope speech detection for tamil"
	continuouslyindexeddomainadaptation = "continuously indexed domain adaptation"
	networkcongestioncontrol = "network congestion control"
	multipleinstancelearningmultipleinstancelearning = "multiple instance learningmultiple instance learning"
	kbtolanguagegeneration = "kb-to-language generation"
	redteaming = "red teaming"
	textannotation = "text annotation"
	videoanomalydetection = "video anomaly detection"
	dynamictexturerecognition = "dynamic texture recognition"
	businesstaxonomyconstruction = "business taxonomy construction"
	soilmoistureestimation = "soil moisture estimation"
	ecgclassification = "ecg classification"
	meetingsummarization = "meeting summarization"
	longtailvideoobjectsegmentation = "long-tail video object segmentation"
	pythoncodesynthesis = "python code synthesis"
	motioncompensation = "motion compensation"
	multilabelimageretrieval = "multi-label image retrieval"
	structuredprediction = "structured prediction"
	mhcpresentationprediction = "mhc presentation prediction"
	conversationalresponseselection = "conversational response selection"
	speechtotexttranslation = "speech-to-text translation"
	model = "model"
	geochemistry = "geochemistry"
	musiccompression = "music compression"
	rdfdatasetdiscovery = "rdf dataset discovery"
	wordattributetransfer = "word attribute transfer"
	tabularclassification = "tabular-classification"
	dreamboothpersonalizedgeneration = "dreambooth personalized generation"
	naturallanguagetransduction = "natural language transduction"
	wordsenseinduction = "word sense induction"
	overlapped1005 = "overlapped 100-5"
	mmvetv2 = "mm-vet v2"
	deen = "de-en"
	semanticshiftdetection = "semantic shift detection"
	distributedcomputing = "distributed computing"
	faultdiagnosis = "fault diagnosis"
	imagesentencealignment = "image-sentence alignment"
	documentranking = "document ranking"
	opticaltweezerssimulations = "optical tweezers simulations"
	riemannianoptimization = "riemannian optimization"
	documentlayoutanalysis = "document layout analysis"
	crimeprediction = "crime prediction"
	visualreasoning = "visual reasoning"
	sansregression = "sans regression"
	mriclassification = "mri classification"
	visualquestionanswering = "visual question answering"
	damagedtissuedetection = "damaged tissue detection"
	generating3dpointclouds = "generating 3d point clouds"
	keyphraseextraction = "keyphrase extraction"
	faceimageretrieval = "face image retrieval"
	lambada = "lambada"
	d3humanposetracking = "3d human pose tracking"
	singleimageportraitrelighting = "single-image portrait relighting"
	iotdeviceidentification = "iot device identification"
	readingcomprehensionfewshot = "reading comprehension (few-shot)"
	prediction = "prediction"
	musicrecommendation = "music recommendation"
	d3pointcloudmatching = "3d point cloud matching"
	functionalconnectivity = "functional connectivity"
	naturallanguageinference = "natural language inference"
	instancesearch = "instance search"
	trafficprediction = "traffic prediction"
	chatbot = "chatbot"
	knowledgeawarerecommendation = "knowledge-aware recommendation"
	spokenlanguageidentification = "spoken language identification"
	postagging = "pos tagging"
	clotheschangingpersonreidentification = "clothes changing person re-identification"
	inferenceattack = "inference attack"
	distractorgeneration = "distractor generation"
	codeclassification = "code classification"
	simultaneousspeechtospeechtranslation = "simultaneous speech-to-speech translation"
	audiototextretrieval = "audio to text retrieval"
	morphologyclassification = "morphology classification"
	jpegartifactremoval = "jpeg artifact removal"
	datasetdistillation = "dataset distillation"
	rhythm = "rhythm"
	amodalinstancesegmentation = "amodal instance segmentation"
	imageguidedstoryendinggeneration = "image-guided story ending generation"
	videoforecasting = "video forecasting"
	multimodaldocumentclassification = "multi-modal document classification"
	transductivelearning = "transductive learning"
	scenetextediting = "scene text editing"
	collegecomputerscience = "college computer science"
	automaticmachinelearningmodelselection = "automatic machine learning model selection"
	image1stitching22 = "1 image, 2*2 stitching"
	distantspeechrecognition = "distant speech recognition"
	boundarydetection = "boundary detection"
	wordspottinginhandwrittendocuments = "word spotting in handwritten documents"
	modelcompression = "model compression"
	featurecorrelation = "feature correlation"
	stanceclassification = "stance classification"
	pointcloudreconstruction = "point cloud reconstruction"
	lesionclassification = "lesion classification"
	unsupervised3dsemanticsegmentation = "unsupervised 3d semantic segmentation"
	braintumorsegmentation = "brain tumor segmentation"
	openintentdiscovery = "open intent discovery"
	textsimplification = "text simplification"
	continuedfraction = "continued fraction"
	pokerhandclassification = "poker hand classification"
	compileroptimization = "compiler optimization"
	kgtotextgeneration = "kg-to-text generation"
	solarflareprediction = "solar flare prediction"
	lungcancerdiagnosis = "lung cancer diagnosis"
	unfairnessdetection = "unfairness detection"
	subgraphcountingc4 = "subgraph counting - c4"
	playingthegameof2048 = "playing the game of 2048"
	actionrecognitioninstillimages = "action recognition in still images"
	zeroshotaudioclassification = "zero-shot audio classification"
	tropicalcycloneintensityforecasting = "tropical cyclone intensity forecasting"
	unsupervisedobjectsegmentation = "unsupervised object segmentation"
	informationplane = "information plane"
	celldetection = "cell detection"
	collaborativeranking = "collaborative ranking"
	nlgevaluation = "nlg evaluation"
	senter = "senter"
	multiview3dshaperetrieval = "multi-view 3d shape retrieval"
	stockmarketprediction = "stock market prediction"
	winogrande = "winogrande"
	dataaugmentation = "data augmentation"
	localcommunitydetection = "local community detection"
	professionalmedicine = "professional medicine"
	correlatedtimeseriesforecasting = "correlated time series forecasting"
	continualpanopticsegmentation = "continual panoptic segmentation"
	meterreading = "meter reading"
	dialogueevaluation = "dialogue evaluation"
	classification = "classification"
	outofdistributiondetection = "out-of-distribution detection"
	logparsing = "log parsing"
	multimodalunsupervisedimagetoimagetranslation = "multimodal unsupervised image-to-image translation"
	personalitygeneration = "personality generation"
	understandingfables = "understanding fables"
	faceidentification = "face identification"
	oneshotpartsegmentationofscoopaffordanceintraclass = "one-shot part segmentation of scoop affordance - intra class"
	roaddamagedetection = "road damage detection"
	universalnavigation = "universal navigation"
	fsmevqa = "fs-mevqa"
	singleview3dreconstruction = "single-view 3d reconstruction"
	gpsembeddings = "gps embeddings"
	wordsimilarity = "word similarity"
	singleimagehazeremoval = "single image haze removal"
	causallanguagemodeling = "causal language modeling"
	wrldsleepstaging = "w-r-l-d sleep staging"
	imagedenoising = "image denoising"
	multiagentreinforcementlearning = "multi-agent reinforcement learning"
	unsupervisedspatialclustering = "unsupervised spatial clustering"
	withinsessionmotorimagerylefthandvsrighthand = "within-session motor imagery (left hand vs. right hand)"
	yieldmappinginappleorchards = "yield mapping in apple orchards"
	readingcomprehensionzeroshot = "reading comprehension (zero-shot)"
	splitandrephrase = "split and rephrase"
	d3surfacegeneration = "3d surface generation"
	noveltydetection = "novelty detection"
	classlevelcodegeneration = "class-level code generation"
	multihumanparsing = "multi-human parsing"
	documentleveleventextraction = "document-level event extraction"
	audiomomentretrieval = "audio moment retrieval"
	codecommentgeneration = "code comment generation"
	crossdomainirispresentationattackdetection = "cross-domain iris presentation attack detection"
	templatematching = "template matching"
	zeroshotcrossmodalretrieval = "zero-shot cross-modal retrieval"
	scientificdocumentsummarization = "scientific document summarization"
	referenceexpressiongeneration = "reference expression generation"
	logsolubility = "log solubility"
	imagetovideogeneration = "image to video generation"
	svbrdfestimation = "svbrdf estimation"
	lvsegmentation = "lv segmentation"
	unsupervisedtextclassification = "unsupervised text classification"
	xraypdfregression = "x-ray pdf regression"
	actiondetection = "action detection"
	d3openvocabularyinstancesegmentation = "3d open-vocabulary instance segmentation"
	d6poseestimationusingrgb = "6d pose estimation using rgb"
	multiviewgeolocalisation = "multi-view geo-localisation"
	entitycrossdocumentcoreferenceresolution = "entity cross-document coreference resolution"
	trainegopathdetection = "train ego-path detection"
	graphmatching = "graph matching"
	slotfilling = "slot filling"
	marketing = "marketing"
	saliencyprediction = "saliency prediction"
	multimodalreasoning = "multimodal reasoning"
	objectdetectioninindoorscenes = "object detection in indoor scenes"
	generativeadversarialnetwork = "generative adversarial network"
	editcompletion = "editcompletion"
	microexpressionspotting = "micro-expression spotting"
	d3multipersonposeestimationabsolute = "3d multi-person pose estimation (absolute)"
	molecularpropertyprediction = "molecular property prediction"
	zeroshotsceneclassificationunifiedclasses = "zero-shot scene classification (unified classes)"
	multiarmedbandits = "multi-armed bandits"
	unconstrainedlipsynchronization = "unconstrained lip-synchronization"
	timeintervalprediction = "time-interval prediction"
	sketch = "sketch"
	multilingualtexttoimagegeneration = "multilingual text-to-image generation"
	humanscenecontactdetection = "human-scene contact detection"
	tableretrieval = "table retrieval"
	conditionaltextgeneration = "conditional text generation"
	voiceantispoofing = "voice anti-spoofing"
	deepimbalancedregression = "deep imbalanced regression"
	aspectbasedsentimentanalysisabsa = "aspect-based sentiment analysis (absa)"
	anomalyclassification = "anomaly classification"
	onlyconnectwallsdatasettask2connections = "only connect walls dataset task 2 (connections)"
	classincrementallearning = "class incremental learning"
	adversarialattack = "adversarial attack"
	humanobjectinteractionmotiontracking = "human-object-interaction motion tracking"
	predictiveprocessmonitoring = "predictive process monitoring"
	speakeridentification = "speaker identification"
	tropicalcyclonetrackforecasting = "tropical cyclone track forecasting"
	corpusvideomomentretrieval = "corpus video moment retrieval"
	masssegmentationfrommammograms = "mass segmentation from mammograms"
	incrementallearning = "incremental learning"
	zeroshotcounting = "zero-shot counting"
	collaborativefiltering = "collaborative filtering"
	interpretablemachinelearning = "interpretable machine learning"
	timeseriesfewshotlearningwithheterogeneouschannels = "time-series few-shot learning with heterogeneous channels"
	visionlanguagesegmentation = "vision-language segmentation"
	fewshotimageclassification = "few-shot image classification"
	eegdecoding = "eeg decoding"
	imagetosketchrecognition = "image to sketch recognition"
	imageenhancement = "image enhancement"
	sourcefreeobjectdetection = "source free object detection"
	texttomusicgeneration = "text-to-music generation"
	stereoimagesuperresolution = "stereo image super-resolution"
	videoframeinterpolation = "video frame interpolation"
	d2objectdetection = "2d object detection"
	d5egradationblindallinoneimagerestoration = "5-degradation blind all-in-one image restoration"
	keydetection = "key detection"
	psoconvnetsdynamics2 = "pso-convnets dynamics 2"
	smc = "smc会议"
	computersecurity = "computer security"
	subgraphcountingchordalc4 = "subgraph counting - chordal c4"
	entityretrieval = "entity retrieval"
	heartrateestimation = "heart rate estimation"
	timbreinterpolation = "timbre interpolation"
	populationmapping = "population mapping"
	faceantispoofing = "face anti-spoofing"
	activeobjectdetection = "active object detection"
	semanticframeparsing = "semantic frame parsing"
	hiddenaspectdetection = "hidden aspect detection"
	audiodeclipping = "audio declipping"
	videoadverbretrieval = "video-adverb retrieval"
	ssim = "ssim"
	electricalengineering = "electrical engineering"
	acousticnoveltydetection = "acoustic novelty detection"
	activespeakerdetection = "active speaker detection"
	fewshotclassincrementallearning = "few-shot class-incremental learning"
	changedatageneration = "change data generation"
	disguisedfaceverification = "disguised face verification"
	learningwithcoarselabels = "learning with coarse labels"
	hateintensityprediction = "hate intensity prediction"
	spacefortress = "space fortress"
	adroidhammerhuman = "adroid hammer-human"
	factselection = "fact selection"
	videobasedgenerativeperformancebenchmarkingcontextualunderstanding = "video-based generative performance benchmarking (contextual understanding)"
	cellsegmentation = "cell segmentation"
	mme = "mme"
	medqa = "medqa"
	superresolution = "super-resolution"
	matrixcompletion = "matrix completion"
	d2semanticsegmentation = "2d semantic segmentation"
	eventrelationextraction = "event relation extraction"
	gazeredirection = "gaze redirection"
	genderprediction = "gender prediction"
	unsupervised3dhumanposeestimation = "unsupervised 3d human pose estimation"
	multilingualimagetextclassification = "multilingual image-text classification"
	realtimeinstancesegmentation = "real-time instance segmentation"
	interactive3dinstancesegmentation = "interactive 3d instance segmentation"
	extractingcovid19eventsfromtwitter = "extracting covid-19 events from twitter"
	focusnewstest = "focusnews (test)"
	mujocogames = "mujoco games"
	unsupervisedvideoframeinterpolation = "unsupervised video frame interpolation"
	videotextretrieval = "video-text retrieval"
	programrepair = "program repair"
	longtaillearningoncifar10ltρ100 = "long-tail learning on cifar-10-lt (ρ=100)"
	texttovideoretrieval = "text to video retrieval"
	blinkestimation = "blink estimation"
	lowrankmatrixcompletion = "low-rank matrix completion"
	videopolypsegmentation = "video polyp segmentation"
	maximumseparation = "maximum separation"
	documenttranslation = "document translation"
	cognateprediction = "cognate prediction"
	extractivesummarization = "extractive summarization"
	visualobjecttracking = "visual object tracking"
	crossviewimagetoimagetranslation = "cross-view image-to-image translation"
	concepttotextgeneration = "concept-to-text generation"
	facemorphingattackdetection = "face morphing attack detection"
	ordinalclassification = "ordinal classification"
	cameralocalization = "camera localization"
	newsannotation = "news annotation"
	oneshotpartsegmentationofcontainaffordanceintraclass = "one-shot part segmentation of contain affordance - intra class"
	pianomusicmodeling = "piano music modeling"
	zeroshotvideoaudioretrieval = "zero-shot video-audio retrieval"
	abstractgeneration = "abstract generation"
	citationvisualization = "citation visualization"
	fakevoicedetection = "fake voice detection"
	overlapped53 = "overlapped 5-3"
	depthcompletion = "depth completion"
	endtoenddialoguemodelling = "end-to-end dialogue modelling"
	graphstructurelearning = "graph structure learning"
	d3semanticinstancesegmentation = "3d semantic instance segmentation"
	coderepair = "code repair"
	solitaire = "solitaire"
	resultaggregation = "result aggregation"
	remainingusefullifetimeestimation = "remaining useful lifetime estimation"
	variablemisuse = "variable misuse"
	multimodalknowledgegraph = "multi-modal knowledge graph"
	imagecompression = "image compression"
	subgraphcounting = "subgraph counting"
	depthaleatoricuncertaintyestimation = "depth aleatoric uncertainty estimation"
	multilabelimagerecognitionwithpartiallabels = "multi-label image recognition with partial labels"
	depthanomalydetectionandsegmentation = "depth anomaly detection and segmentation"
	formalitystyletransfer = "formality style transfer"
	cloudcomputing = "cloud computing"
	videocaptioningonmsrvtt = "video captioning on msr-vtt"
	empiricaljudgments = "empirical judgments"
	visualsentimentprediction = "visual sentiment prediction"
	privacypreserving = "privacy preserving"
	adversarialtext = "adversarial text"
	automatedpancreassegmentation = "automated pancreas segmentation"
	irissegmentation = "iris segmentation"
	survey = "survey"
	operatorlearning = "operator learning"
	cracksegmentation = "crack segmentation"
	unsupervisedvehiclereidentification = "unsupervised vehicle re-identification"
	d3geometry = "3d geometry"
	rgbtsalientobjectdetection = "rgb-t salient object detection"
	lowlightpedestriandetection = "low-light pedestrian detection"
	contrastivelearning = "contrastive learning"
	readingorderdetection = "reading order detection"
	fantasyreasoning = "fantasy reasoning"
	curvedtextdetection = "curved text detection"
	speakerprofiling = "speaker profiling"
	d3facealignment = "3d face alignment"
	malwaredetection = "malware detection"
	multivariatetimeseriesforecasting = "multivariate time series forecasting"
	rootcauseranking = "root cause ranking"
	targetsoundextraction = "target sound extraction"
	methodnameprediction = "method name prediction"
	composedimageretrievalcoir = "composed image retrieval (coir)"
	colornormalization = "color normalization"
	moralpermissibility = "moral permissibility"
	anomalydetection = "anomaly detection"
	medicalimageanalysis = "medical image analysis"
	weaklysuperviseddefectdetection = "weakly supervised defect detection"
	medicalprocedure = "medical procedure"
	darkhumordetection = "dark humor detection"
	presuppositionsasnli = "presuppositions as nli"
	vietnamesetextdiacritization = "vietnamese text diacritization"
	deepseatreasure = "deep-sea treasure, image version"
	singlecellmodeling = "single-cell modeling"
	actindetection = "actin detection"
	heterogeneousnodeclassification = "heterogeneous node classification"
	sleepspindlesdetection = "sleep spindles detection"
	phenotypeclassification = "phenotype classification"
	insuranceprediction = "insurance prediction"
	unsupervisedclotheschangingpersonreidentification = "unsupervised clothes changing person re-identification"
	indoorscenereconstruction = "indoor scene reconstruction"
	linearprobingobjectlevel3dawareness = "linear probing object-level 3d awareness"
	multipleobjecttrackandsegmentation = "multiple object track and segmentation"
	microvideorecommendations = "micro-video recommendations"
	radiomicsbasedclassification = "radiomics-based classification"
	sampleprobing = "sample probing"
	textcategorization = "text categorization"
	semisupervisedpersonboundingboxdetection = "semi-supervised person bounding box detection"
	automl = "automl"
	fewshotageestimation = "few-shot age estimation"
	exemplarfreecounting = "exemplar-free counting"
	latviantextdiacritization = "latvian text diacritization"
	questionrewriting = "question rewriting"
	hopespeechdetectionformalayalam = "hope speech detection for malayalam"
	lesiondetection = "lesion detection"
	metalartifactreduction = "metal artifact reduction"
	clinicallanguagetranslation = "clinical language translation"
	negationandspeculationcuedetection = "negation and speculation cue detection"
	imagegenerationfromscenegraphs = "image generation from scene graphs"
	knowledgeediting = "knowledge editing"
	partialvideocopydetection = "partial video copy detection"
	continuousobjectrecognition = "continuous object recognition"
	reinforcementlearning = "reinforcement learning"
	generativevisualquestionanswering = "generative visual question answering"
	sketchrecognition = "sketch recognition"
	splitmnist = "split-mnist"
	syntheticfacerecognition = "synthetic face recognition"
	valueprediction = "value prediction"
	icumortality = "icu mortality"
	clinicalassertionstatusdetection = "clinical assertion status detection"
	N3940 = "陕西、四川、重庆地区2017-2022年末人口数（如图1-7所示）中，陕西省人口数较稳 定，维持在3940万人，四川和重庆均呈上升趋势。"
	misinformation = "misinformation"
	dichotomousimagesegmentation = "dichotomous image segmentation"
	handwritingverification = "handwriting verification"
	querywellformedness = "query wellformedness"
	fewshotlearningfewshotimageclassification = "few-shot learningfew-shot image classification"
	hatespeechnormalization = "hate speech normalization"
	spo2estimation = "spo2 estimation"
	markerlessmotioncapture = "markerless motion capture"
	abstractivetextsummarization = "abstractive text summarization"
	generalizedfewshotlearning = "generalized few-shot learning"
	pillclassificationbothsides = "pill classification (both sides)"
	pointprocesses = "point processes"
	continuousaffectestimation = "continuous affect estimation"
	thaiwordsegmentation = "thai word segmentation"
	placentasegmentation = "placenta segmentation"
	multimodalabstractivetextsummarization = "multimodal abstractive text summarization"
	noisyquantumcircuitclassificationquantumml = "noisy quantum circuit classification (quantum ml, error mitigation)"
	motionestimation = "motion estimation"
	eventbasedmotionestimation = "event-based motion estimation"
	roadsceneunderstanding = "road scene understanding"
	humanobjectinteractiongeneration = "human-object interaction generation"
	visualtextcorrection = "visual text correction"
	acousticechocancellation = "acoustic echo cancellation"
	deceptiondetection = "deception detection"
	weaklysupervisedvideoanomalydetection = "weakly-supervised video anomaly detection"
	bilinguallexiconinduction = "bilingual lexicon induction"
	breastcancerhistologyimageclassification20labels = "breast cancer histology image classification (20% labels)"
	portraitanimation = "portrait animation"
	d3shaperecognition = "3d shape recognition"
	polyphonedisambiguation = "polyphone disambiguation"
	colorization = "colorization"
	spatialreasoning = "spatial reasoning"
	contextualisedwordrepresentations = "contextualised word representations"
	neuralnetworksecurity = "neural network security"
	glossfreesignlanguagetranslation = "gloss-free sign language translation"
	knowledgebasecompletion = "knowledge base completion"
	tableextraction = "table extraction"
	multiwordexpressionembedding = "multi-word expression embedding"
	interactiverecommendation = "interactive recommendation"
	strategyqa = "strategyqa"
	audiosynthesis = "audio synthesis"
	coherenceevaluation = "coherence evaluation"
	crossmodalretrievalwithnoisycorrespondence = "cross-modal retrieval with noisy correspondence"
	crossmodalitypersonreidentification = "cross-modality person re-identification"
	unsupervisedkgtotextgeneration = "unsupervised kg-to-text generation"
	abusivelanguage = "abusive language"
	cubeengravingclassification = "cube engraving classification"
	faceclustering = "face clustering"
	selflearning = "self-learning"
	reasoningsegmentation = "reasoning segmentation"
	pointgoalnavigation = "pointgoal navigation"
	zeroshotlongvideobreakpointmodequestionanswering = "zero-shot long video breakpoint-mode question answering"
	unsupervisedanomalydetectioninsound = "unsupervised anomaly detection in sound"
	openworldvideosegmentation = "open-world video segmentation"
	molecularpropertyprediction1shot = "molecular property prediction (1-shot))"
	unsupervisedfaciallandmarkdetection = "unsupervised facial landmark detection"
	semanticpartdetection = "semantic part detection"
	zeroshoteventextraction = "zero-shot event extraction"
	subgraphcountingk4 = "subgraph counting - k4"
	earthobservation = "earth observation"
	d3carinstanceunderstanding = "3d car instance understanding"
	multilabelimageclassification = "multi-label image classification"
	zeroshotcrosslingualtransfer = "zero-shot cross-lingual transfer"
	phonemerecognition = "phoneme recognition"
	learningnetworkrepresentations = "learning network representations"
	transparentobjects = "transparent objects"
	airpollutionprediction = "air pollution prediction"
	exposurefairness = "exposure fairness"
	texttovideosearch = "text-to-video search"
	multipleobjecttracking = "multiple object tracking"
	d3objectretrieval = "3d object retrieval"
	supervisedtextretrieval = "supervised text retrieval"
	sentenceordering = "sentence ordering"
	unsupervisedanomalydetectionwithspecifiedsettings01anomaly = "unsupervised anomaly detection with specified settings -- 0.1% anomaly"
	contentbasedimageretrieval = "content-based image retrieval"
	sokoban = "sokoban"
	robustfacealignment = "robust face alignment"
	facialbeautyprediction = "facial beauty prediction"
	zeroshotcrosslingualdocumentclassification = "zero-shot cross-lingual document classification"
	ner = "ner"
	buildingdamageassessment = "building damage assessment"
	roboticgrasping = "robotic grasping"
	timeseriesregression = "time series regression"
	authorshipverification = "authorship verification"
	handposeestimation = "hand pose estimation"
	summarizationconsistencyevaluation = "summarization consistency evaluation"
	dominanceestimation = "dominance estimation"
	multipleobjecttrackingwithtransformer = "multiple object tracking with transformer"
	definitionextraction = "definition extraction"
	pointcloudsemanticcompletion = "point cloud semantic completion"
	posebasedhumaninstancesegmentation = "pose-based human instance segmentation"
	actionrecognitiononhmdb51 = "action recognition on hmdb-51"
	fontstyletransfer = "font style transfer"
	combinatorialoptimization = "combinatorial optimization"
	pupildetection = "pupil detection"
	spinalcordgraymattersegmentation = "spinal cord gray matter - segmentation"
	emotionclassification = "emotion classification"
	mrpc = "mrpc"
	unsupervisedfewshotlearning = "unsupervised few-shot learning"
	acrobot = "acrobot"
	friction = "friction"
	crosslingualbitextmining = "cross-lingual bitext mining"
	fashionsynthesis = "fashion synthesis"
	ecgqrsdetection = "ecg qrs detection"
	pollgeneration = "poll generation"
	breastmasssegmentationinwholemammograms = "breast mass segmentation in whole mammograms"
	clinicalsectionidentification = "clinical section identification"
	spacegroupclassification = "space group classification"
	explanationgeneration = "explanation generation"
	multilingualtextclassification = "multilingual text classification"
	zeroshottextretrieval = "zero-shot text retrieval"
	toxiccommentclassification = "toxic comment classification"
	crowdcounting = "crowd counting"
	papergeneration = "paper generation"
	nucleiclassification = "nuclei classification"
	virtualtryonstreet2street = "virtual try-on (street2street)"
	pulmonarynodulesclassification = "pulmonary nodules classification"
	modelpredictivecontrol = "model predictive control"
	k4 = "4k"
	fashionunderstanding = "fashion understanding"
	removepq = "remove - pq"
	hyperspectralimagebasedfruitripenessprediction = "hyperspectral image-based fruit ripeness prediction"
	classifymurmurs = "classify murmurs"
	robustsemisupervisedrgbdsemanticsegmentation = "robust semi-supervised rgbd semantic segmentation"
	multimodelguidedimageediting = "multimodel-guided image editing"
	entityalignment = "entity alignment"
	zeroshotnamedentityrecognitionner = "zero-shot named entity recognition (ner)"
	texttosql = "text-to-sql"
	d3objectdetectionfrommonocularimages = "3d object detection from monocular images"
	graspcontactprediction = "grasp contact prediction"
	textattributetransfer = "text attribute transfer"
	emergentcommunicationsonrelations = "emergent communications on relations"
	entityresolution = "entity resolution"
	classifiercalibration = "classifier calibration"
	signlanguageproduction = "sign language production"
	eventcausalityidentification = "event causality identification"
	probabilisticdeeplearning = "probabilistic deep learning"
	visualtracking = "visual tracking"
	medicalimagedenoising = "medical image denoising"
	grammaticalerrordetection = "grammatical error detection"
	brdfestimation = "brdf estimation"
	multipleaffordancedetection = "multiple affordance detection"
	propagandadetection = "propaganda detection"
	vocalensembleseparation = "vocal ensemble separation"
	oneshotpartsegmentationofpoundaffordanceinterclass = "one-shot part segmentation of pound affordance - inter class"
	navigate = "navigate"
	facegeneration = "face generation"
	modalitycompletion = "modality completion"
	multimodaldialoguegeneration = "multi-modal dialogue generation"
	sequentialplacelearning = "sequential place learning"
	predictionofoccupancygridmaps = "prediction of occupancy grid maps"
	management = "management"
	sequentialrecommendation = "sequential recommendation"
	attentionscoreprediction = "attention score prediction"
	visualkeywordspotting = "visual keyword spotting"
	alsdetection = "als detection"
	scenechangedetection = "scene change detection"
	denovomoleculegenerationfrommsmsspectrum = "de novo molecule generation from ms/ms spectrum"
	zeroshotrelationclassification = "zero-shot relation classification"
	papergenerationtitletoabstract = "paper generation (title-to-abstract)"
	denoising = "denoising"
	explainablerecommendation = "explainable recommendation"
	semisupervisedhumanposeestimation = "semi-supervised human pose estimation"
	selfsupervisedsoundclassification = "self-supervised sound classification"
	wrnsleepstaging = "w-r-n sleep staging"
	medicalimagedetection = "medical image detection"
	imagetopointcloudregistration = "image to point cloud registration"
	promptlearning = "prompt learning"
	passageretrieval = "passage retrieval"
	czechtextdiacritization = "czech text diacritization"
	relationlinking = "relation linking"
	temporalrelationclassification = "temporal relation classification"
	chinesesentencepairclassification = "chinese sentence pair classification"
	highlightremoval = "highlight removal"
	semisupervisedvideoclassification = "semi-supervised video classification"
	lineartcolorization = "line art colorization"
	speakeranonymization = "speaker anonymization"
	object = "object"
	physicalsimulations = "physical simulations"
	rainremoval = "rain removal"
	hyperspectralimageclassification = "hyperspectral image classification"
	onlinesurgicalphaserecognition = "online surgical phase recognition"
	faceanonymization = "face anonymization"
	commonsensereasoningfewshot = "common sense reasoning (few-shot)"
	medicalrelationextraction = "medical relation extraction"
	transcriptionfactorbindingsiteprediction = "transcription factor binding site prediction"
	videoenhancement = "video enhancement"
	licenseplatedetection = "license plate detection"
	d3multiobjecttracking = "3d multi-object tracking"
	modelpoisoning = "model poisoning"
	questionanswergeneration = "question-answer-generation"
	enfelectricnetworkfrequencyextraction = "enf (electric network frequency) extraction"
	holdoutset = "holdout set"
	d3objectsuperresolution = "3d object super-resolution"
	oneshotpartsegmentationofcutaffordanceinterclass = "one-shot part segmentation of cut affordance - inter class"
	pointsupervisedinstancesegmentation = "point-supervised instance segmentation"
	provableadversarialdefense = "provable adversarial defense"
	lungsoundclassification = "lung sound classification"
	tripleclassification = "triple classification"
	posebasedanomalydetection = "pose-based anomaly detection"
	hyperspectralsemanticsegmentation = "hyperspectral semantic segmentation"
	humaneval = "humaneval"
	driverattentionmonitoring = "driver attention monitoring"
	visibilityestimationfrompointcloud = "visibility estimation from point cloud"
	longtailedobjectdetection = "long-tailed object detection"
	gameoffootball = "game of football"
	gitcommitmessagegeneration = "git commit message generation"
	d2semanticsegmentationtask18classes = "2d semantic segmentation task 1 (8 classes)"
	pointcloudclassificationdataset = "point cloud classification dataset"
	speechlanguageidentification = "speech language identification"
	vehiclekeypointandorientationestimation = "vehicle key-point and orientation estimation"
	instructionfollowing = "instruction following"
	materialsegmentation = "material segmentation"
	contourdetection = "contour detection"
	imageclassidataaugmentationfication = "image classidata augmentationfication"
	ontologysubsumptioninferece = "ontology subsumption inferece"
	speechemotionrecognition = "speech emotion recognition"
	intrusiondetection = "intrusion detection"
	equilibriumtrafficassignment = "equilibrium traffic assignment"
	computationalphenotyping = "computational phenotyping"
	splicesiteprediction = "splice site prediction"
	hypernymdiscovery = "hypernym discovery"
	temporalviewsynthesis = "temporal view synthesis"
	finegrainedvehicleclassification = "fine-grained vehicle classification"
	zeroshottextsearch = "zero-shot text search"
	devicecloudcollaboration = "device-cloud collaboration"
	philosophy = "philosophy"
	airbubblesdetection = "airbubbles detection"
	medicalreportgeneration = "medical report generation"
	metriclearning = "metric learning"
	adhocinformationretrieval = "ad-hoc information retrieval"
	selfsupervisedsceneflowestimation = "self-supervised scene flow estimation"
	openvocabularykeypointdetection = "open vocabulary keypoint detection"
	linksignprediction = "link sign prediction"
	parameterprediction = "parameter prediction"
	imagemanipulationlocalization = "image manipulation localization"
	papergenerationabstracttoconclusion = "paper generation (abstract-to-conclusion)"
	referringexpressiongeneration = "referring expression generation"
	multiobjectcolocalization = "multi-object colocalization"
	truthfulqa = "truthfulqa"
	d2humanposeestimation = "2d human pose estimation"
	glinear = "glinear"
	generalizedzeroshotlearningunseen = "generalized zero-shot learning - unseen"
	socialcueforecasting = "social cue forecasting"
	statespacemodels = "state space models"
	novellidarviewsynthesis = "novel lidar view synthesis"
	supervisedimageretrieval = "supervised image retrieval"
	generalizedreferringexpressioncomprehension = "generalized referring expression comprehension"
	dialogrelationextraction = "dialog relation extraction"
	fewshotimitationlearning = "few-shot imitation learning"
	relationalcaptioning = "relational captioning"
	robust3dobjectdetection = "robust 3d object detection"
	imagecompressedsensing = "image compressed sensing"
	deformableobjectmanipulation = "deformable object manipulation"
	lexicalcomplexityprediction = "lexical complexity prediction"
	promoterdetection = "promoter detection"
	brainmorphometry = "brain morphometry"
	classificationofvariablestars = "classification of variable stars"
	zeroshotgeneralization = "zero-shot generalization"
	jointdemosaicinganddenoising = "joint demosaicing and denoising"
	adroidpenhuman = "adroid pen-human"
	objectstatechangeclassificationonego4d = "object state change classification on ego4d"
	nonintrusiveloadmonitoring = "non-intrusive load monitoring"
	temporalforgerylocalization = "temporal forgery localization"
	modelselection = "model selection"
	spectralgraphclustering = "spectral graph clustering"
	localcolorenhancement = "local color enhancement"
	moleculecaptioning = "molecule captioning"
	setmatching = "set matching"
	oneshotpartsegmentationofsupportaffordanceinterclass = "one-shot part segmentation of support affordance - inter class"
	timeseriesdenoising = "time series denoising"
	sportsunderstanding = "sports understanding"
	semisupervisedtextclassification = "semi-supervised text classification"
	incongruitydetection = "incongruity detection"
	objectrearrangement = "object rearrangement"
	humanjudgmentclassification = "human judgment classification"
	graphoutlierdetection = "graph outlier detection"
	binaryclassification = "binary classification"
	audiosourceseparation = "audio source separation"
	dynamiccommunitydetection = "dynamic community detection"
	constrainedclustering = "constrained clustering"
	extremesummarization = "extreme summarization"
	environmentalsoundclassification = "environmental sound classification"
	continualnamedentityrecognition = "continual named entity recognition"
	videotovideosynthesis = "video-to-video synthesis"
	spatialtokenmixer = "spatial token mixer"
	conversationalwebnavigation = "conversational web navigation"
	humandetectionofdeepfakes = "human detection of deepfakes"
	sequentialpatternmining = "sequential pattern mining"
	dialoguestatetracking = "dialogue state tracking"
	reversestyletransfer = "reverse style transfer"
	d3shapereconstruction = "3d shape reconstruction"
	objectdiscovery = "object discovery"
	zeroshotonbeirinferencefreemodel = "zero shot on beir (inference free model)"
	ifun = "ifun"
	nonadversarialrobustness = "non-adversarial robustness"
	clinicalconceptextraction = "clinical concept extraction"
	anomalyinstancesegmentation = "anomaly instance segmentation"
	pedestrianattributerecognition = "pedestrian attribute recognition"
	crosslingualabstractivesummarization = "cross-lingual abstractive summarization"
	topologicaldataanalysis = "topological data analysis"
	modeloptimization = "model optimization"
	unconditionalvideogeneration = "unconditional video generation"
	sketchtoimagetranslation = "sketch-to-image translation"
	multilingualnamedentityrecognition = "multilingual named entity recognition"
	multiobjecttracking = "multi-object tracking"
	dialectidentification = "dialect identification"
	partiallyobservablereinforcementlearning = "partially observable reinforcement learning"
	profilegeneration = "profile generation"
	uncertaintyvisualization = "uncertainty visualization"
	gaussianprocesses = "gaussian processes"
	d3parameterefficientfinetuningforclassification = "3d parameter-efficient fine-tuning for classification"
	crosslingualzeroshotdependencyparsing = "cross-lingual zero-shot dependency parsing"
	videonarrativegrounding = "video narrative grounding"
	leftatriumsegmentation = "left atrium segmentation"
	diabeticfootulcerdetection = "diabetic foot ulcer detection"
	semanticimagesimilarity = "semantic image similarity"
	languagemodelevaluation = "language model evaluation"
	vocalburstsintensityprediction = "vocal bursts intensity prediction"
	infraredandvisibleimagefusion = "infrared and visible image fusion"
	garmentreconstruction = "garment reconstruction"
	semisupervisedobjectdetection = "semi-supervised object detection"
	medicalwaveformanalysis = "medical waveform analysis"
	semisupervisedtimeseriesclassification = "semi-supervised time series classification"
	storyvisualization = "story visualization"
	radarobjectdetection = "radar object detection"
	openinformationextraction = "open information extraction"
	naturallanguageinferencefewshot = "natural language inference (few-shot)"
	clonedetection = "clone detection"
	weatherforecasting = "weather forecasting"
	visionbasednavigationwithlanguagebasedassistance = "vision-based navigation with language-based assistance"
	starcraftii = "starcraft ii"
	sentencerewriting = "sentence rewriting"
	survivalanalysis = "survival analysis"
	explanatoryvisualquestionanswering = "explanatory visual question answering"
	probinglanguagemodels = "probing language models"
	humanparsing = "human parsing"
	irregulartextrecognition = "irregular text recognition"
	graphregression = "graph regression"
	indoormonoculardepthestimation = "indoor monocular depth estimation"
	largescalepersonreidentification = "large-scale person re-identification"
	interpretabilitytechniquesfordeeplearning = "interpretability techniques for deep learning"
	sleepquality = "sleep quality"
	explainableartificialintelligencexai = "explainable artificial intelligence (xai)"
	fakesongdetection = "fake song detection"
	reranking = "re-ranking"
	vnla = "vnla"
	explainableartificialintelligence = "explainable artificial intelligence"
	printedtextrecognition = "printed text recognition"
	disjoint191 = "disjoint 19-1"
	moleculardocking = "molecular docking"
	describedobjectdetection = "described object detection"
	d4reconstruction = "4d reconstruction"
	problemdecomposition = "problem decomposition"
	deepfakedetection = "deepfake detection"
	cervicalnucleusdetection = "cervical nucleus detection"
	d3pointcloudclassification = "3d point cloud classification"
	videorestoration = "video restoration"
	ontologyembedding = "ontology embedding"
	unsupervisedmachinetranslation = "unsupervised machine translation"
	imagecategorization = "image categorization"
	cloudremoval = "cloud removal"
	documentshadowremoval = "document shadow removal"
	storycontinuation = "story continuation"
	monocular3dobjectdetection10nyu37 = "monocular 3d object detection (10 / nyu-37)"
	groupanomalydetection = "group anomaly detection"
	multimodalrecommendation = "multi-modal recommendation"
	sequentialdiagnosis = "sequential diagnosis"
	atom3dbenchmark = "atom3d benchmark"
	audiodenoising = "audio denoising"
	activationfunctionsynthesis = "activation function synthesis"
	d3faciallandmarklocalization = "3d facial landmark localization"
	superviseddefectdetection = "supervised defect detection"
	keywordspotting = "keyword spotting"
	realtimeobjectdetection = "real-time object detection"
	depthawarevideopanopticsegmentation = "depth-aware video panoptic segmentation"
	musictagging = "music tagging"
	medicalimageenhancement = "medical image enhancement"
	naturalimageorientationangledetection = "natural image orientation angle detection"
	skeletonbasedactionrecognition = "skeleton based action recognition"
	camouflagedobjectsegmentationwithasingletaskgenericprompt = "camouflaged object segmentation with a single task-generic prompt"
	zeroshotskeletonbasedactionrecognition = "zero-shot skeleton-based action recognition"
	rte = "rte"
	overalltest = "overall - test"
	burnedareadelineation = "burned area delineation"
	imageparagraphcaptioning = "image paragraph captioning"
	emotionalspeechsynthesis = "emotional speech synthesis"
	unetquantization = "unet quantization"
	videocaptioning = "video captioning"
	counterfactualexplanation = "counterfactual explanation"
	kissdetection = "kiss detection"
	multiobjectdiscovery = "multi-object discovery"
	videorecognition = "video recognition"
	keypointmatching = "key point matching"
	acousticsceneclassification = "acoustic scene classification"
	motioninbetweening = "motion in-betweening"
	signlanguagerecognition = "sign language recognition"
	nounphrasecanonicalization = "noun phrase canonicalization"
	semisuperviseddomaingeneralization = "semi-supervised domain generalization"
	romaniantextdiacritization = "romanian text diacritization"
	wificsibasedimagereconstruction = "wifi csi-based image reconstruction"
	temporalsequences = "temporal sequences"
	clothchangingpersonreidentification = "cloth-changing person re-identification"
	oneshotmodelfusion = "one-shot model fusion"
	imagetoimageregression = "image-to-image regression"
	unsuperviseddomainadaptationsynthetictorealtranslation = "unsupervised domain adaptation,synthetic-to-real translation"
	argumentmining = "argument mining"
	steeringcontrol = "steering control"
	negationscoperesolution = "negation scope resolution"
	fashioncompatibilitylearning = "fashion compatibility learning"
	infraredimagesuperresolution = "infrared image super-resolution"
	multitissuenucleussegmentation = "multi-tissue nucleus segmentation"
	obfuscationdetection = "obfuscation detection"
	multispeakersourceseparation = "multi-speaker source separation"
	grammaticalerrorcorrection = "grammatical error correction"
	knowledgebaseconstruction = "knowledge base construction"
	generalknowledge = "general knowledge"
	sleepmicroeventdetection = "sleep micro-event detection"
	moviedialogsameordifferent = "movie dialog same or different"
	gaitrecognition = "gait recognition"
	eventextraction = "event extraction"
	ecgpatientidentificationgalleryprobe = "ecg patient identification (gallery-probe)"
	finegrainedvisualcategorization = "fine-grained visual categorization"
	visualcommonsensetests = "visual commonsense tests"
	classificationontimeserieswithmissingdata = "classification on time series with missing data"
	smalllanguagemodel = "small language model"
	videoinpainting = "video inpainting"
	unifiedimagerestoration = "unified image restoration"
	crossviewgeolocalisation = "cross-view geo-localisation"
	trafficobjectdetection = "traffic object detection"
	electroencephalogrameeg = "electroencephalogram (eeg)"
	driftdetection = "drift detection"
	modelposioning = "model posioning"
	dependencyparsing = "dependency parsing"
	graphquestionanswering = "graph question answering"
	retrosynthesis = "retrosynthesis"
	specificity = "specificity"
	videobasedgenerativeperformancebenchmarkingconsistency = "video-based generative performance benchmarking (consistency)"
	communitydetection = "community detection"
	cameraabsoluteposeregression = "camera absolute pose regression"
	videostoryqa = "video story qa"
	camerashotsegmentation = "camera shot segmentation"
	imageclassificationwithhumannoise = "image classification with human noise"
	figureofspeechdetection = "figure of speech detection"
	d3shaperepresentation = "3d shape representation"
	unsuperviseddomainexpansion = "unsupervised domain expansion"
	cadreconstruction = "cad reconstruction"
	signlanguagetranslation = "sign language translation"
	sequentialsentencesegmentation = "sequential sentence segmentation"
	visualgrounding = "visual grounding"
	incontextlearning = "in-context learning"
	contactrichmanipulation = "contact-rich manipulation"
	occupationprediction = "occupation prediction"
	videosaliencyprediction = "video saliency prediction"
	drugprot = "drugprot"
	medicalvisualquestionanswering = "medical visual question answering"
	missingvalues = "missing values"
	mortalityprediction = "mortality prediction"
	burstimagereconstruction = "burst image reconstruction"
	facialexpressionrecognition = "facial expression recognition"
	negationdetection = "negation detection"
	k8 = "8k"
	droneviewtargetlocalization = "drone-view target localization"
	crosslingualwordembeddings = "cross-lingual word embeddings"
	multiviewlearning = "multi-view learning"
	gunshotdetection = "gunshot detection"
	relationnetwork = "relation network"
	d3generation = "3d generation"
	withinsessionerp = "within-session erp"
	linkpropertyprediction = "link property prediction"
	adversarialdefenseagainstautoattack = "adversarial defense against autoattack"
	alldaysemanticsegmentation = "all-day semantic segmentation"
	foregroundsegmentation = "foreground segmentation"
	logicalfallacydetection = "logical fallacy detection"
	textinfilling = "text infilling"
	imageretargeting = "image retargeting"
	loadforecasting = "load forecasting"
	steganographics = "steganographics"
	orangesum = "orangesum"
	infantbrainmrisegmentation = "infant brain mri segmentation"
	impactedlocationdetection = "impacted location detection"
	inductiverelationprediction = "inductive relation prediction"
	topiccoverage = "topic coverage"
	tgiftransition = "tgif-transition"
	equilibriumreactionenergyevatom = "equilibrium reaction energy (ev/atom)"
	laysummarization = "lay summarization"
	clozetest = "cloze test"
	activeobservationcompletion = "active observation completion"
	keyphrasegeneration = "keyphrase generation"
	saliencyranking = "saliency ranking"
	oneshotlearning = "one-shot learning"
	intentclassification = "intent classification"
	objectcounting = "object counting"
	discoursesegmentation = "discourse segmentation"
	multilabelclassification = "multi-label classification"
	visualnavigation = "visual navigation"
	noderegression = "node regression"
	summarization = "summarization"
	chinesespellchecking = "chinese spell checking"
	navsim = "navsim"
	enusdomainclassification = "en-us domain classification"
	furnituresegmentation = "furniture segmentation"
	textto3dhuman = "text-to-3d-human"
	semanticdependencyparsing = "semantic dependency parsing"
	initialstructuretorelaxedenergyis2re = "initial structure to relaxed energy (is2re)"
	landmarktracking = "landmark tracking"
	overlapped10010 = "overlapped 100-10"
	deepattention = "deep attention"
	speechinterruptiondetection = "speech interruption detection"
	gamestatereconstruction = "game state reconstruction"
	audiocaptioning = "audio captioning"
	graphreconstruction = "graph reconstruction"
	personalityalignment = "personality alignment"
	misconceptions = "misconceptions"
	factualprobe = "factual probe"
	videoderaining = "video deraining"
	chinesepartofspeechtagging = "chinese part-of-speech tagging"
	singleobjectcolocalization = "single-object colocalization"
	spindledetection = "spindle detection"
	synthetictorealtranslation = "synthetic-to-real translation"
	videosemanticsegmentation = "video semantic segmentation"
	unsupervisedlongtermpersonreidentification = "unsupervised long term person re-identification"
	unbalancedsegmentation = "unbalanced segmentation"
	automatedessayscoring = "automated essay scoring"
	nestedtermextraction = "nested term extraction"
	wearableactivityrecognition = "wearable activity recognition"
	pitchclassification = "pitch classification"
	incontextreinforcementlearning = "in-context reinforcement learning"
	supervisedanomalydetection = "supervised anomaly detection"
	tabletotextgeneration = "table-to-text generation"
	linearmodeconnectivity = "linear mode connectivity"
	offlinehandwrittenchinesecharacterrecognition = "offline handwritten chinese character recognition"
	abstractanaphoraresolution = "abstract anaphora resolution"
	trainingfree3dpartsegmentation = "training-free 3d part segmentation"
	datatotextgeneration = "data-to-text generation"
	openworldobjectdetection = "open world object detection"
	d3rotationestimation = "3d rotation estimation"
	useridentification = "user identification"
	videovisualrelationtagging = "video visual relation tagging"
	knowledgebasequestionanswering = "knowledge base question answering"
	covidvariantprediction = "covid variant prediction"
	auxiliarylearning = "auxiliary learning"
	semisupervisedmedicalimageclassification = "semi-supervised medical image classification"
	livevideocaptioning = "live video captioning"
	speechsynthesisbodo = "speech synthesis - bodo"
	disambiguationqa = "disambiguation qa"
	imagematting = "image matting"
	taskgraphlearning = "task graph learning"
	boundarygrounding = "boundary grounding"
	objectrecognition = "object recognition"
	ironyidentification = "irony identification"
	interspeciesfacialkeypointtransfer = "interspecies facial keypoint transfer"
	adpprediction = "adp prediction"
	socialmediapopularityprediction = "social media popularity prediction"
	graphpropertyprediction = "graph property prediction"
	imageclassification = "image classification"
	unsupervisedvideoobjectsegmentation = "unsupervised video object segmentation"
	seizuredetection = "seizure detection"
	scientificdatausagedetection = "scientific data usage detection"
	statisticaldatausagedetection = "statistical data usage detection"
	graphranking = "graph ranking"
	textsegmentation = "text segmentation"
	semanticslam = "semantic slam"
	satelliteimagesuperresolution = "satellite image super-resolution"
	phraseextractionandgroundingpeg = "phrase extraction and grounding (peg)"
	unsupervisedzeroshotinstancesegmentation = "unsupervised zero-shot instance segmentation"
	nestednamedentityrecognition = "nested named entity recognition"
	zeroshotcomposedimageretrievalzscir = "zero-shot composed image retrieval (zs-cir)"
	kidneyfunction = "kidney function"
	pupiltracking = "pupil tracking"
	graphanomalydetection = "graph anomaly detection"
	seismicimaging = "seismic imaging"
	answerselection = "answer selection"
	semisupervisedtextregression = "semi-supervised text regression"
	semanticsegmentation = "semantic segmentation"
	paclearning = "pac learning"
	d2cyclistdetection = "2d cyclist detection"
	multilabelimagerecognition = "multi-label image recognition"
	semantictextualsimilarity = "semantic textual similarity"
	semisupervisedpersoninstancesegmentation = "semi-supervised person instance segmentation"
	patientphenotyping = "patient phenotyping"
	classify3dpointclouds = "classify 3d point clouds"
	politicalevalutation = "political evalutation"
	d3gs = "3dgs"
	pneumoniadetection = "pneumonia detection"
	roomlayoutestimation = "room layout estimation"
	multihypotheses3dhumanposeestimation = "multi-hypotheses 3d human pose estimation"
	dielectricconstant = "dielectric constant"
	motionplanning = "motion planning"
	zeroshotsentimentclassification = "zero-shot sentiment classification"
	videoqualityassessment = "video quality assessment"
	crossdomaintextclassification = "cross-domain text classification"
	jerseynumberrecognition = "jersey number recognition"
	unsupervisedfacerecognition = "unsupervised face recognition"
	earthsurfaceforecasting = "earth surface forecasting"
	phototocaricaturetranslation = "photo-to-caricature translation"
	histopathologicalimageclassification = "histopathological image classification"
	developmentallearning = "developmental learning"
	codedocumentationgeneration = "code documentation generation"
	subgraphcounting3star = "subgraph counting - 3 star"
	monocular3dhumanposeestimation = "monocular 3d human pose estimation"
	imagecaptioning = "image captioning"
	domaingeneralization = "domain generalization"
	arrhythmiadetection = "arrhythmia detection"
	temporalhumanmotioncomposition = "temporal human motion composition"
	robustspeechrecognition = "robust speech recognition"
	unsupervisedanomalydetectionwithspecifiedsettings10anomaly = "unsupervised anomaly detection with specified settings -- 10% anomaly"
	blindallinoneimagerestoration = "blind all-in-one image restoration"
	myocardialinfarctiondetection = "myocardial infarction detection"
	lightweightfacerecognition = "lightweight face recognition"
	timeseriesclustering = "time series clustering"
	hierarchicaltextclassificationofblurbsgermeval2019 = "hierarchical text classification of blurbs (germeval 2019)"
	multimodalpatchmatching = "multimodal patch matching"
	graphattention = "graph attention"
	dialogueunderstanding = "dialogue understanding"
	imageretrievalwithmultimodalquery = "image retrieval with multi-modal query"
	handwrittenwordgeneration = "handwritten word generation"
	onlinebeattracking = "online beat tracking"
	omnnidirectionalstereodepthestimation = "omnnidirectional stereo depth estimation"
	unsupervisedanomalydetectionwithspecifiedsettings30anomaly = "unsupervised anomaly detection with specified settings -- 30% anomaly"
	airqualityinference = "air quality inference"
	computedtomographyct = "computed tomography (ct)"
	audiodrivenbodyanimation = "audio-driven body animation"
	humanactiongeneration = "human action generation"
	zeroshotimageretrieval = "zero-shot image retrieval"
	metaphorboolean = "metaphor boolean"
	adhocvideosearch = "ad-hoc video search"
	musicqualityassessment = "music quality assessment"
	discretechoicemodels = "discrete choice models"
	featureimportance = "feature importance"
	phonocardiogramclassification = "phonocardiogram classification"
	outcomepredictioninmultimodalmri = "outcome prediction in multimodal mri"
	personreposing = "person reposing"
	disasterresponse = "disaster response"
	pointcloudregistration = "point cloud registration"
	robusttrafficprediction = "robust traffic prediction"
	vehiclespeedestimation = "vehicle speed estimation"
	computationalefficiency = "computational efficiency"
	naturallanguageinferenceoneshot = "natural language inference (one-shot)"
	referencebasedvideosuperresolution = "reference-based video super-resolution"
	realtimedirectionalhearing = "real-time directional hearing"
	learningsemanticrepresentations = "learning semantic representations"
	bonesuppressionfromdualenergychestxrays = "bone suppression from dual energy chest x-rays"
	mono3dvg = "mono3dvg"
	multiinstrumentmusictranscription = "multi-instrument music transcription"
	pharmacovigilance = "pharmacovigilance"
	sentencesegmentation = "sentence segmentation"
	polypsegmentation = "polyp segmentation"
	automatedfeatureengineering = "automated feature engineering"
	eventdetection = "event detection"
	d3scenegraphalignment = "3d scene graph alignment"
	organdetection = "organ detection"
	suspicousbirads45nosuspicousbirads123perimageclassification = "suspicous (birads 4,5)-no suspicous (birads 1,2,3) per image classification"
	replaygrounding = "replay grounding"
	facedubbing = "face dubbing"
	chinesenamedentityrecognition = "chinese named entity recognition"
	covid19diagnosis = "covid-19 diagnosis"
	trajectorymodeling = "trajectory modeling"
	citationprediction = "citation prediction"
	therapeuticsdatacommons = "therapeutics data commons"
	diabeticretinopathydetection = "diabetic retinopathy detection"
	braindecoding = "brain decoding"
	d3absolutehumanposeestimation = "3d absolute human pose estimation"
	linedetection = "line detection"
	medicalconceptnormalization = "medical concept normalization"
	pointcloudretrieval = "point cloud retrieval"
	humanjudgmentcorrelation = "human judgment correlation"
	pointsetupsampling = "point set upsampling"
	fewshotstancedetection = "few-shot stance detection"
	lemma = "lemma"
	relationextraction = "relation extraction"
	videosimilarity = "video similarity"
	counterfactualreasoning = "counterfactual reasoning"
	physicalintuition = "physical intuition"
	factclaimingcommentclassification = "fact-claiming comment classification"
	proteininterfaceprediction = "protein interface prediction"
	gymhalfcheetahexpert = "gym halfcheetah-expert"
	videocorrespondenceflow = "video correspondence flow"
	datasetcondensation = "dataset condensation"
	keyframebasedvideosuperresolutionk15 = "key-frame-based video super-resolution (k = 15)"
	eeg4classes = "eeg 4 classes"
	scientificdiscovery = "scientific discovery"
	abstractmeaningrepresentation = "abstract meaning representation"
	panopticsegmentation = "panoptic segmentation"
	newstargetdetection = "news target detection"
	stancedetectionuselection2020biden = "stance detection (us election 2020 - biden)"
	chinesespellingerrorcorrection = "chinese spelling error correction"
	fewshotaudioclassification = "few-shot audio classification"
	multiview3dreconstruction = "multi-view 3d reconstruction"
	openworldinstancesegmentation = "open-world instance segmentation"
	timelinesummarization = "timeline summarization"
	quanttrading = "quant trading"
	anxietydetection = "anxiety detection"
	violenceandweaponizedviolencedetection = "violence and weaponized violence detection"
	nodeclustering = "node clustering"
	factchecking = "fact checking"
	facealignment = "face alignment"
	realtime3dsemanticsegmentation = "real-time 3d semantic segmentation"
	artificiallife = "artificial life"
	actionclassification = "action classification"
	inductivebias = "inductive bias"
	rightventriclesegmentation = "right ventricle segmentation"
	monocular3dobjectlocalization = "monocular 3d object localization"
	generalizedfewshotsemanticsegmentation = "generalized few-shot semantic segmentation"
	networkidentification = "network identification"
	multimodalsleepstagedetection = "multimodal sleep stage detection"
	cola = "cola"
	skinlesionclassification = "skin lesion classification"
	nwppostprocessing = "nwp post-processing"
	errorunderstanding = "error understanding"
	hypergraphcontrastivelearning = "hypergraph contrastive learning"
	actionrecognitioninvideos = "action recognition in videos"
	liversegmentation = "liver segmentation"
	symmetricfaceinpainting = "symmetric face inpainting"
	multilingualmachinecomprehensioninenglishhindi = "multilingual machine comprehension in english hindi"
	semisupervisedimageclassification = "semi-supervised image classification"
	sensormodeling = "sensor modeling"
	highlightdetection = "highlight detection"
	d3anomalydetectionandsegmentation = "3d anomaly detection and segmentation"
	orpusvideomomentretrieval = "orpus video moment retrieval"
	causalemotionentailment = "causal emotion entailment"
	gesturerecognition = "gesture recognition"
	extractivetextsummarization = "extractive text summarization"
	chinesereadingcomprehension = "chinese reading comprehension"
	targetspeakerextraction = "target speaker extraction"
	starcraft = "starcraft"
	gesturegeneration = "gesture generation"
	nodepropertyprediction = "node property prediction"
	contributionassessment = "contribution assessment"
	unsupervisedfaciallandmarkdetectiononmafl = "unsupervised facial landmark detection on mafl"
	articulatedobjectmodelling = "articulated object modelling"
	clusteringensemble = "clustering ensemble"
	decisionmakingunderuncertainty = "decision making under uncertainty"
	smallobjectdetection = "small object detection"
	sentenceretrieval = "sentence retrieval"
	electrocardiographyecg = "electrocardiography (ecg)"
	lipreading = "lipreading"
	imageregistration = "image registration"
	semisupervisedsemanticsegmentation = "semi-supervised semantic segmentation"
	hdrreconstruction = "hdr reconstruction"
	active3dreconstruction = "active 3d reconstruction"
	calibrationforlinkprediction = "calibration for link prediction"
	mathematicalproofs = "mathematical proofs"
	d3lanedetection = "3d lane detection"
	d4spatiotemporalsemanticsegmentation = "4d spatio temporal semantic segmentation"
	atarigames100k = "atari games 100k"
	qlearning = "q-learning"
	openapicodecompletion = "openapi code completion"
	thermalinfraredobjecttracking = "thermal infrared object tracking"
	d6poseestimation = "6d pose estimation"
	extendedsummarization = "extended summarization"
	physicalattributeprediction = "physical attribute prediction"
	irregulartimeseries = "irregular time series"
	imagecropping = "image cropping"
	enfelectricnetworkfrequencyextractionfromvideo = "enf (electric network frequency) extraction from video"
	texttoaudioretrieval = "text to audio retrieval"
	opticdiscsegmentation = "optic disc segmentation"
	photoplethysmographyppg = "photoplethysmography (ppg)"
	abnormaleventdetectioninvideo = "abnormal event detection in video"
	opensetactionrecognition = "open set action recognition"
	adroidhammercloned = "adroid hammer-cloned"
	shot10imagegeneration = "10-shot image generation"
	miscellaneous = "miscellaneous"
	transitionbaseddependencyparsing = "transition-based dependency parsing"
	facialmakeuptransfer = "facial makeup transfer"
	sceneflowestimation = "scene flow estimation"
	cameraautocalibration = "camera auto-calibration"
	overlappingmentionrecognition = "overlapping mention recognition"
	lungdiseaseclassification = "lung disease classification"
	crashblossom = "crash blossom"
	codesearch = "code search"
	fastvehicledetection = "fast vehicle detection"
	videoalignment = "video alignment"
	skillmastery = "skill mastery"
	layoutdesign = "layout design"
	persuasivewritingstrategydetection = "persuasive writing strategy detection"
	blindsuperresolution = "blind super-resolution"
	claimextractionwithstanceclassificationcesc = "claim extraction with stance classification (cesc)"
	videoquestionansweringlevel4 = "video question answering (level 4)"
	d3facereconstruction = "3d face reconstruction"
	exceptiontype = "exception type"
	unsupervisedfewshotimageclassification = "unsupervised few-shot image classification"
	text2sparql = "text2sparql"
	engagingcommentclassification = "engaging comment classification"
	hallucinationevaluation = "hallucination evaluation"
	d3sceneediting = "3d scene editing"
	onlyconnectwallsdatasettask1grouping = "only connect walls dataset task 1 (grouping)"
	semantictextualsimilaritywithinbiencoder = "semantic textual similarity within bi-encoder"
	dialoglearning = "dialog learning"
	zeroshotlearning = "zero-shot learning"
	visionlanguageaction = "vision-language-action"
	extractingbuildingsinremotesensingimages = "extracting buildings in remote sensing images"
	textpairclassification = "text pair classification"
	videoadverbretrievalunseencompositions = "video-adverb retrieval (unseen compositions)"
	multimodalattributevalueextraction = "multimodal attribute value extraction"
	singleimagedehazing = "single image dehazing"
	graphmining = "graph mining"
	stereomatchinghand = "stereo matching hand"
	explainablemodels = "explainable models"
	trafficsignalcontrol = "traffic signal control"
	position = "position"
	conversationalinformationaccess = "conversational information access"
	diffusionmri = "diffusion mri"
	d6poseestimationusingrgbd = "6d pose estimation using rgbd"
	musicmodeling = "music modeling"
	animalactionrecognition = "animal action recognition"
	handwrittenwordsegmentation = "handwritten word segmentation"
	vietnamesedatasets = "vietnamese datasets"
	objectdiscoveryinvideos = "object discovery in videos"
	norealdatabinarization = "no real data binarization"
	materialsimaging = "materials imaging"
	texttoaudiovisualretrieval = "text-to-audiovisual retrieval"
	affordancerecognition = "affordance recognition"
	compositionalzeroshotlearning = "compositional zero-shot learning"
	facialexpressiongeneration = "facial expression generation"
	weaklysupervisedactionlocalization = "weakly supervised action localization"
	speakerrecognition = "speaker recognition"
	programmingerrordetection = "programming error detection"
	semisupervised2dand3dlandmarklabeling = "semi-supervised 2d and 3d landmark labeling"
	overlapped141 = "overlapped 14-1"
	zeroshotactionrecognition = "zero-shot action recognition"
	dialogstatetracking = "dialog state tracking"
	imagebasedrecommendationexplainability = "image-based recommendation explainability"
	zeroshotimageclassification = "zero-shot image classification"
	llmreallifetasks = "llm real-life tasks"
	d3pointcloudreconstruction = "3d point cloud reconstruction"
	chemicalproteininteractionextraction = "chemical-protein interaction extraction"
	extremevideoframeinterpolation = "extreme-video-frame-interpolation"
	knowledgetracing = "knowledge tracing"
	relationclassification = "relation classification"
	musicgenrerecognition = "music genre recognition"
	sequentialplacerecognition = "sequential place recognition"
	facialactionunitdetection = "facial action unit detection"
	textualanalogyparsing = "textual analogy parsing"
	detectgroundreflections = "detect ground reflections"
	molecularsystemprediction = "molecular system prediction"
	learningtheory = "learning theory"
	classificationofbreastcancerhistologyimages = "classification of breast cancer histology images"
	gymhalfcheetahmediumexpert = "gym halfcheetah-medium-expert"
	webpageobjectdetection = "webpage object detection"
	adroiddoorhuman = "adroid door-human"
	bodydetection = "body detection"
	relationprediction = "relation prediction"
	magneticresonancefingerprinting = "magnetic resonance fingerprinting"
	orientedobjectdetctio = "oriented object detctio"
	entityextractionusinggan = "entity extraction using gan"
	singleclassfewshotimagesynthesis = "single class few-shot image synthesis"
	conditionalimagegeneration = "conditional image generation"
	intrinsicimagedecomposition = "intrinsic image decomposition"
	mentalworkloadestimation = "mental workload estimation"
	musicsourceseparation = "music source separation"
	prototypeselection = "prototype selection"
	generalizedzeroshotobjectdetectiononmscoco = "generalized zero-shot object detection on ms-coco"
	tilingdeployment = "tiling & deployment"
	speakerseparation = "speaker separation"
	logicalargs = "logical args"
	paraphraseidentification = "paraphrase identification"
	generalclassification = "general classification"
	hungariantextdiacritization = "hungarian text diacritization"
	spokencommandrecognition = "spoken command recognition"
	pansharpening = "pansharpening"
	factverification = "fact verification"
	chartquestionanswering = "chart question answering"
	arabicspeechrecognition = "arabic speech recognition"
	novelconcepts = "novel concepts"
	d3objectdetectionfromstereoimages = "3d object detection from stereo images"
	sourcecodesummarization = "source code summarization"
	musicemotionrecognition = "music emotion recognition"
	multiexposureimagefusion = "multi-exposure image fusion"
	unsupervisedsemanticsegmentationwithlanguageimagepretraining = "unsupervised semantic segmentation with language-image pre-training"
	textwithinimagegeneration = "text within image generation"
	unity = "unity"
	reviewgeneration = "review generation"
	acousticunitdiscovery = "acoustic unit discovery"
	faceswapping = "face swapping"
	sentimentdependencylearning = "sentiment dependency learning"
	compressivesensing = "compressive sensing"
	pointcloudvideounderstanding = "point cloud video understanding"
	humaninstancesegmentation = "human instance segmentation"
	citationrecommendation = "citation recommendation"
	robustcameraonly3dobjectdetection = "robust camera only 3d object detection"
	openrelationmodeling = "open relation modeling"
	zeroshotvideoquestionanswer = "zero-shot video question answer"
	gazetargetestimation = "gaze target estimation"
	timeseriesanomalydetection = "time series anomaly detection"
	generativequestionanswering = "generative question answering"
	weaklysupervisedsegmentation = "weakly supervised segmentation"
	sarcasmdetection = "sarcasm detection"
	quantumcircuitgeneration = "quantum circuit generation"
	mvbench = "mvbench"
	satiredetection = "satire detection"
	productcategorization = "product categorization"
	discourseparsing = "discourse parsing"
	zeroshotslotfilling = "zero-shot slot filling"
	virtualtryonmodel2street = "virtual try-on (model2street)"
	statisticalindependencetesting = "statistical independence testing"
	causticssegmentation = "caustics segmentation"
	videoquestionansweringlevel3 = "video question answering (level 3)"
	trueorfalsequestionanswering = "true or false question answering"
	spacecraftposeestimation = "spacecraft pose estimation"
	graphonestimation = "graphon estimation"
	semanticimagematting = "semantic image matting"
	realtimesemanticsegmentation = "real-time semantic segmentation"
	grayscaleimagedenoising = "grayscale image denoising"
	variabledisambiguation = "variable disambiguation"
	unsuperviseddependencyparsing = "unsupervised dependency parsing"
	mixturepropertyprediction = "mixture property prediction"
	humanfmriresponseprediction = "human fmri response prediction"
	domain115 = "domain 11-5"
	multimodalnamedentityrecognition = "multi-modal named entity recognition"
	domain11 = "domain 1-1"
	programinduction = "program induction"
	groundedmultipleobjecttracking = "grounded multiple object tracking"
	weaklysupervisedinstancesegmentation = "weakly-supervised instance segmentation"
	zeroshotfacialexpressionrecognition = "zero-shot facial expression recognition"
	texttoaudiovideoretrieval = "text to audio/video retrieval"
	icuadmission = "icu admission"
	fakeimagedetection = "fake image detection"
	novelviewsynthesis = "novel view synthesis"
	d3facemodelling = "3d face modelling"
	tflmsequencegeneration = "tflm sequence generation"
	d4panopticsegmentation = "4d panoptic segmentation"
	sentencepairclassification = "sentence-pair classification"
	humanobjectinteractiondetection = "human-object interaction detection"
	blendedtargetdomainadaptation = "blended-target domain adaptation"
	howtorefundawrongtransactioninphonepe = "how to refund a wrong transaction in phonepe"
	fundustoangiographygeneration = "fundus to angiography generation"
	speechtogesturetranslation = "speech-to-gesture translation"
	hurtfulsentencecompletion = "hurtful sentence completion"
	cyberattackdetection = "cyber attack detection"
	longquestionanswer = "long question answer"
	weaklysupervisedactionsegmentationactionset = "weakly supervised action segmentation (action set))"
	onlinedownbeattracking = "online downbeat tracking"
	newsrecommendation = "news recommendation"
	edgeclassification = "edge classification"
	removepo = "remove - po"
	egocentricposeestimation = "egocentric pose estimation"
	tumorsegmentation = "tumor segmentation"
	withinsessionmotorimageryrighthandvsfeet = "within-session motor imagery (right hand vs. feet)"
	arabicsentimentanalysis = "arabic sentiment analysis"
	microgesturerecognition = "micro-gesture recognition"
	metamerism = "metamerism"
	driveridentification = "driver identification"
	semanticsegmentationoforthoimagery = "semantic segmentation of orthoimagery"
	surveillancetosingle = "surveillance-to-single"
	promptengineering = "prompt engineering"
	activityprediction = "activity prediction"
	lossycompressionartifactreduction = "lossy-compression artifact reduction"
	commonsenseknowledgebaseconstruction = "commonsense knowledge base construction"
	actionanalysis = "action analysis"
	emotionalintelligence = "emotional intelligence"
	videoguidedmachinetranslation = "video-guided machine translation"
	sonnetgeneration = "sonnet generation"
	wikipediasummarization = "wikipedia summarization"
	faceimagequality = "face image quality"
	atarigames = "atari games"
	logicalfallacies = "logical fallacies"
	d3densecaptioning = "3d dense captioning"
	actionclassificationzeroshot = "action classification (zero-shot)"
	surgicalskillsevaluation = "surgical skills evaluation"
	cancernocancerperbreastclassification = "cancer-no cancer per breast classification"
	multimediagenerativescriptlearning = "multimedia generative script learning"
	crosspartevaluation = "cross-part evaluation"
	stereodisparityestimation = "stereo disparity estimation"
	d3objecttracking = "3d object tracking"
	empatheticresponsegeneration = "empathetic response generation"
	multilabellearning = "multi-label learning"
	speechsynthesis = "speech synthesis"
	referringimagemattingrefmatterw100 = "referring image matting (refmatte-rw100)"
	videounderstanding = "video understanding"
	dependencygrammarinduction = "dependency grammar induction"
	nerf = "nerf"
	malwareanalysis = "malware analysis"
	imagetotext = "image to text"
	motiondetection = "motion detection"
	contextualanomalydetection = "contextual anomaly detection"
	subjecttransfer = "subject transfer"
	causaljudgment = "causal judgment"
	d3objectrecognition = "3d object recognition"
	classificationwithcostlyfeatures = "classification with costly features"
	d3videoframeinterpolation = "3d video frame interpolation"
	serialstyletransfer = "serial style transfer"
	thermalimagedenoising = "thermal image denoising"
	fewshothtc = "few-shot htc"
	languagemodelling = "language modelling"
	clozemultichoiceszeroshot = "cloze (multi-choices) (zero-shot)"
	eventcrossdocumentcoreferenceresolution = "event cross-document coreference resolution"
	adversarialdefenseagainstpgd = "adversarial defense against pgd"
	turkishtextdiacritization = "turkish text diacritization"
	realtimemultiobjecttracking = "real-time multi-object tracking"
	visualquestionansweringvqasplita = "visual question answering (vqa) split a"
	d3multipersonhumanposeestimation = "3d multi-person human pose estimation"
	satelliteimageclassification = "satellite image classification"
	imagestitching = "image stitching"
	trafficdataimputation = "traffic data imputation"
	duplicatequestionretrieval = "duplicate-question retrieval"
	cryogenicelectronmicroscopycryoem = "cryogenic electron microscopy (cryo-em)"
	zeroshotaudiocaptioning = "zero-shot audio captioning"
	localizationinvideoforgery = "localization in video forgery"
	objecttracking = "object tracking"
	poseretrieval = "pose retrieval"
	deeplearning = "deep learning"
	metaheuristicoptimization = "metaheuristic optimization"
	spatiotemporalactionlocalization = "spatio-temporal action localization"
	semisupervisedimageclassificationonimagenet10labeleddata = "semi-supervised image classification on imagenet - 10% labeled data"
	automaticliverandtumorsegmentation = "automatic liver and tumor segmentation"
	speakerspecificliptospeechsynthesis = "speaker-specific lip to speech synthesis"
	metalearning = "meta-learning"
	speechsynthesisrajasthani = "speech synthesis - rajasthani"
	textto3d = "text to 3d"
	keyvaluepairextraction = "key-value pair extraction"
	unconditionalimagegeneration = "unconditional image generation"
	audioquestionanswering = "audio question answering"
	efficientvits = "efficient vits"
	foodrecognition = "food recognition"
	opensetmultitargetdomainadaptation = "open-set multi-target domain adaptation"
	speechextraction = "speech extraction"
	automaticspeechrecognitionasr = "automatic speech recognition (asr)"
	conceptalignment = "concept alignment"
	imageshadowremoval = "image shadow removal"
	videoshadowdetection = "video shadow detection"
	sequencetosequencelanguagemodeling = "sequence-to-sequence language modeling"
	diseasetrajectoryforecasting = "disease trajectory forecasting"
	pulmonaryembolismdetection = "pulmonary embolism detection"
	ageandgenderclassification = "age and gender classification"
	ageclassification = "age classification"
	textbasedpersonretrievalwithnoisycorrespondence = "text-based person retrieval with noisy correspondence"
	toponymrecognition = "toponym recognition"
	strokeclassification = "stroke classification"
	misclassificationratenaturaladversarialsamples = "misclassification rate - natural adversarial samples"
	parallelcorpusmining = "parallel corpus mining"
	minecraft = "minecraft"
	layouttoimagegeneration = "layout-to-image generation"
	fewshottemporalactionlocalization = "few shot temporal action localization"
	finegrainedactionrecognition = "fine-grained action recognition"
	multipersonposeestimation = "multi-person pose estimation"
	scenegraphdetection = "scene graph detection"
	connectivityestimation = "connectivity estimation"
	networkembedding = "network embedding"
	segmentationofremotesensingimagery = "segmentation of remote sensing imagery"
	reasoningaboutcoloredobjects = "reasoning about colored objects"
	sketchbasedimageretrieval = "sketch-based image retrieval"
	momentqueries = "moment queries"
	texttovideoediting = "text-to-video editing"
	heterogeneousfacerecognition = "heterogeneous face recognition"
	d3objectdetection = "3d object detection"
	generative3dobjectclassification = "generative 3d object classification"
	nativelanguageidentification = "native language identification"
	robottaskplanning = "robot task planning"
	overlappingposeestimation = "overlapping pose estimation"
	referringmultiobjecttracking = "referring multi-object tracking"
	learningtoexecute = "learning to execute"
	diachronicwordembeddings = "diachronic word embeddings"
	rna3dstructureprediction = "rna 3d structure prediction"
	srftest = "srf (test)"
	nonsensewordsgrammar = "nonsense words grammar"
	openvocabularyimageclassification = "open vocabulary image classification"
	networkintrusiondetection = "network intrusion detection"
	rubikscube = "rubik's cube"
	environmentsoundclassification = "environment sound classification"
	quantumcircuitclassificationclassicalml = "quantum circuit classification (classical ml)"
	skincancerclassification = "skin cancer classification"
	stylegeneralization = "style generalization"
	microexpressiongenerationmegc2021 = "micro-expression generation (megc2021)"
	microactionrecognition = "micro-action recognition"
	questionanswercategorization = "question-answer categorization"
	ersonreidentification = "erson re-identification"
	jpegartifactcorrection = "jpeg artifact correction"
	lightfield = "lightfield"
	crosslingualparaphraseidentification = "cross-lingual paraphrase identification"
	d3pointcloudreinforcementlearning = "3d point cloud reinforcement learning"
	anomalydetectioninsurveillancevideos = "anomaly detection in surveillance videos"
	documentenhancement = "document enhancement"
	blinddocking = "blind docking"
	speechsynthesiskannada = "speech synthesis - kannada"
	rpusvideomomentretrieval = "rpus video moment retrieval"
	drugdesign = "drug design"
	numericalintegration = "numerical integration"
	ecgdenoising = "ecg denoising"
	englishconversationalspeechrecognition = "english conversational speech recognition"
	socialmediamentalhealthdetection = "social media mental health detection"
	diversity = "diversity"
	imagetoimagetranslation = "image-to-image translation"
	harvestingdateprediction = "harvesting date prediction"
	binaryrelationextraction = "binary relation extraction"
	pancreassegmentation = "pancreas segmentation"
	oneshotpartsegmentationofscoopaffordanceinterclass = "one-shot part segmentation of scoop affordance - inter class"
	arabicmmlu = "arabicmmlu"
	isvr = "isvr"
	scenesegmentation = "scene segmentation"
	imagequalityassessment = "image quality assessment"
	planeinstancesegmentation = "plane instance segmentation"
	videodescription = "video description"
	safereinforcementlearning = "safe reinforcement learning"
	generativetemporalnursing = "generative temporal nursing"
	automatedwritingevaluation = "automated writing evaluation"
	imageclassificationwithdifferentialprivacy = "image classification with differential privacy"
	travelingsalesmanproblem = "traveling salesman problem"
	multimediarecommendation = "multimedia recommendation"
	causaldiscoveryinvideoreasoning = "causal discovery in video reasoning"
	blindimagequalityassessment = "blind image quality assessment"
	nonexemplarbasedclassincrementallearning = "non-exemplar-based class incremental learning"
	anatomicallandmarkdetection = "anatomical landmark detection"
	consistentcharactergeneration = "consistent character generation"
	hyperspectralunmixing = "hyperspectral unmixing"
	causeeffectrelationclassification = "cause-effect relation classification"
	vectorquantizationkmeansproblem = "vector quantization (k-means problem)"
	documentunderstanding = "document understanding"
	bigbenchmachinelearning = "big-bench machine learning"
	imagecompressionartifactreduction = "image compression artifact reduction"
	semi_supervisedvideoactiondetection = "semi_supervised video action detection"
	changedetection = "change detection"
	dialoguemanagement = "dialogue management"
	embodiedquestionanswering = "embodied question answering"
	datainteraction = "data interaction"
	groundedmultimodalnamedentityrecognition = "grounded multimodal named entity recognition"
	signlanguageretrieval = "sign language retrieval"
	musicquestionanswering = "music question answering"
	webpagetagging = "web page tagging"
	camouflagesegmentation = "camouflage segmentation"
	graphtographtranslation = "graph-to-graph translation"
	tensornetworks = "tensor networks"
	typeprediction = "type prediction"
	equationdiscovery = "equation discovery"
	aspectcategorypolarity = "aspect category polarity"
	unsupervisedobjectlocalization = "unsupervised object localization"
	clickthroughrateprediction = "click-through rate prediction"
	d3visualgrounding = "3d visual grounding"
	geneticie = "genetic ie"
	unsupervisedanomalydetection = "unsupervised anomaly detection"
	imageretouching = "image retouching"
	audiovisualquestionansweringavqa = "audio-visual question answering (avqa)"
	selfevolvingai = "self-evolving ai"
	questionsimilarity = "question similarity"
	speechtospeechtranslation = "speech-to-speech translation"
	zeroshotsegmentation = "zero shot segmentation"
	imageinstanceretrieval = "image instance retrieval"
	hypergraphpartitioning = "hypergraph partitioning"
	fewshot3dsemanticsegmentation = "few-shot 3d semantic segmentation"
	tensoralgebra = "tensor algebra"
	rulesofthumbgeneration = "rules-of-thumb generation"
	sequentialbayesianinference = "sequential bayesian inference"
	facialediting = "facial editing"
	partialdomainadaptation = "partial domain adaptation"
	aspectorientedopinionextraction = "aspect-oriented opinion extraction"
	multimodalmusicgeneration = "multimodal music generation"
	multiwordexpressionsememeprediction = "multi-word expression sememe prediction"
	multimodallargelanguagemodel = "multimodal large language model"
	objectdetection = "object detection"
	regression = "regression"
	maskedlanguagemodeling = "masked language modeling"
	speechsynthesismanipuri = "speech synthesis - manipuri"
	shaperepresentationof3dpointclouds = "shape representation of 3d point clouds"
	constituencygrammarinduction = "constituency grammar induction"
	pupildiameterestimation = "pupil diameter estimation"
	cropyieldprediction = "crop yield prediction"
	sentinel1sarprocessing = "sentinel-1 sar processing"
	visualentailment = "visual entailment"
	bugfixing = "bug fixing"
	zeroshotaudioretrieval = "zero-shot audio retrieval"
	landcoverclassification = "land cover classification"
	actionlocalization = "action localization"
	componentclassification = "component classification"
	datavisualization = "data visualization"
	multistepretrosynthesis = "multi-step retrosynthesis"
	d3actionrecognition = "3d action recognition"
	dronebasedobjecttracking = "drone-based object tracking"
	spoofdetection = "spoof detection"
	directionofarrivalestimation = "direction of arrival estimation"
	automaticspeechtranslation = "automatic-speech-translation"
	syntaxrepresentation = "syntax representation"
	eventargumentextraction = "event argument extraction"
	medicalimageclassification = "medical image classification"
	federatedunsupervisedlearning = "federated unsupervised learning"
	vehicleposeestimation = "vehicle pose estimation"
	chineselandscapepaintinggeneration = "chinese landscape painting generation"
	semisupervisedchangedetection = "semi-supervised change detection"
	disjoint155 = "disjoint 15-5"
	d3objectreconstructionfromasingleimage = "3d object reconstruction from a single image"
	bridginganaphoraresolution = "bridging anaphora resolution"
	readabilityoptimization = "readability optimization"
	pedestrianimagecaption = "pedestrian image caption"
	snarks = "snarks"
	neuralradiancecaching = "neural radiance caching"
	lakedetection = "lake detection"
	worldknowledge = "world knowledge"
	keyinformationextraction = "key information extraction"
	explanationfidelityevaluation = "explanation fidelity evaluation"
	bandwidthextension = "bandwidth extension"
	proceduresteprecognition = "procedure step recognition"
	scoringrule = "scoring rule"
	pdesurrogatemodeling = "pde surrogate modeling"
	videosynchronization = "video synchronization"
	zeroshotlongvideoglobalmodequestionanswering = "zero-shot long video global-mode question answering"
	stochasticblockmodel = "stochastic block model"
	opinionmining = "opinion mining"
	videosummarization = "video summarization"
	deephashing = "deep hashing"
	distanceregression = "distance regression"
	secondordermethods = "second-order methods"
	structuralhealthmonitoring = "structural health monitoring"
	multiviewsubspaceclustering = "multi-view subspace clustering"
	fewshotner = "few-shot ner"
	lightingestimation = "lighting estimation"
	eventbasedvision = "event-based vision"
	mrisegmentation = "mri segmentation"
	longvideoretrievalbackgroundremoved = "long video retrieval (background removed)"
	quantumstatetomography = "quantum state tomography"
	imagetextmatching = "image-text matching"
	sand = "sand"
	defectdetection = "defect detection"
	multimodalinteraction = "multimodal interaction"
	pos = "pos"
	imageanimation = "image animation"
	foldedtissuedetection = "folded tissue detection"
	testdrivendevelopment = "test driven development"
	motionstyletransfer = "motion style transfer"
	propertyprediction = "property prediction"
	largelanguagemodel = "large language model"
	virology = "virology"
	fullbodyanonymization = "full-body anonymization"
	authorattribution = "author attribution"
	dialogueinterpretation = "dialogue interpretation"
	semisupervisedlearningforimagecaptioning = "semi supervised learning for image captioning"
	overlapped191 = "overlapped 19-1"
	iupacnameprediction = "iupac name prediction"
	musicclassification = "music classification"
	breastcancerdetection = "breast cancer detection"
	disjoint101 = "disjoint 10-1"
	semisupervisedaudioregression = "semi-supervised audio regression"
	waveletstructuresimilarityloss = "wavelet structure similarity loss"
	overlapped2525 = "overlapped 25-25"
	shooterlocalization = "shooter localization"
	transferlearning = "transfer learning"
	physicsmc = "physics mc"
	modelsalignment = "models alignment"
	informationextraction = "information extraction"
	task2 = "task 2"
	unsupervisedsentencesummarization = "unsupervised sentence summarization"
	magetotextretrieval = "mage-to-text retrieval"
	spacetimevideosuperresolution = "space-time video super-resolution"
	prepositionalphraseattachment = "prepositional phrase attachment"
	proteinannotation = "protein annotation"
	faciesclassification = "facies classification"
	papergenerationconclusiontotitle = "paper generation (conclusion-to-title)"
	referencebasedsuperresolution = "reference-based super-resolution"
	imagesteganography = "image steganography"
	patientspecificsegmentation = "patient-specific segmentation"
	imageinpainting = "image inpainting"
	videovisualrelationdetection = "video visual relation detection"
	earthquakeprediction = "earthquake prediction"
	objecthallucination = "object hallucination"
	trajectoryclustering = "trajectory clustering"
	dftzisomerpipiwavelength = "dft z isomer pi-pi* wavelength"
	openknowledgegraphcanonicalization = "open knowledge graph canonicalization"
	singleimagegeneration = "single-image-generation"
	automaticspeechrecognition = "automatic speech recognition"
	relationexplanation = "relation explanation"
	shadowdetection = "shadow detection"
	chinesespamdetection = "chinese spam detection"
	smacplus = "smac+"
	zeroshottexttoimagegeneration = "zero-shot text-to-image generation"
	fewshottransferlearningforsaliencyprediction = "few-shot transfer learning for saliency prediction"
	fakenewsdetection = "fake news detection"
	astronomy = "astronomy"
	d3objectediting = "3d object editing"
	irishtextdiacritization = "irish text diacritization"
	gymhalfcheetahfullreplay = "gym halfcheetah-full-replay"
	finegrainedopinionanalysis = "fine-grained opinion analysis"
	codetranslation = "code translation"
	personcentricvisualgrounding = "person-centric visual grounding"
	d3handposeestimation = "3d hand pose estimation"
	melanomadiagnosis = "melanoma diagnosis"
	panopticscenegraphgeneration = "panoptic scene graph generation"
	gymhalfcheetahmedium = "gym halfcheetah-medium"
	ctreconstruction = "ct reconstruction"
	personalizedandemotionalconversation = "personalized and emotional conversation"
	deepclustering = "deep clustering"
	semanticcomposition = "semantic composition"
	syntheticimageattribution = "synthetic image attribution"
	morphemesegmentaiton = "morpheme segmentaiton"
	graphembedding = "graph embedding"
	textanonymization = "text anonymization"
	cancerclassification = "cancer classification"
	writerretrieval = "writer retrieval"
	spamdetection = "spam detection"
	rgbsalientobjectdetection = "rgb salient object detection"
	ecgpatientidentification = "ecg patient identification"
	crossdomainfewshotlearning = "cross-domain few-shot learning"
	datasourcesdetection = "data sources detection"
	disjoint151 = "disjoint 15-1"
	simultaneousspeechtotexttranslation = "simultaneous speech-to-text translation"
	filedifference = "file difference"
	hatespeechdetectioncrisishatemmbenchmark = "hate speech detection crisishatemm benchmark"
	activeobjectlocalization = "active object localization"
	multiorientedscenetextdetection = "multi-oriented scene text detection"
	eeg = "eeg"
	reversedictionary = "reverse dictionary"
	hippocampus = "hippocampus"
	soundeventlocalizationanddetection = "sound event localization and detection"
	pointtracking = "point tracking"
	implicitrelations = "implicit relations"
	emotionaldialogueacts = "emotional dialogue acts"
	multiinstanceretrieval = "multi-instance retrieval"
	mitosisdetection = "mitosis detection"
	d3shapemodeling = "3d shape modeling"
	hdsemanticmaplearning = "hd semantic map learning"
	handobjectpose = "hand-object pose"
	handwrittentextrecognition = "handwritten text recognition"
	uncertaintyawarepanopticsegmentation = "uncertainty-aware panoptic segmentation"
	efficientdiffusionpersonalization = "efficient diffusion personalization"
	storycompletion = "story completion"
	aspectsentimentopiniontripletextraction = "aspect-sentiment-opinion triplet extraction"
	deformablemedicalimageregistration = "deformable medical image registration"
	generalizedreferringexpressionsegmentation = "generalized referring expression segmentation"
	entailedpolarity = "entailed polarity"
	zeroshotoutofdomaindetection = "zero-shot out-of-domain detection"
	analyticentailment = "analytic entailment"
	ganimageforensics = "gan image forensics"
	superpixels = "superpixels"
	modeldiscovery = "model discovery"
	smallfootprintkeywordspotting = "small-footprint keyword spotting"
	spatiotemporalscenegraphs = "spatio-temporal scene graphs"
	handdetection = "hand detection"
	d3buildingmeshlabeling = "3d building mesh labeling"
	wordtranslation = "word translation"
	pointcloudpretraining = "point cloud pre-training"
	continuouscontrol = "continuous control"
	realtimeinstancemeasurement = "real-time instance measurement"
	maliciousdetection = "malicious detection"
	recordlinking = "record linking"
	unseenobjectinstancesegmentation = "unseen object instance segmentation"
	medicalsuperresolution = "medical super-resolution"
	counterfactualplanning = "counterfactual planning"
	logicalsequence = "logical sequence"
	aerialvideosemanticsegmentation = "aerial video semantic segmentation"
	motiongeneration = "motion generation"
	incompletemultiviewclustering = "incomplete multi-view clustering"
	extractivequestionanswering = "extractive question-answering"
	medicalimagegeneration = "medical image generation"
	videocompressivesensing = "video compressive sensing"
	fishdetection = "fish detection"
	multivariatetimeseriesimputation = "multivariate time series imputation"
	aiagent = "ai agent"
	datasetdistillation1ipc = "dataset distillation - 1ipc"
	handsegmentation = "hand segmentation"
	singleimagedesnowing = "single image desnowing"
	gameofhanabi = "game of hanabi"
	d3semanticscenecompletionfromasingle2dimage = "3d semantic scene completion from a single 2d image"
	trustableandfocussedllmgeneratedcontent = "trustable and focussed llm generated content"
	densityestimation = "density estimation"
	lowdosexrayctreconstruction = "low-dose x-ray ct reconstruction"
	soundeventdetection = "sound event detection"
	videobasedgenerativeperformancebenchmarkingcorrectnessofinformation = "video-based generative performance benchmarking (correctness of information)"
	diagnostic = "diagnostic"
	silentspeechrecognition = "silent speech recognition"
	valeneestimation = "valene estimation"
	dynamicregionsegmentation = "dynamic region segmentation"
	portfoliooptimization = "portfolio optimization"
	sowingdateprediction = "sowing date prediction"
	sparserepresentationbasedclassification = "sparse representation-based classification"
	computationalchemistry = "computational chemistry"
	gameofshogi = "game of shogi"
	sequencetosequencespeechrecognition = "sequence-to-sequence speech recognition"
	medicalcodesprediction = "medical codes prediction"
	energytrading = "energy trading"
	potraitgeneration = "potrait generation"
	futurehandprediction = "future hand prediction"
	replacepq = "replace - pq"
	smac = "smac"
	hallucination = "hallucination"
	graphlearning = "graph learning"
	temporallocalization = "temporal localization"
	automaticphonemerecognition = "automatic phoneme recognition"
	medicalgenetics = "medical genetics"
	audiovisualquestionanswering = "audio-visual question answering"
	croatiantextdiacritization = "croatian text diacritization"
	dimensionalityreduction = "dimensionality reduction"
	medicalimageretrieval = "medical image retrieval"
	humanpartsegmentation = "human part segmentation"
	sequentialskipprediction = "sequential skip prediction"
	lemmatization = "lemmatization"
	texttospeechtranslation = "text-to-speech translation"
	landmine = "landmine"
	controllablelanguagemodelling = "controllable language modelling"
	monocularcrossviewroadsceneparsingroad = "monocular cross-view road scene parsing(road)"
	drumtranscription = "drum transcription"
	distributionalreinforcementlearning = "distributional reinforcement learning"
	speculationscoperesolution = "speculation scope resolution"
	eyeblinkdetection = "eyeblink detection"
	lexicalsimplification = "lexical simplification"
	emotionrecognitioninconversation = "emotion recognition in conversation"
	rgbttracking = "rgb-t tracking"
	circulatoryfailure = "circulatory failure"
	inverserendering = "inverse rendering"
	hopespeechdetectionforenglish = "hope speech detection for english"
	autonomousflightdenseforest = "autonomous flight (dense forest)"
	testunseen = "test unseen"
	fewshotopensetobjectdetection = "few shot open set object detection"
	cancernocancerperimageclassification = "cancer-no cancer per image classification"
	pagestreamsegmentation = "page stream segmentation"
	natureinspiredoptimizationalgorithm = "nature-inspired optimization algorithm"
	humandetection = "human detection"
	handwritingrecognition = "handwriting recognition"
	compositionalgeneralizationavg = "compositional generalization (avg)"
	movingobjectdetection = "moving object detection"
	videoreconstruction = "video reconstruction"
	lowresourceneuralmachinetranslation = "low-resource neural machine translation"
	xraydiffractionxrd = "x-ray diffraction (xrd)"
	sentenceembeddingsforbiomedicaltexts = "sentence embeddings for biomedical texts"
	textdependentspeakerverification = "text-dependent speaker verification"
	singleimagedeblurring = "single image deblurring"
	knowledgeprobing = "knowledge probing"
	sidechannelanalysis = "side channel analysis"
	networkcommunitypartition = "network community partition"
	photoplethysmographyppgheartrateestimation = "photoplethysmography (ppg) heart rate estimation"
	immunerepertoireclassification = "immune repertoire classification"
	semanticityprediction = "semanticity prediction"
	zeroshottransfer3dpointcloudclassification = "zero-shot transfer 3d point cloud classification"
	productionforecasting = "production forecasting"
	opticalflowestimation = "optical flow estimation"
	visualabductivereasoning = "visual abductive reasoning"
	partiallyrelevantvideoretrieval = "partially relevant video retrieval"
	dronecontroller = "drone controller"
	videogeneration = "video generation"
	questiongeneration = "question generation"
	motorimagery = "motor imagery"
	audiogeneration = "audio generation"
	videosaliencydetection = "video saliency detection"
	neuralnetworkcompression = "neural network compression"
	simultaneouslocalizationandmapping = "simultaneous localization and mapping"
	sourcefreedomaingeneralization = "source-free domain generalization"
	d3canonicalization = "3d canonicalization"
	weaklysupervisedreferringexpressionsegmentation = "weakly supervised referring expression segmentation"
	transductivezeroshotclassification = "transductive zero-shot classification"
	fewshotvideoobjectdetection = "few-shot video object detection"
	goalorienteddialog = "goal-oriented dialog"
	singeridentification = "singer identification"
	zeroshotactionrecognitiononucf101 = "zero-shot action recognition on ucf101"
	imagecomprehension = "image comprehension"
	entitytypingondhkgs = "entity typing on dh-kgs"
	universalmetalearning = "universal meta-learning"
	bayesianoptimization = "bayesian optimization"
	muscletendonjunctionidentification = "muscle tendon junction identification"
	binarytextclassification = "binary text classification"
	veracityclassification = "veracity classification"
	depthimageupsampling = "depth image upsampling"
	transliteration = "transliteration"
	tablesearch = "table search"
	videoretrievalframelevel = "video retrieval (frame-level)"
	acousticquestionanswering = "acoustic question answering"
	aspectbasedsentimentanalysis = "aspect-based sentiment analysis"
	questionselection = "question selection"
	fewshotclassificationandsegmentation = "few-shot classification and segmentation"
	rollingshuttercorrection = "rolling shutter correction"
	zeroshotcrosslingualtexttoimageretrieval = "zero-shot cross-lingual text-to-image retrieval"
	imitationlearning = "imitation learning"
	humandynamics = "human dynamics"
	vietnamesesocialmediatextprocessing = "vietnamese social media text processing"
	imagetotextretrieval = "image-to-text retrieval"
	textindependentspeakerrecognition = "text-independent speaker recognition"
	sentenceclassification = "sentence classification"
	multimodaltextprediction = "multimodal text prediction"
	saliencydetection = "saliency detection"
	humanmotionprediction = "human motion prediction"
	crossdocumentcoreferenceresolution = "cross document coreference resolution"
	surgicaltooldetection = "surgical tool detection"
	imagebasedlocalization = "image-based localization"
	implicitdiscourserelationclassification = "implicit discourse relation classification"
	virtualtryonshop2street = "virtual try-on (shop2street)"
	lowrankcompression = "low-rank compression"
	textbasedpersonsearch = "text based person search"
	groundedvideoquestionanswering = "grounded video question answering"
	faceparsing = "face parsing"
	mathematicalproblemsolving = "mathematical problem-solving"
	noisyspeechrecognition = "noisy speech recognition"
	enusintentclassification = "en-us intent classification"
	languageacquisition = "language acquisition"
	videosalientobjectdetection = "video salient object detection"
	intentrecognition = "intent recognition"
	oneclassclassifier = "one-class classifier"
	actiongeneration = "action generation"
	zeroshottexttoaudioretrieval = "zero-shot text to audio retrieval"
	avbtwo = "a-vb two"
	chinesesemanticrolelabeling = "chinese semantic role labeling"
	demoire = "demoire"
	occlusionhandling = "occlusion handling"
	ticketsearch = "ticket search"
	colorimagedenoising = "color image denoising"
	speechdenoising = "speech denoising"
	spatiotemporalforecasting = "spatio-temporal forecasting"
	floodedbuildingsegmentation = "flooded building segmentation"
	videostabilization = "video stabilization"
	imagemanipulationdetection = "image manipulation detection"
	zeroshottexttoimageretrieval = "zero-shot text-to-image retrieval"
	drivableareadetection = "drivable area detection"
	multihopreadingcomprehension = "multi-hop reading comprehension"
	interactivesegmentation = "interactive segmentation"
	attributevalueextraction = "attribute value extraction"
	speechrepresentationlearning = "speech representation learning"
	mamba = "mamba"
	fewshot3dpointcloudsemanticsegmentation = "few-shot 3d point cloud semantic segmentation"
	fpsgames = "fps games"
	onlineclustering = "online clustering"
	motionprediction = "motion prediction"
	videoforensics = "video forensics"
	identifyoddmetapor = "identify odd metapor"
	imagevariation = "image-variation"
	ultrasound = "ultrasound"
	covid19tracking = "covid-19 tracking"
	faceimagequalityassessment = "face image quality assessment"
	radiologistbinaryclassification = "radiologist binary classification"
	figurativelanguagevisualization = "figurative language visualization"
	personidentification = "person identification"
	historicalcolorimagedating = "historical color image dating"
	procgenhard100m = "procgen hard (100m)"
	blindimagedeblurring = "blind image deblurring"
	braincomputerinterface = "brain computer interface"
	vietnameselanguagemodels = "vietnamese language models"
	shorttextconversation = "short-text conversation"
	crossdomainactivityrecognition = "cross-domain activity recognition"
	nonparametricdeepclustering = "nonparametric deep clustering"
	supervisedonly3dpointcloudclassification = "supervised only 3d point cloud classification"
	trajectoryforecasting = "trajectory forecasting"
	myocardiumsegmentation = "myocardium segmentation"
	speechsynthesishindi = "speech synthesis - hindi"
	graphtosequence = "graph-to-sequence"
	distributedoptimization = "distributed optimization"
	zeroshotimageparagraphcaptioning = "zero-shot image paragraph captioning"
	frenchtextdiacritization = "french text diacritization"
	robotmanipulationgeneralization = "robot manipulation generalization"
	temporalgroupactivitylocalization = "temporal group activity localization"
	catalyticactivityprediction = "catalytic activity prediction"
	evolutionaryalgorithms = "evolutionary algorithms"
	grereadingcomprehension = "gre reading comprehension"
	egocentricactivityrecognition = "egocentric activity recognition"
	structuralnodeembedding = "structural node embedding"
	breasttissueidentification = "breast tissue identification"
	bathymetryprediction = "bathymetry prediction"
	aspectcategorydetection = "aspect category detection"
	neutronpdfregression = "neutron pdf regression"
	sketchtotextgeneration = "sketch-to-text generation"
	multiobjectivemultiagentreinforcementlearning = "multi-objective multi-agent reinforcement learning"
	dnaanalysis = "dna analysis"
	embeddingsevaluation = "embeddings evaluation"
	reagentprediction = "reagent prediction"
	activitydetection = "activity detection"
	vietnamesewordsegmentation = "vietnamese word segmentation"
	amodallayoutestimation = "amodal layout estimation"
	atomicnumberclassification = "atomic number classification"
	d3planedetection = "3d plane detection"
	causalidentification = "causal identification"
	imagedehazing = "image dehazing"
	collaborativeinference = "collaborative inference"
	onlinecommunitydetection = "online community detection"
	sqlsynthesis = "sql synthesis"
	conversationdisentanglement = "conversation disentanglement"
	birdviewsynthesis = "bird view synthesis"
	actionsegmentation = "action segmentation"
	steganalysis = "steganalysis"
	eikonaltomography = "eikonal tomography"
	d3shaperetrieval = "3d shape retrieval"
	perceptualdistance = "perceptual distance"
	semisupervisedsketchbasedimageretrieval = "semi-supervised sketch based image retrieval"
	dronenavigation = "drone navigation"
	ancienttextrestoration = "ancient text restoration"
	guielementdetection = "gui element detection"
	temporalactionsegmentation = "temporal action segmentation"
	landmarkrecognition = "landmark recognition"
	lidarabsoluteposeregression = "lidar absolute pose regression"
	analogvideorestoration = "analog video restoration"
	interactive3dinstancesegmentationtrainedonscannet40evaluatedonscannet40 = "interactive 3d instance segmentation -trained on scannet40 - evaluated on scannet40"
	columnspropertyannotation = "columns property annotation"
	taskandmotionplanning = "task and motion planning"
	dynamictopicmodeling = "dynamic topic modeling"
	adversarialnaturallanguageinference = "adversarial natural language inference"
	trafficaccidentdetection = "traffic accident detection"
	referringimagemattingpromptbased = "referring image matting (prompt-based)"
	tableannotation = "table annotation"
	pseudolabel = "pseudo label"
	commonsensereasoningzeroshot = "common sense reasoning (zero-shot)"
	lowlatencyprocessing = "low-latency processing"
	multidocumentsummarization = "multi-document summarization"
	surfacenormalestimation = "surface normal estimation"
	documentai = "document ai"
	contactmechanics = "contact mechanics"
	usersimulation = "user simulation"
	occludedpersonreidentification = "occluded person re-identification"
	unbiasedscenegraphgeneration = "unbiased scene graph generation"
	sourcecodeauthorshipattribution = "source code authorship attribution"
	objectgoalnavigation = "objectgoal navigation"
	surveillancetobooking = "surveillance-to-booking"
	documentlevelclosedinformationextraction = "document-level closed information extraction"
	outofdistributionooddetection = "out of distribution (ood) detection"
	physicalcommonsensereasoning = "physical commonsense reasoning"
	namedentityrecognitioninvietnamese = "named entity recognition in vietnamese"
	actionqualityassessment = "action quality assessment"
	d3pointclouddataaugmentation = "3d point cloud data augmentation"
	mtebbenchmark = "mteb benchmark"
	personreidentification = "person re-identification"
	tdcadmetbenchmarkinggroup = "tdc admet benchmarking group"
	landmarkbasedlipreading = "landmark-based lipreading"
	socialnavigation = "social navigation"
	motionmagnification = "motion magnification"
	d3semanticsegmentation = "3d semantic segmentation"
	imagesimilaritysearch = "image similarity search"
	querybasedextractivesummarization = "query-based extractive summarization"
	classificationofhyperspectralimages = "classification of hyperspectral images"
	blooddetection = "blood detection"
	domainadaptation = "domain adaptation"
	microexpressiongeneration = "micro-expression generation"
	adversarialpurification = "adversarial purification"
	taxonomyexpansion = "taxonomy expansion"
	racem = "race-m"
	phishingwebsitedetection = "phishing website detection"
	medicalcodeprediction = "medical code prediction"
	structuredoutputgeneration = "structured output generation"
	treedecomposition = "tree decomposition"
	distortioncorrection = "distortion correction"
	backdoordefense = "backdoor defense"
	demandforecasting = "demand forecasting"
	zeroshot3dpointcloudclassificationclassification = "zero-shot 3d point cloud classificationclassification"
	oralcancerclassification = "oral cancer classification"
	permutedmnist = "permuted-mnist"
	textbasedimageediting = "text-based image editing"
	dialogueactclassification = "dialogue act classification"
	alignementvisualisation = "alignement visualisation"
	zeroshotrelationtripletextraction = "zero-shot relation triplet extraction"
	spellingcorrection = "spelling correction"
	actionassessment = "action assessment"
	morphologicaltagging = "morphological tagging"
	multiagentpathfinding = "multi-agent path finding"
	similaritiesabstraction = "similarities abstraction"
	syntheticdatageneration = "synthetic data generation"
	quantitativemri = "quantitative mri"
	oneshotobjectdetection = "one-shot object detection"
	behaviouralcloning = "behavioural cloning"
	unetsegmentation = "unet segmentation"
	microvideorecommendation = "microvideo recommendation"
	monoculardepthestimation = "monocular depth estimation"
	audiocompression = "audio compression"
	rgbdinstancesegmentation = "rgb-d instance segmentation"
	subspacemethods = "subspace methods"
	diabeticretinopathygrading = "diabetic retinopathy grading"
	categoricaldataclustering = "categorical data clustering"
	d3questionanswering3dqa = "3d question answering (3d-qa)"
	imageharmonization = "image harmonization"
	temporalsentencegrounding = "temporal sentence grounding"
	dialogue = "dialogue"
	topicclassification = "topic classification"
	mbpp = "mbpp"
	semisupervisedactiondetection = "semi-supervised action detection"
	pretrainedmultilinguallanguagemodels = "pretrained multilingual language models"
	breasttumourclassification = "breast tumour classification"
	drawingpictures = "drawing pictures"
	textcomplexityassessmentgermeval2022 = "text complexity assessment (germeval 2022)"
	musicautotagging = "music auto-tagging"
	symmetrydetection = "symmetry detection"
	decisionmaking = "decision making"
	wireframeparsing = "wireframe parsing"
	dialoguesafetyprediction = "dialogue safety prediction"
	temporalactionlocalization = "temporal action localization"
	attributeextraction = "attribute extraction"
	probabilisticprogramming = "probabilistic programming"
	objectstatechangeclassification = "object state change classification"
	recipegeneration = "recipe generation"
	zeroshotintentclassification = "zero-shot intent classification"
	humanobjectrelationshipdetection = "human-object relationship detection"
	visionlanguagenavigation = "vision-language navigation"
	imagetextclassification = "image-text classification"
	d3classification = "3d classification"
	entitytyping = "entity typing"
	captiongeneration = "caption generation"
	trafficsigndetection = "traffic sign detection"
	drugdruginteractionextraction = "drug–drug interaction extraction"
	facerecognitionclosedset = "face recognition (closed-set)"
	speakerverification = "speaker verification"
	videodomainadapation = "video domain adapation"
	densecaptioning = "dense captioning"
	steinertreeproblem = "steiner tree problem"
	twitterbotdetection = "twitter bot detection"
	heartratevariability = "heart rate variability"
	inductivelearning = "inductive learning"
	videopanopticsegmentation = "video panoptic segmentation"
	materialsscreening = "materials screening"
	personidentification1shot = "person identification (1-shot)"
	crossdomaindocumentclassification = "cross-domain document classification"
	morphologicalanalysis = "morphological analysis"
	retinaloctdiseaseclassification = "retinal oct disease classification"
	counterfactualdetection = "counterfactual detection"
	tinyqabenchmark = "tinyqa benchmark++"
	stancedetection = "stance detection"
	facepresentationattackdetection = "face presentation attack detection"
	imageoutpainting = "image outpainting"
	multimodalemotionrecognition = "multimodal emotion recognition"
	hardlabelattack = "hard-label attack"
	termextraction = "term extraction"
	imageclustering = "image clustering"
	phraseranking = "phrase ranking"
	isaacgympreview = "isaac gym preview"
	gameofsudoku = "game of sudoku"
	subjectdrivenvideogeneration = "subject-driven video generation"
	unsupervisedinstancesegmentation = "unsupervised instance segmentation"
	listops = "listops"
	d3openvocabularyobjectdetection = "3d open-vocabulary object detection"
	abstractalgebra = "abstract algebra"
	imageimputation = "image imputation"
	imagegeneration = "image generation"
	depthestimation = "depth estimation"
	superclassclassification = "superclass classification"
	referringexpressioncomprehension = "referring expression comprehension"
	controllableimagecaptioning = "controllable image captioning"
	handwritinggeneration = "handwriting generation"
	claimverification = "claim verification"
	stochastichumanmotionprediction = "stochastic human motion prediction"
	vehiclecolorrecognition = "vehicle color recognition"
	learningwordembeddings = "learning word embeddings"
	contactdetection = "contact detection"
	univariatetimeseriesforecasting = "univariate time series forecasting"
	multipleactiondetection = "multiple action detection"
	faceverification = "face verification"
	knownunknowns = "known unknowns"
	qrscomplexdetection = "qrs complex detection"
	seismicinversion = "seismic inversion"
	handwrittendigitrecognition = "handwritten digit recognition"
	eventbasedobjectsegmentation = "event-based object segmentation"
	persuasivewritingstrategydetectionlevel4 = "persuasive writing strategy detection level-4"
	skullstripping = "skull stripping"
	panopticsegmentationpanopticndtinstances = "panoptic segmentation (panopticndt instances)"
	oneshotpartsegmentationofwrapgraspaffordanceintraclass = "one-shot part segmentation of wrap-grasp affordance - intra class"
	partiallabellearning = "partial label learning"
	oneshotpartsegmentationofsupportaffordanceintraclass = "one-shot part segmentation of support affordance - intra class"
	privacypreservingdeeplearning = "privacy preserving deep learning"
	discoursemarkerprediction = "discourse marker prediction"
	mmlu = "mmlu"
	imagedeblocking = "image deblocking"
	abstractivedialoguesummarization = "abstractive dialogue summarization"
	speechsynthesisbengali = "speech synthesis - bengali"
	zeroshotmultispeakertts = "zero-shot multi-speaker tts"
	temporalcasualqa = "temporal/casual qa"
	hypergraphmatching = "hypergraph matching"
	soundclassification = "sound classification"
	carracing = "car racing"
	actiontripletdetection = "action triplet detection"
	densevideocaptioning = "dense video captioning"
	openvocabularyattributedetection = "open vocabulary attribute detection"
	sleepstagedetection = "sleep stage detection"
	hyperspectralimagesegmentation = "hyperspectral image segmentation"
	arteryveinsretinalvesselsegmentation = "artery/veins retinal vessel segmentation"
	unsupervisedclassincrementallearning = "unsupervised class-incremental learning"
	rfbasedposeestimation = "rf-based pose estimation"
	mirrordetection = "mirror detection"
	rowannotation = "row annotation"
	texttoshapegeneration = "text-to-shape generation"
	mammographicbreastpositioningassessment = "mammographic breast positioning assessment"
	abusedetection = "abuse detection"
	math = "math"
	robustfacerecognition = "robust face recognition"
	longtermactionanticipation = "long term action anticipation"
	offlinesurgicalphaserecognition = "offline surgical phase recognition"
	multimodaltextandimageclassification = "multimodal text and image classification"
	gymhalfcheetahmediumreplay = "gym halfcheetah-medium-replay"
	gravitationalwavedetection = "gravitational wave detection"
	eegleftrighthand = "eeg left/right hand"
	multimodalimagesegmentation = "multi-modal image segmentation"
	chinesezeropronounresolution = "chinese zero pronoun resolution"
	visualquestionansweringvqa = "visual question answering (vqa)"
	entity_extraction = "entity_extraction"
	universalsegmentation = "universal segmentation"
	actionrecognition = "action recognition"
	parameterefficientfinetuning = "parameter-efficient fine-tuning"
	humanmeshrecovery = "human mesh recovery"
	medicalxrayimagesegmentation = "medical x-ray image segmentation"
	epistemicreasoning = "epistemic reasoning"
	thompsonsampling = "thompson sampling"
	groupactivityrecognition = "group activity recognition"
	deepfeatureinversion = "deep feature inversion"
	robotnavigation = "robot navigation"
	unsupervisedreinforcementlearning = "unsupervised reinforcement learning"
	motioninterpolation = "motion interpolation"
	portraitsegmentation = "portrait segmentation"
	faultdetection = "fault detection"
	form = "form"
	anomalylocalization = "anomaly localization"
	quantization = "quantization"
	hierarchicalreinforcementlearning = "hierarchical reinforcement learning"
	prognosis = "prognosis"
	quantileregression = "quantile regression"
	autonomousnavigation = "autonomous navigation"
	monocularvisualodometry = "monocular visual odometry"
	reconstructionattack = "reconstruction attack"
	indoorscenesynthesis = "indoor scene synthesis"
	graspgeneration = "grasp generation"
	exemplarfree = "exemplar-free"
	traditionalspamdetection = "traditional spam detection"
	personsearch = "person search"
	sentencefusion = "sentence fusion"
	referringexpression = "referring expression"
	zeroshotmomentretrieval = "zero-shot moment retrieval"
	patchmatching = "patch matching"
	fairness = "fairness"
	proteinsecondarystructureprediction = "protein secondary structure prediction"
	zeroguidancesegmentation = "zero-guidance segmentation"
	selfsupervisedactionrecognition = "self-supervised action recognition"
	blindsourceseparation = "blind source separation"
	unsupervisedkeypoints = "unsupervised keypoints"
	vgsi = "vgsi"
	vulnerabilitydetection = "vulnerability detection"
	wrongbinaryoperator = "wrong binary operator"
	styletransfer = "style transfer"
	motiondisentanglement = "motion disentanglement"
	d3pointcloudinterpolation = "3d point cloud interpolation"
	drsparsing = "drs parsing"
	pillclassification = "pill classification"
	bileveloptimization = "bilevel optimization"
	imagereconstruction = "image reconstruction"
	surgicalgesturerecognition = "surgical gesture recognition"
	brainlesionsegmentationfrommri = "brain lesion segmentation from mri"
	interactiveevaluationofdialog = "interactive evaluation of dialog"
	instancesegmentation = "instance segmentation"
	rllib = "rllib"
	generalizablepersonreidentification = "generalizable person re-identification"
	phonelevelpronunciationscoring = "phone-level pronunciation scoring"
	selfsupervisedanomalydetection = "self-supervised anomaly detection"
	conditionaltexttoimagesynthesis = "conditional text-to-image synthesis"
	datacompression = "data compression"
	facesketchsynthesis = "face sketch synthesis"
	onsetdetection = "onset detection"
	abstentionprediction = "abstention prediction"
	actionanticipation = "action anticipation"
	d3unsuperviseddomainadaptation = "3d unsupervised domain adaptation"
	objectslam = "object slam"
	codesummarization = "code summarization"
	textdetection = "text detection"
	humanitarian = "humanitarian"
	gaitidentification = "gait identification"
	depthimageestimation = "depth image estimation"
	multifocusmicroscopicalimagesfusion = "multi-focus microscopical images fusion"
	pathfinderx = "pathfinder-x"
	videoemotiondetection = "video emotion detection"
	vietnamesemultimodalsentimentanalysis = "vietnamese multimodal sentiment analysis"
	isomorphismtesting = "isomorphism testing"
	objectproposalgeneration = "object proposal generation"
	domain111 = "domain 11-1"
	counterfactualinference = "counterfactual inference"
	columntypeannotation = "column type annotation"
	gsm8k = "gsm8k"
	heartsegmentation = "heart segmentation"
	phrasetagging = "phrase tagging"
	directionalhearing = "directional hearing"
	vietnamesefactchecking = "vietnamese fact checking"
	brainlandmarkdetection = "brain landmark detection"
	surveillancetosurveillance = "surveillance-to-surveillance"
	knowledgebasepopulation = "knowledge base population"
	knowledgegraphembedding = "knowledge graph embedding"
	imagetovideopersonreidentification = "image-to-video person re-identification"
	policygradientmethods = "policy gradient methods"
	videobasedgenerativeperformancebenchmarking = "video-based generative performance benchmarking"
	weaklysupervised3ddetection = "weakly supervised 3d detection"
	depthprediction = "depth prediction"
	personalitytraitrecognition = "personality trait recognition"
	syntheticdatabinarization = "synthetic data binarization"
	unsupervisedpretraining = "unsupervised pre-training"
	persuasivewritingstrategydetectionlevel2 = "persuasive writing strategy detection level-2"
	theoryofmindmodeling = "theory of mind modeling"
	negation = "negation"
	musicstyletransfer = "music style transfer"
	weaklysupervisedactionrecognition = "weakly-supervised action recognition"
	sequentialimageclassification = "sequential image classification"
	keywordextraction = "keyword extraction"
	singleimagebasedhdrreconstruction = "single-image-based hdr reconstruction"
	carlaleaderboard20 = "carla leaderboard 2.0"
	damagedbuildingdetection = "damaged building detection"
	dictionarylearning = "dictionary learning"
	d3scenegraphgeneration = "3d scene graph generation"
	edgecomputing = "edge-computing"
	pointcloudclassification = "point cloud classification"
	semanticcorrespondence = "semantic correspondence"
	zeroshottransferimageclassificationcn = "zero-shot transfer image classification (cn)"
	documentlevelrelationextraction = "document-level relation extraction"
	fewshotobjectdetection = "few-shot object detection"
	gameofgo = "game of go"
	crossmodalalignment = "cross-modal alignment"
	lanelabeling = "lane labeling"
	hallucinationpairwisedetection4ref = "hallucination pair-wise detection (4-ref)"
	videoobjectdetection = "video object detection"
	servicecomposition = "service composition"
	multibehaviorrecommendation = "multibehavior recommendation"
	zeroshot3dpointcloudclassification = "zero-shot 3d point cloud classification"
	hyperedgeprediction = "hyperedge prediction"
	citationworhtiness = "citation worhtiness"
	featureupsampling = "feature upsampling"
	coresetdiscovery = "core set discovery"
	speculationdetection = "speculation detection"
	linkprediction = "link prediction"
	bayesianinference = "bayesian inference"
	lakeicemonitoring = "lake ice monitoring"
	predicatedetection = "predicate detection"
	symbolicregression = "symbolic regression"
	seeingbeyondthevisible = "seeing beyond the visible"
	passagereranking = "passage re-ranking"
	multimodalforgerydetection = "multimodal forgery detection"
	dialogactclassification = "dialog act classification"
	pointcloudsuperresolution = "point cloud super resolution"
	pairtrading = "pair trading"
	intentclassificationandslotfilling = "intent classification and slot filling"
	constrainedlipsynchronization = "constrained lip-synchronization"
	robustdesign = "robust design"
	emotioncausepairextraction = "emotion-cause pair extraction"
	splenomegalysegmentationonmultimodalmri = "splenomegaly segmentation on multi-modal mri"
	germeval2024sharedtask1subtask2 = "germeval2024 shared task 1 subtask 2"
	textindependentspeakerverification = "text-independent speaker verification"
	aspectsentimenttripletextraction = "aspect sentiment triplet extraction"
	sentencesimilarity = "sentence similarity"
	communitysearch = "community search"
	audioeffectsmodeling = "audio effects modeling"
	uccaparsing = "ucca parsing"
	documenttextclassification = "document text classification"
	zeroshotsingleobjecttracking = "zero-shot single object tracking"
	multispectralobjectdetection = "multispectral object detection"
	skillsevaluation = "skills evaluation"
	complexwordidentification = "complex word identification"
	humanactivityrecognition = "human activity recognition"
	weaklysupervisedlearning = "weakly-supervised learning"
	assetmanagement = "asset management"
	diffusionpersonalizationtuningfree = "diffusion personalization tuning free"
	fullreferenceimagequalityassessment = "full-reference image quality assessment"
	phrasevectorembedding = "phrase vector embedding"
	robustobjectdetection = "robust object detection"
	videobackgroundsubtraction = "video background subtraction"
	tar = "tar"
	codeswitchedtranslationtoara = "code-switched translation to ara"
	multiclassoneshotimagesynthesis = "multi class one-shot image synthesis"
	dynamiclinkprediction = "dynamic link prediction"
	imageattribution = "image attribution"
	contextualembeddingforsourcecode = "contextual embedding for source code"
	taskcompletiondialoguepolicylearning = "task-completion dialogue policy learning"
	callerdetection = "caller detection"
	structureddataclassification = "structured data classification"
	fetchpushv1 = "fetchpush-v1"
	linguisticsteganography = "linguistic steganography"
	nonhomogeneousimagedehazing = "nonhomogeneous image dehazing"
	proteinlanguagemodel = "protein language model"
	categoryagnosticposeestimation = "category-agnostic pose estimation"
	emailthreadsummarization = "email thread summarization"
	multiagentintegration = "multi-agent integration"
	virtualstaining = "virtual staining"
	inductivelinkprediction = "inductive link prediction"
	zeroshotlongvideoglobalmodelquestionanswering = "zero-shot long video global-model question answering"
	texteffectstransfer = "text effects transfer"
	lungnoduleclassification = "lung nodule classification"
	videoscenegraphgeneration = "video scene graph generation"
	unsupervisedtextsummarization = "unsupervised text summarization"
	mixtureofexperts = "mixture-of-experts"
	lifelonglearning = "lifelong learning"
	finegrainedvisualrecognition = "fine-grained visual recognition"
	medicalquestionpairsimilaritycomputation = "medical question pair similarity computation"
	bokeheffectrendering = "bokeh effect rendering"
	multilabelzeroshotlearning = "multi-label zero-shot learning"
	spanextractionmrc = "span-extraction mrc"
	d3scenereconstruction = "3d scene reconstruction"
	commonsensereasoning = "common sense reasoning"
	ontheflysketchbasedimageretrieval = "on-the-fly sketch based image retrieval"
	vowelclassification = "vowel classification"
	documentembedding = "document embedding"
	interestpointdetection = "interest point detection"
	timedial = "timedial"
	streetsceneparsing = "street scene parsing"
	personcentricknowledgegraphs = "person-centric knowledge graphs"
	biasdetection = "bias detection"
	decipherment = "decipherment"
	longtaillearningwithclassdescriptors = "long-tail learning with class descriptors"
	facemodel = "face model"
	llmgeneratedtextdetection = "llm-generated text detection"
	filmremoval = "film removal"
	multimodalsubspaceclustering = "multi-modal subspace clustering"
	systematicliteraturereview = "systematic literature review"
	semisupervisedpersonreidentification = "semi-supervised person re-identification"
	personretrieval = "person retrieval"
	stockpriceprediction = "stock price prediction"
	artistclassification = "artist classification"
	eegbasedsleepstaging = "eeg based sleep staging"
	instanceshadowdetection = "instance shadow detection"
	videochaptering = "video chaptering"
	snesgames = "snes games"
	nethack = "nethack"

In [9]:
class DeviceClass(str, Enum):
	server = "server"
	desktop = "desktop"
	laptop = "laptop"
	mobile = "mobile"
	embedded = "embedded"

class DatasetSplit(str, Enum):
	train = "train"
	validation = "validation"
	test = "test"

class Result(BaseModel):
	task: Optional[Task]
	model_name: Optional[str]
	model_architecture: Optional[str]
	parameter_count: Optional[int]
	metric: str
	metric_higher_is_better: Optional[bool]
	value: float
	value_error: Optional[float]
	dataset: Optional[str]
	dataset_version: Optional[str]
	dataset_split: Optional[DatasetSplit]
	inference_time: Optional[float]
	inference_time_unit: Optional[str]
	inference_device_class: Optional[DeviceClass]

class Results(BaseModel):
	results: list[Result]

In [25]:
def extract_results(inputs):
	filename, sample = inputs

	try:
		response = client.responses.parse(
			model="gpt-5-mini",
			reasoning={"effort": "minimal"},
			text={"verbosity": "low"},
			input=[
				{
					"role": "system",
					"content": "You are an expert at structured data extraction. You will be given unstructured text from a research paper and should extract the paper's results into the given structure. Extract an array of results mentioned in the text (one or many). Each result's struct fields should contain minimal information and strictly adhere to the type."
				},
				{
					"role": "user",
					"content": sample
				}
			],
			text_format=Results
		)
		output = response.output_parsed.results
	except Exception as e:
		print(e)
		sleep(5)
		return extract_results(inputs)

	if len(output) == 0:
		return None

	results = []
	for result in output:
		results.append({
			"task": result.task,
			"model_name": result.model_name,
			"model_architecture": result.model_architecture,
			"parameter_count": result.parameter_count,
			"metric": result.metric,
			"metric_higher_is_better": result.metric_higher_is_better,
			"value": result.value,
			"value_error": result.value_error,
			"dataset": result.dataset,
			"dataset_version": result.dataset_version,
			"dataset_split": result.dataset_split,
			"inference_time": result.inference_time,
			"inference_time_unit": result.inference_time_unit,
			"inference_device_class": result.inference_device_class
		})

	return {
		"filename": filename,
		"results": results,
		"text": sample
	}

with ThreadPoolExecutor(max_workers=64) as executor:
	trainData = list(tqdm(executor.map(extract_results, samples), total=len(samples)))

100%|██████████| 5261/5261 [43:45<00:00,  2.00it/s]  


In [ ]:
with open("train_data.json", "w") as f:
	dump([e for e in trainData if e is not None], f, indent=2)